<a href="https://colab.research.google.com/github/Facunditos/PanAndCake/blob/master/TP_II_Resoluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación de librerías

In [ ]:
%%capture
! pip install selenium
! pip install langdetect
! pip install import os
import gdown
! pip install tensorflow tensorflow-text
! pip install pydub
! pip install SpeechRecognition
! pip install PyPDF2
! pip install chromadb
! pip install python-decouple
! pip install deep-translator
!pip install rdflib
!python -m spacy download es_core_news_sm
!pip install llama-index-llms-ollama
!pip install wikipedia
!pip install litellm[proxy]
!pip install llama-index
!pip install rank-bm25
!pip install nltk
!apt-get update
!pip install selenium webdriver-manager

# Importación de librerías

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
from langdetect import detect
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import rdflib
import urllib.parse
from rdflib import Namespace
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import sentencepiece
from pydub import AudioSegment
import speech_recognition as sr
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
import chromadb
from chromadb.config import Settings
import requests
import json
from decouple import config, Config, Csv
from jinja2 import Template
import pandas as pd
from deep_translator import GoogleTranslator
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from huggingface_hub import InferenceClient
from typing import List, Dict, Any
import spacy
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
import wikipedia
from chromadb.config import Settings
from llama_index.core.agent.react.formatter import ReActChatFormatter
from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
nltk.data.path.append('/root/nltk_data/tokenizers/punkt')
import ast
import pickle
from typing import List, Dict, Any
import os
import gdown
from google.colab import userdata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/tecnicatura_ia/procesamiento_lenguaje_natural/TP_II/'

Mounted at /content/drive


# Parte 1 : Base de datos de grafos

La base de datos de grafos se utiliza para modelar datos acerca del juego que no tienen que ver de manera estricta con su jugabilidad y que no son de amplia explicación, sino más bien relaciones entre distintas personas, otros juegos, premios y nacionalidades. Incluye:

Nombres alternativos.
Diseñadores: nombres, nacionalidad, relaciones entre ellos, premios ganados, otros juegos hechos.
Artistas: nombre, nacionalidad, relación con diseñadores, premios, otros juegos.
Publicadores: nombres, nacionalidad.
Juegos de su colección / familia: nombres, relación con diseñadores y artistas del mismo juego.

## Extracción de la información

In [ ]:
## Url de partida
url_credits = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges/credits"

In [ ]:
def encontrar_url(dato:str)->list:

    '''
    Descripción:
    La función `encontrar_url` extrae todas las URLs relevantes de un tipo específico (diseñadores, artistas, editores, etc.)
    desde la página de créditos de un juego en el sitio web BoardGameGeek. Utiliza Selenium para manejar la carga dinámica
    de contenido en la página.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - dato (str): Cadena que especifica el tipo de enlace a buscar.
      Valores comunes incluyen 'designer', 'artist', 'publisher', o 'family'.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - list: Lista de cadenas que representan las URLs encontradas en la página web para el tipo de enlace especificado.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Configura Selenium con opciones específicas para ejecutar Chrome en modo headless.
      - Incluye configuraciones como ejecución sin interfaz gráfica, deshabilitación de GPU, y optimización para entornos de servidor.
    2. Define el controlador (driver) de Chrome utilizando la ruta del ejecutable de ChromeDriver.
    3. Accede a la página web de referencia:
      - En este caso, `https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges/credits`.
    4. Utiliza el parámetro `dato` para construir un selector CSS que busca enlaces específicos en la página.
      - Por ejemplo, si `dato = 'designer'`, se buscarán enlaces relacionados con diseñadores.
    5. Encuentra todos los elementos que coincidan con el selector CSS y extrae los valores del atributo `href`.
    6. Cierra el navegador y devuelve la lista de URLs extraídas.
    '''

    # Como tienen carga con JavaScript utilizo selenuim.
    # Configuración de Selenium
    # Configuración más detallada de las opciones de Chrome
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    # Configurar la ruta del driver
    service = Service('/usr/bin/chromedriver')

    driver = webdriver.Chrome(options=chrome_options)


    # URL del sitio web
    url = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges/credits"
    driver.get(url)
    url = []
    finder = f'a.ng-binding[ng-href*="/boardgame{dato}/"]'
    links = driver.find_elements(By.CSS_SELECTOR, finder)
    # Extraer los href
    url = [link.get_attribute('href') for link in links]
    driver.close()
    return url

url_designers = encontrar_url('designer')
url_artists = encontrar_url('artist')
url_publishers = encontrar_url('publisher')
url_related_to = encontrar_url('family')

In [ ]:
print(url_designers,url_artists ,url_publishers ,url_related_to )


['https://boardgamegeek.com/boardgamedesigner/6940/inka-brand', 'https://boardgamegeek.com/boardgamedesigner/6941/markus-brand'] ['https://boardgamegeek.com/boardgameartist/12484/dennis-lohausen'] ['https://boardgamegeek.com/boardgamepublisher/3490/huch', 'https://boardgamegeek.com/boardgamepublisher/267/999-games', 'https://boardgamegeek.com/boardgamepublisher/2366/devir', 'https://boardgamegeek.com/boardgamepublisher/46544/dice-realm', 'https://boardgamegeek.com/boardgamepublisher/917/dv-games', 'https://boardgamegeek.com/boardgamepublisher/7496/egmont-polska', 'https://boardgamegeek.com/boardgamepublisher/37681/fabrika-igr', 'https://boardgamegeek.com/boardgamepublisher/12540/game-harbor', 'https://boardgamegeek.com/boardgamepublisher/17130/hot-games', 'https://boardgamegeek.com/boardgamepublisher/52507/nostalgia-iii', 'https://boardgamegeek.com/boardgamepublisher/105/r-and-r-games'] ['https://boardgamegeek.com/boardgamefamily/22783/admin-better-description-needed', 'https://boardga

Encontrar informacion dentro de cada link
Defino información que quiero encontrar según cada link:

Designer: Nombre, Pareja(si la tiene), Premios ganados, Otros juegos, nacionalidad.

Artist: Nombre, Pareja(si la tiene), Premios ganados, Otros juegos, nacionalidad.

Family: temática con las que se relaciona el juego.
Publishers: Nombre.

Nombres Alternativos: Nombre, idioma.

Año de publicación.

Premios: nombre de premio ganado, año.

In [ ]:
def encontrar_datos(entidad, lista_url)->dict:
    '''
    Descripción:
    La función `encontrar_datos` recopila información específica (como diseñadores, artistas, editores, familias relacionadas,
    nombres alternativos o premios) de URLs proporcionadas asociadas a un juego de mesa en el sitio web BoardGameGeek.
    La función utiliza Selenium para manejar la carga dinámica del contenido.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - entidad (str): Tipo de información a buscar. Los valores posibles son:
      - 'designer': Diseñadores de juegos.
      - 'artist': Artistas asociados al juego.
      - 'publisher': Editoriales del juego.
      - 'related_to': Familias relacionadas con el juego.
      - 'alternative_names': Nombres alternativos del juego.
      - 'award_game': Premios asociados al juego.
    - lista_url (list): Lista de URLs donde se buscará la información. Para 'alternative_names' y 'award_game', se espera una única URL.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - dict: Lista de diccionarios que contienen la información recopilada de cada URL, dependiendo de la entidad solicitada.
      Si ocurre un error, puede devolver `None`.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Configura Selenium con opciones específicas para ejecutar Chrome en modo headless.
    2. Define el controlador (driver) de Chrome utilizando la ruta del ejecutable de ChromeDriver.
    3. Según el valor de `entidad`, realiza las siguientes acciones:
      - **'designer' o 'artist':**
        - Extrae el nombre del diseñador/artista, su compañero frecuente (*partner*), premios obtenidos y otros juegos relacionados.
      - **'publisher':**
        - Obtiene el nombre de la editorial.
      - **'related_to':**
        - Divide la información de las familias relacionadas en dos partes: temática y nombre.
      - **'alternative_names':**
        - Encuentra todos los nombres alternativos del juego.
      - **'award_game':**
        - Recopila los premios obtenidos por el juego, separando el año y el nombre del premio.
    4. Si la información requiere navegación adicional (por ejemplo, otros juegos relacionados o nombres alternativos),
      realiza la navegación y extrae los datos correspondientes.
    5. Cierra el navegador una vez que se ha recopilado toda la información.
    6. Devuelve una lista de diccionarios con la información obtenida.
    '''

    lista_dict = []
    # Como tienen carga con JavaScript utilizo selenuim.
    # Configuración de Selenium
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    # Configurar la ruta del driver
    service = Service('/usr/bin/chromedriver')

    driver = webdriver.Chrome(options=chrome_options)

    if entidad == 'designer' or entidad == 'artist':

        for url in lista_url:
            driver.get(url)
            finder = f'a.ng-binding[ng-href*="/boardgame{entidad}/"]'
            links = driver.find_element(By.CSS_SELECTOR, finder)
            name_designer = links.text.strip()
            # Esperar hasta que la imagen esté cargada
            WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, "//img[contains(@alt, 'Microbadge:')]")))
            # Encontrar todas las etiquetas <img> con Microbadge
            img_elements = driver.find_elements(By.XPATH, "//img[contains(@alt, 'Microbadge:')]")

            # Verificar si hay más de un microbadge
            if len(img_elements) > 1:
                # Tomar el segundo microbadge (índice 1)
                second_img = img_elements[1]
                # Extraer el atributo 'alt'
                alt_text = second_img.get_attribute("alt")
                # Limpiar el texto para obtener solo el nombre
                partner = alt_text.replace("Microbadge: ", "").replace(" fan", "").strip()
            else:
                partner = ''
            # Buscar el enlace que contiene el nombre del premio
            try:
                award = driver.find_element(By.CSS_SELECTOR, 'a.ng-binding[ng-href*="/boardgamehonor/"]')
                award = award.text.strip()  # Elimina espacios extra al inicio y fin
            except:
                award = ''


            see_all_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'See All')]"))
            )
            # Extraer el enlace del botón
            see_all_link = see_all_button.get_attribute("href")

             # Navegar al enlace extraído
            driver.get(see_all_link)

            # Esperar hasta que los nombres estén cargados
            WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.XPATH, "//h3[@class='summary-item-title summary-item-title-separated fs-inherit mb-0']/a"))
            )

            # Extraer los primeros 10 nombres
            name_other_games = driver.find_elements(By.XPATH, "//h3[@class='summary-item-title summary-item-title-separated fs-inherit mb-0']/a")[:10]
            other_games = [name.text.strip() for name in name_other_games]

            dict = {'Name': name_designer, 'Partner': partner, 'Award': award, 'Other Games': other_games}
            lista_dict.append(dict)

    elif entidad == 'publisher':

        for url in lista_url:
            driver.get(url)
            finder = f'a.ng-binding[ng-href*="/boardgame{entidad}/"]'
            links = driver.find_element(By.CSS_SELECTOR, finder)
            name_publisher = links.text.strip()
            dict = {'Name': name_publisher}
            lista_dict.append(dict)

    elif entidad == 'related_to':
        for url in lista_url:
            driver.get(url)
            finder = f'a.ng-binding[ng-href*="/boardgamefamily/"]'
            links = driver.find_element(By.CSS_SELECTOR, finder)
            subject = links.text.strip()
            # Dividir en temática y nombre
            if ":" in subject:
                partes = subject.split(":", 1)  # Dividir solo en el primer ":"
                tematica = partes[0].strip()  # Eliminar espacios en exceso
                nombre = partes[1].strip()  # Eliminar espacios en exceso
            else:
                tematica = subject.strip()
                nombre = ""  # Si no hay ":", dejar el nombre vacío

            # Crear el diccionario
            dict = {'Subject': tematica, 'Name': nombre}
            lista_dict.append(dict)
    elif entidad == 'alternative_names':
        driver.get(lista_url)
        try:
            # Esperar hasta que los nombres alternativos estén cargados
            WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.XPATH, "//div[@ng-switch-when='alternatename']"))
            )

            # Encontrar todos los nombres alternativos
            alt_name_elements = driver.find_elements(By.XPATH, "//div[@ng-switch-when='alternatename']")
            alt_names = [element.text.strip() for element in alt_name_elements if element.text.strip()]

            for name in alt_names:
                dict = {'Alternative Name' : name}
                lista_dict.append(dict)
        except:
            lista_dict = None


        return lista_dict
    elif entidad == 'award_game':
        driver.get(lista_url)
        try:
            # URL inicial
            url = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges"
            driver.get(url)

            # Esperar a que los elementos <a> con premios estén cargados
            WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.XPATH, "//a[@class='ng-binding']"))
            )

            # Encontrar todos los premios
            awards_elements = driver.find_elements(By.XPATH, "//a[@class='ng-binding'][contains(@ng-href, '/boardgamehonor/')]")

            # Lista para almacenar los premios procesados
            awards_list = []

            for award in awards_elements:
                text = award.text.strip()
                # Separar el año del nombre del premio usando una expresión regular
                match = re.match(r"(\d{4})\s*(.*)", text)
                if match:
                    year = match.group(1)
                    name = match.group(2).strip()
                    awards_list.append({'year': year, 'name': name})
            for award in awards_list:
                dict = {'Prize': award['name'], 'Year': award['year']}
                lista_dict.append(dict)
        except:
            lista_dict = None
    driver.close()
    return lista_dict

In [ ]:
dict_desingers =  encontrar_datos('designer', url_designers)
dict_artist = encontrar_datos('artist', url_artists)
dict_publishers = encontrar_datos('publisher', url_publishers)
dict_related_to = encontrar_datos('related_to', url_related_to)
dict_alternative_names = encontrar_datos('alternative_names', url_credits)
dict_awards = encontrar_datos('award_game', url_credits)

In [ ]:
for designer in dict_desingers:
    designer['Nacionalidad'] = 'German'
for artist in dict_artist:
    artist['Nacionalidad'] = 'German'

In [ ]:
dict_desingers

[{'Name': 'Inka Brand',
  'Partner': 'Markus Brand',
  'Award': '2012 Spiel des Jahres Kennerspiel des Jahres Winner',
  'Other Games': ['Andor Junior: Die Fährtenleserin Fennah / Der Fährtenleser Fenn (2020)',
   'Andor: The Family Fantasy Game (2020)',
   'Andor: The Family Fantasy Game – The Danger in the Shadows (2022)',
   'Bibi & Tina: Das Spiel zum Film (2013)',
   'Bitte nicht öffnen: Bissig! (2022)',
   'Black Stories: Das Spiel (2022)',
   'Blitzed! (2013)',
   'La Boca (2013)',
   'Bravo! (2019)',
   'Burgenland (2014)'],
  'Nacionalidad': 'German'},
 {'Name': 'Markus Brand',
  'Partner': 'Inka Brand',
  'Award': '2012 Spiel des Jahres Kennerspiel des Jahres Winner',
  'Other Games': ['Andor Junior: Die Fährtenleserin Fennah / Der Fährtenleser Fenn (2020)',
   'Andor: The Family Fantasy Game (2020)',
   'Andor: The Family Fantasy Game – The Danger in the Shadows (2022)',
   'Bibi & Tina: Das Spiel zum Film (2013)',
   'Bitte nicht öffnen: Bissig! (2022)',
   'Black Stories: 

In [ ]:
dict_desingers = [{'Name': 'Inka Brand',
  'Partner': 'Markus Brand',
  'Award': '2012 Spiel des Jahres Kennerspiel des Jahres Winner',
  'Other Games': ['Andor Junior: Die Fährtenleserin Fennah / Der Fährtenleser Fenn (2020)',
   'Andor: The Family Fantasy Game (2020)',
   'Andor: The Family Fantasy Game – The Danger in the Shadows (2022)',
   'Bibi & Tina: Das Spiel zum Film (2013)',
   'Bitte nicht öffnen: Bissig! (2022)',
   'Black Stories: Das Spiel (2022)',
   'Blitzed! (2013)',
   'La Boca (2013)',
   'Bravo! (2019)',
   'Burgenland (2014)'],
  'Nacionalidad': 'German'},
 {'Name': 'Markus Brand',
  'Partner': 'Inka Brand',
  'Award': '2012 Spiel des Jahres Kennerspiel des Jahres Winner',
  'Other Games': ['Andor Junior: Die Fährtenleserin Fennah / Der Fährtenleser Fenn (2020)',
   'Andor: The Family Fantasy Game (2020)',
   'Andor: The Family Fantasy Game – The Danger in the Shadows (2022)',
   'Bibi & Tina: Das Spiel zum Film (2013)',
   'Bitte nicht öffnen: Bissig! (2022)',
   'Black Stories: Das Spiel (2022)',
   'Blitzed! (2013)',
   'La Boca (2013)',
   'Bravo! (2019)',
   'Burgenland (2014)'],
  'Nacionalidad': 'German'}]

In [ ]:
dict_artist

[{'Name': 'Dennis Lohausen',
  'Partner': '',
  'Award': '',
  'Other Games': ['100! (2014)',
   '1001 Karawane (2007)',
   '1001 Karawane: Sonderchips (2007)',
   '112: Brandgefährlich (2017)',
   '15 (2011)',
   'The Academy (2023)',
   'Adventure Land (2023)',
   'Agent DOG (2024)',
   'Albion (2009)',
   'All Change! (and Chameleon) (2017)'],
  'Nacionalidad': 'German'}]

In [ ]:
dict_artist = [{'Name': 'Dennis Lohausen',
  'Partner': '',
  'Award': '',
  'Other Games': ['100! (2014)',
   '1001 Karawane (2007)',
   '1001 Karawane: Sonderchips (2007)',
   '112: Brandgefährlich (2017)',
   '15 (2011)',
   'The Academy (2023)',
   'Adventure Land (2023)',
   'Agent DOG (2024)',
   'Albion (2009)',
   'All Change! (and Chameleon) (2017)'],
  'Nacionalidad': 'German'}]

In [ ]:
dict_publishers

[{'Name': 'HUCH!'},
 {'Name': '999 Games'},
 {'Name': 'Devir'},
 {'Name': 'Dice Realm'},
 {'Name': 'DV Games'},
 {'Name': 'Egmont Polska'},
 {'Name': 'Fabrika Igr'},
 {'Name': 'Game Harbor'},
 {'Name': 'HOT Games'},
 {'Name': 'nostalgia (III)'},
 {'Name': 'R&R Games'}]

In [ ]:
dict_related_to

[{'Subject': 'Admin', 'Name': 'Better Description Needed!'},
 {'Subject': 'Country', 'Name': 'India'},
 {'Subject': 'Digital Implementations', 'Name': 'Tabletopia'},
 {'Subject': 'Digital Implementations', 'Name': 'Yucata'},
 {'Subject': 'Game', 'Name': 'Rajas of the Ganges'},
 {'Subject': 'Mechanism', 'Name': 'Dice Drafting'},
 {'Subject': 'Religious', 'Name': 'Hinduism'}]

In [ ]:
dict_alternative_names

[{'Alternative Name': 'Ganges'},
 {'Alternative Name': 'I Ragià del Gange'},
 {'Alternative Name': "Raja's van de Ganges"},
 {'Alternative Name': 'Раджи Ганга'},
 {'Alternative Name': 'ガンジスの藩王'},
 {'Alternative Name': '恆河王侯'},
 {'Alternative Name': '갠지스의 라자'}]

In [ ]:
dict_alternative_names = [{'Alternative Name': 'Ganges'},
 {'Alternative Name': 'I Ragià del Gange'},
 {'Alternative Name': "Raja's van de Ganges"},
 {'Alternative Name': 'Раджи Ганга'},
 {'Alternative Name': 'ガンジスの藩王'},
 {'Alternative Name': '갠지스의 라자'}]

In [ ]:
dict_awards

[{'Prize': 'MinD-Spielepreis Complex Game Nominee', 'Year': '2019'},
 {'Prize': 'Juego del Año Recommended', 'Year': '2018'},
 {'Prize': 'International Gamers Award - General Strategy: Multi-player Winner',
  'Year': '2018'},
 {'Prize': 'International Gamers Award - General Strategy: Multi-player Nominee',
  'Year': '2018'}]

In [ ]:
dict_awards = [{'Prize': 'MinD-Spielepreis Complex Game Nominee', 'Year': '2019'},
 {'Prize': 'Juego del Año Recommended', 'Year': '2018'},
 {'Prize': 'International Gamers Award - General Strategy: Multi-player Winner',
  'Year': '2018'},
 {'Prize': 'International Gamers Award - General Strategy: Multi-player Nominee',
  'Year': '2018'}]


In [ ]:
# Detección de idioma de los nombres alternativos
for alt_name in dict_alternative_names:
    lang = detect(alt_name['Alternative Name'])
    alt_name['Language'] = lang

In [ ]:
dict_alternative_names

[{'Alternative Name': 'Ganges', 'Language': 'de'},
 {'Alternative Name': 'I Ragià del Gange', 'Language': 'ca'},
 {'Alternative Name': "Raja's van de Ganges", 'Language': 'nl'},
 {'Alternative Name': 'Раджи Ганга', 'Language': 'bg'},
 {'Alternative Name': 'ガンジスの藩王', 'Language': 'ja'},
 {'Alternative Name': '갠지스의 라자', 'Language': 'ko'}]

## Armado de base de datos

Crear el grafo base

In [ ]:
G = nx.DiGraph()
game_node = "Rajas of the Ganges"
G.add_node(game_node, type="game")

# Añadimos nodos de diseñadores y establecemos relación con el juego
for designer in dict_desingers:
    G.add_node(designer['Name'], type="designer", nationality=designer['Nacionalidad'])
    G.add_edge(game_node, designer['Name'], relationship="designed by")
    if designer['Partner']:
        G.add_edge(designer['Name'], designer['Partner'], relationship="partner")

    # Añadir otros juegos que diseñaron
    for other_game in designer['Other Games']:
        G.add_node(other_game, type="game")
        G.add_edge(designer['Name'], other_game, relationship="designed")

        # Añadir los premios de esos otros juegos, si están disponibles en el diccionario de premios
        for award in dict_awards:  # Premios globales del juego "Rajas of the Ganges"
            if award['Prize'] in other_game:  # Si el premio pertenece a este juego (por ejemplo, por nombre o año)
                award_node = f"{award['Year']} - {award['Prize']}"
                G.add_node(award_node, type="award", year=award['Year'])
                G.add_edge(other_game, award_node, relationship="awarded or nominated")

# Añadimos nodo de artista y establecemos relación con el juego
for artist in dict_artist:
    G.add_node(artist['Name'], type="artist", nationality=artist['Nacionalidad'])
    G.add_edge(game_node, artist['Name'], relationship="art by")

    # Añadir otros juegos que hizo el artista
    for other_game in artist['Other Games']:
        G.add_node(other_game, type="game")
        G.add_edge(artist['Name'], other_game, relationship="artist of")

        # Añadir los premios de esos otros juegos
        for award in dict_awards:  # Premios globales del juego "Rajas of the Ganges"
            if award['Prize'] in other_game:  # Si el premio pertenece a este juego (por ejemplo, por nombre o año)
                award_node = f"{award['Year']} - {award['Prize']}"
                G.add_node(award_node, type="award", year=award['Year'])
                G.add_edge(other_game, award_node, relationship="awarded or nominated")

# Publicadores
for publisher in dict_publishers:
    G.add_node(publisher['Name'], type="publisher")
    G.add_edge(game_node, publisher['Name'], relationship="published by")

# País
country = next((item for item in dict_related_to if item['Subject'] == 'Country'), None)
G.add_node(country['Name'], type="country")
G.add_edge(game_node, country['Name'], relationship="set in")

# Religión con la que se relaciona
religion = next((item for item in dict_related_to if item['Subject'] == 'Religious'), None)
G.add_node(religion['Name'], type="religion")
G.add_edge(game_node, religion['Name'], relationship="religion associated with")

# Nombres alternativos
for alt_name in dict_alternative_names:
    G.add_node(alt_name['Alternative Name'], type="alternative name", language=alt_name['Language'])
    G.add_edge(game_node, alt_name['Alternative Name'], relationship="also known as")

# Premios, honores o nominaciones
for award in dict_awards:
    award_node = f"{award['Year']} - {award['Prize']}"
    G.add_node(award_node, type="award", year=award['Year'])
    G.add_edge(game_node, award_node, relationship="awarded or nominated")

#Año
release_year_node = 2017
G.add_node(release_year_node, type = 'year')
G.add_edge(game_node, release_year_node, relationship = "Release year")

In [ ]:
# Generar posiciones de los nodos con un layout de NetworkX
pos = nx.spring_layout(G, seed=42)  # Layout basado en fuerzas para distribuir los nodos

# Extraer las coordenadas de los nodos
x_nodes = [pos[node][0] for node in G.nodes]
y_nodes = [pos[node][1] for node in G.nodes]

# Extraer las relaciones (aristas)
edge_x = []
edge_y = []
edge_labels = []  # Aquí guardaremos las etiquetas de las aristas
edge_midpoints = []  # Para almacenar las posiciones medias de las aristas
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_y.append(y0)
    edge_y.append(y1)

    # Guardamos la relación como etiqueta
    edge_labels.append(G[edge[0]][edge[1]]['relationship'])

    # Calculamos los puntos medios de las aristas
    mid_x = (x0 + x1) / 2
    mid_y = (y0 + y1) / 2
    edge_midpoints.append((mid_x, mid_y))

# Crear el gráfico interactivo con Plotly
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='gray'),
    hoverinfo='none', mode='lines')

node_trace = go.Scatter(
    x=x_nodes, y=y_nodes,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=20,
        colorbar=dict(thickness=15, title='Node Connections', xanchor='left', titleside='right')
    ))

# Añadir información sobre los nodos (nombre, tipo, etc.)
node_text = []
for node in G.nodes():
    node_info = f'{node} ({G.nodes[node].get("type", "unknown")})'
    node_text.append(node_info)

node_trace.marker.color = np.arange(len(G.nodes))  # Colores por nodo
node_trace.text = node_text  # Información que aparece al pasar el ratón por encima

# Añadir las etiquetas de las relaciones (aristas) en el punto medio de cada arista
edge_trace_text = go.Scatter(
    x=[mid[0] for mid in edge_midpoints], y=[mid[1] for mid in edge_midpoints],
    mode='text',
    text=edge_labels,
    textposition='middle center',
    hoverinfo='none',
    showlegend=False,
    textfont=dict(size=10, color="black", family="Arial")
)

# Crear el layout interactivo
layout = go.Layout(
    title='Interactive Network Graph of Rajas of the Ganges',
    titlefont_size=16,
    showlegend=False,
    hovermode='closest',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    plot_bgcolor='white'
)
# Añadir las etiquetas de los nodos (nombre de la entidad) sobre los nodos
node_trace_text = go.Scatter(
    x=x_nodes, y=[y + 0.05 for y in y_nodes],  # Ajuste de posición para que el texto esté encima de los nodos
    mode='text',
    text=node_text,
    textposition='bottom center',  # Asegura que el texto esté por encima del nodo
    hoverinfo='none',
    showlegend=False,
    textfont=dict(size=10, color="black", family="Arial")
)

# Crear la figura de Plotly con los cambios
fig = go.Figure(data=[edge_trace, node_trace, edge_trace_text, node_trace_text], layout=layout)

# Mostrar el gráfico interactivo
fig.show()

Crear un grafo RDF

In [ ]:
# Crear un grafo RDF vacío
g = rdflib.Graph()

# Definir los prefijos
ex = rdflib.Namespace("http://wiki_rajas.org/")
g.bind("ex", ex)

# Función para codificar los nombres a formato URI válido
def encode_to_uri(name):
    return urllib.parse.quote(name.replace(" ", "_"))

# Crear el nodo principal (el juego)
game_node = encode_to_uri("Rajas of the Ganges")
g.add((ex[game_node], rdflib.RDF.type, ex.Game))

# Añadir los diseñadores y sus relaciones con el juego
for designer in dict_desingers:
    designer_node = encode_to_uri(designer['Name'])
    g.add((ex[designer_node], rdflib.RDF.type, ex.Designer))
    g.add((ex[game_node], ex.designedBy, ex[designer_node]))  # Relación: juego -> diseñador

    if designer['Partner']:
        partner_node = encode_to_uri(designer['Partner'])
        g.add((ex[designer_node], ex.partner, ex[partner_node]))  # Relación: diseñador -> pareja

# Añadir los artistas y sus relaciones con el juego
for artist in dict_artist:
    artist_node = encode_to_uri(artist['Name'])
    g.add((ex[artist_node], rdflib.RDF.type, ex.Artist))
    g.add((ex[game_node], ex.artBy, ex[artist_node]))  # Relación: juego -> artista

# Añadir los publicadores
for publisher in dict_publishers:
    publisher_node = encode_to_uri(publisher['Name'])
    g.add((ex[publisher_node], rdflib.RDF.type, ex.Publisher))
    g.add((ex[game_node], ex.publishedBy, ex[publisher_node]))  # Relación: juego -> publicador

# Añadir el país
country_node = encode_to_uri(country['Name'])
g.add((ex[country_node], rdflib.RDF.type, ex.Country))
g.add((ex[game_node], ex.setIn, ex[country_node]))  # Relación: juego -> país

# Añadir la religión
religion_node = encode_to_uri(religion['Name'])
g.add((ex[religion_node], rdflib.RDF.type, ex.Religion))
g.add((ex[game_node], ex.religionAssociatedWith, ex[religion_node]))  # Relación: juego -> religión

# Añadir los nombres alternativos
for alt_name in dict_alternative_names:
    alt_name_node = encode_to_uri(alt_name['Alternative Name'])
    g.add((ex[alt_name_node], rdflib.RDF.type, ex.AlternativeName))
    g.add((ex[game_node], ex.alsoKnownAs, ex[alt_name_node]))  # Relación: juego -> nombre alternativo

# Añadir premios
for award in dict_awards:
    award_node = encode_to_uri(f"{award['Year']} - {award['Prize']}")
    g.add((ex[award_node], rdflib.RDF.type, ex.Award))
    g.add((ex[game_node], ex.awardedOrNominated, ex[award_node]))  # Relación: juego -> premio

release_year_literal = rdflib.Literal(2017)
g.add((ex[game_node], ex.Release_year, release_year_literal))
g.add((ex[release_year_literal], rdflib.RDF.type, ex.Release_year ))
g.add((ex[game_node], ex.Release_year, ex[release_year_literal]))

#Serializar y exportar el grafo a RDF/XML
rdf_output = g.serialize(format='xml')
print(rdf_output)

# Guardar el RDF en un archivo
with open(drive_path+'bases_datos/graph_db.rdf', "w") as file:
    file.write(rdf_output)

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:ex="http://wiki_rajas.org/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://wiki_rajas.org/Rajas_of_the_Ganges">
    <rdf:type rdf:resource="http://wiki_rajas.org/Game"/>
    <ex:designedBy rdf:resource="http://wiki_rajas.org/Inka_Brand"/>
    <ex:designedBy rdf:resource="http://wiki_rajas.org/Markus_Brand"/>
    <ex:artBy rdf:resource="http://wiki_rajas.org/Dennis_Lohausen"/>
    <ex:publishedBy rdf:resource="http://wiki_rajas.org/HUCH%21"/>
    <ex:publishedBy rdf:resource="http://wiki_rajas.org/999_Games"/>
    <ex:publishedBy rdf:resource="http://wiki_rajas.org/Devir"/>
    <ex:publishedBy rdf:resource="http://wiki_rajas.org/Dice_Realm"/>
    <ex:publishedBy rdf:resource="http://wiki_rajas.org/DV_Games"/>
    <ex:publishedBy rdf:resource="http://wiki_rajas.org/Egmont_Polska"/>
    <ex:publishedBy rdf:resource="http://wiki_rajas.org/Fabrika_Igr"/>
    <ex:publishedBy rdf:resour

In [ ]:

query = """
    SELECT (STRAFTER(STR(?Year), "http://wiki_rajas.org/") AS ?name)
    WHERE {
        ?game a <http://wiki_rajas.org/Game> .
        ?game <http://wiki_rajas.org/Release_year> ?Year .
    }
"""

# Ejecutar consulta y mostrar resultados
results = g.query(query)
for row in results:
    print(row)

(rdflib.term.Literal(''),)
(rdflib.term.Literal('2017'),)


Importación del grafo

In [ ]:
# Cargar el archivo RDF
from rdflib import Graph
rdf_file = drive_path+'bases_datos/graph_db.rdf'  # Reemplaza con la ruta a tu archivo
g = Graph()
# Parsear el archivo para construir el grafo
g.parse(rdf_file, format="xml")  # Cambia el formato si no es RDF/XML

<Graph identifier=Nd290dacf802e4d648e7f9c1bb2b560ce (<class 'rdflib.graph.Graph'>)>

# Parte 2: Base vectorial

La base de datos vectorial tendrá el fin de almacenar:

- Reglas del juego generales.
- Información sobre las casillas.
- Información sobre el objetivo, juego en general.
- Opiniones.

Primero debemos extraer la información.
- Para la información de las reglas Tenemos 3 fuentes de datos:

El documento en español del manual original del juego.
PDF de reglas resumidas tanto para el modo normal como nirvana.
- Información de casillas

El documento en español del manual original del juego.
Información sobre el objetivo, juego en general

El documento en español del manual original del juego.
- Opiniones

Foro de opiniones de usuarios.
Video.

**Fuente: documento en español del manual del juego.**

In [ ]:
def limpiar_texto(texto):
    '''
    Descripción:
    La función `limpiar_texto` procesa una cadena de texto para eliminar caracteres no deseados, reducir espacios múltiples
    y conservar solo caracteres alfanuméricos, algunos signos de puntuación comunes y letras con acentos o diéresis.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - texto (str): Cadena de texto que se desea limpiar.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - str: Cadena de texto procesada, sin caracteres no deseados, con un formato más uniforme y sin espacios redundantes.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Elimina caracteres no codificados en formato Unicode (`uXXXX`) o hexadecimal (`xXX`) utilizando expresiones regulares.
    2. Conserva únicamente caracteres alfanuméricos, letras con acentos, diéresis y signos básicos de puntuación
      (espacios, comas, puntos, punto y coma, signos de exclamación/interrogación, guiones y paréntesis).
    3. Sustituye múltiples espacios consecutivos por un solo espacio.
    4. Quita los espacios al inicio y al final del texto.
    5. Devuelve la cadena procesada.
    '''
    # Eliminar caracteres no codificados (\uXXXX, \xXX, etc.)
    texto = re.sub(r'\\u[0-9a-fA-F]{4}|\\x[0-9a-fA-F]{2}', '', texto)
    # Eliminar caracteres no alfanuméricos (excepto básicos como espacios, puntuación común)
    texto = re.sub(r'[^a-zA-Z0-9áéíóúüñÁÉÍÓÚÜÑ\s.,;!?()-]', '', texto)
    # Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto)
    # Quitar espacios iniciales y finales
    texto = texto.strip()
    return texto

# 1. Extraer texto del PDF

def extract_text_with_pypdf2(pdf_path):
  '''
  Descripción:
  La función `extract_text_with_pypdf2` extrae texto de un archivo PDF utilizando la biblioteca PyPDF2, limpia el texto extraído
  con la función `limpiar_texto` y concatena el contenido de todas las páginas en una sola cadena.

  -------------------------------------------------------------------------------------------------

  Parámetros:
  - pdf_path (str): Ruta del archivo PDF del cual se extraerá el texto.

  -------------------------------------------------------------------------------------------------

  Retorna:
  - str: Cadena que contiene el texto extraído y procesado de todas las páginas del PDF.

  -------------------------------------------------------------------------------------------------

  Procedimiento:
  1. Abre el archivo PDF en modo binario de solo lectura.
  2. Crea un objeto lector (`PdfReader`) para acceder al contenido del PDF.
  3. Itera sobre todas las páginas del PDF:
    - Extrae el texto de la página actual usando `extract_text()`.
    - Procesa el texto extraído con la función `limpiar_texto` para eliminar caracteres no deseados y formatearlo.
    - Sustituye saltos de línea (`\n`) por espacios y concatena el texto procesado a la variable `full_text`.
  4. Cierra el archivo PDF automáticamente al salir del bloque `with`.
  5. Devuelve la cadena completa con el texto extraído y procesado.
  '''

  full_text = ""
  with open(pdf_path, "rb") as pdf_file:
      reader = PyPDF2.PdfReader(pdf_file)
      for page in reader.pages:
          text = page.extract_text()
          text = limpiar_texto(text)
          if text:
              full_text +=  text.replace("\n", " ")
  return full_text

#  Dividir texto en secciones usando los títulos del indice
def split_into_sections_auto(full_text, titles):
  '''
  Descripción:
  La función `split_into_sections_auto` divide un texto completo en secciones basadas en títulos predefinidos,
  asignando a cada sección el contenido que sigue a su respectivo título hasta el siguiente título.

  -------------------------------------------------------------------------------------------------

  Parámetros:
  - full_text (str): Texto completo que se desea dividir en secciones.
  - titles (list): Lista de títulos que se utilizarán como delimitadores para dividir el texto. Cada título corresponde
    a una sección del texto.

  -------------------------------------------------------------------------------------------------

  Retorna:
  - dict: Diccionario donde las claves son los títulos de las secciones y los valores son los fragmentos de texto
    correspondientes a cada sección.

  -------------------------------------------------------------------------------------------------

  Procedimiento:
  1. Inicializa un diccionario `sections` donde las claves son los títulos de las secciones y los valores son cadenas vacías.
  2. Construye una expresión regular que coincida con cualquiera de los títulos proporcionados en la lista `titles`.
  3. Utiliza la función `re.split()` para dividir el texto completo (`full_text`) en fragmentos, usando los títulos como delimitadores.
  4. Itera sobre los fragmentos divididos:
    - Si el fragmento es un título, establece ese título como la sección actual.
    - Si el fragmento pertenece a una sección (es decir, sigue a un título), se agrega al contenido correspondiente de esa sección.
  5. Devuelve el diccionario `sections` con el contenido dividido en función de los títulos.
  '''

  sections = {title: "" for title in titles}
  pattern = "|".join(re.escape(title) for title in titles)
  split_text = re.split(f"({pattern})", full_text)

  current_section = None
  for fragment in split_text:
      fragment = fragment.strip()
      if fragment in titles:
          current_section = fragment
      elif current_section:
          sections[current_section] += fragment + " "
  return sections

In [ ]:

# 2. Procesar el texto en vectores
def process_text_to_vectors(text_segments, model):
    '''
    Descripción:
    La función `process_text_to_vectors` convierte segmentos de texto en vectores de incrustación (embeddings) utilizando un modelo de incrustación de texto.
    Devuelve una lista de diccionarios, donde cada diccionario contiene el texto original y su correspondiente vector de incrustación.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - text_segments (list): Lista de segmentos de texto que se desean convertir en vectores.
    - model (object): Modelo de incrustación de texto que se utiliza para generar los vectores. Este modelo debe tener un método `encode`
      que procese los segmentos de texto y los convierta en vectores.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - list: Lista de diccionarios, donde cada diccionario tiene dos claves:
      - "text": El texto original del segmento.
      - "embedding": El vector de incrustación correspondiente, convertido en una lista.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Utiliza el método `encode` del modelo para convertir los segmentos de texto en vectores de incrustación.
    2. Para cada segmento de texto y su respectivo vector de incrustación, crea un diccionario con dos claves:
      - "text": El texto original.
      - "embedding": El vector de incrustación como una lista.
    3. Devuelve la lista de diccionarios, cada uno conteniendo un segmento de texto y su vector de incrustación correspondiente.
    '''

    embeddings = model.encode(text_segments, convert_to_tensor=False)
    return [{"text": text, "embedding": embedding.tolist()} for text, embedding in zip(text_segments, embeddings)]

## Reglamento

In [ ]:
indices_manual = [
    'OBJETIVO DEL JUEGO',
    'SETUP','TRANSCURSO DEL JUEGO',
    'ACCIÓN CONSTRUIR (CANTERA)',
    'TABLERO DE PROVINCIA',
    'Las mejoras de edificios',
    'ACCIÓN DE MERCADO (MERCADO)',
    'ACCIÓN DE PALACIO',
    'ACCIÓN DE RÍO (Puerto)',
    'Karma',
    'Obtener nuevos trabajadores.',
    'Bonos en los tracks de puntuación.',
    'FIN DE RONDA',
    'FINAL DE PARTIDA',
    'NOTAS',
    'VERSIÓN NAVARATNAS',
    'EL MÓDULO GANGA',
    'MOGUL LEXIGON',
    'CRÉDITOS'
    ]



# 4. Ejecutar el pipeline
pdf_path = drive_path+'datos/Rajas_of_the_Ganjes_Reglamento.pdf'  # Reemplaza con la ruta a tu archivo
# Extraer texto completo del PDF
full_text = extract_text_with_pypdf2(pdf_path)


# Dividir texto en secciones
sections = split_into_sections_auto(full_text, indices_manual)

# Mostrar secciones como ejemplo
for title in indices_manual:
    print(f"Indice: {title}. Largo de seccion: {len(sections[title])}")

Indice: OBJETIVO DEL JUEGO. Largo de seccion: 1054
Indice: SETUP. Largo de seccion: 1727
Indice: TRANSCURSO DEL JUEGO. Largo de seccion: 3199
Indice: ACCIÓN CONSTRUIR (CANTERA). Largo de seccion: 1337
Indice: TABLERO DE PROVINCIA. Largo de seccion: 1441
Indice: Las mejoras de edificios. Largo de seccion: 2481
Indice: ACCIÓN DE MERCADO (MERCADO). Largo de seccion: 1508
Indice: ACCIÓN DE PALACIO. Largo de seccion: 2069
Indice: ACCIÓN DE RÍO (Puerto). Largo de seccion: 1564
Indice: Karma. Largo de seccion: 7770
Indice: Obtener nuevos trabajadores.. Largo de seccion: 1486
Indice: Bonos en los tracks de puntuación.. Largo de seccion: 2208
Indice: FIN DE RONDA. Largo de seccion: 1
Indice: FINAL DE PARTIDA. Largo de seccion: 1
Indice: NOTAS. Largo de seccion: 5346
Indice: VERSIÓN NAVARATNAS. Largo de seccion: 0
Indice: EL MÓDULO GANGA. Largo de seccion: 4006
Indice: MOGUL LEXIGON. Largo de seccion: 1
Indice: CRÉDITOS. Largo de seccion: 546


In [ ]:
sections['FINAL DE PARTIDA'] = sections['NOTAS'].split('es el ganador de la partida.')[0]+'es el ganador de la partida.'
sections['NOTAS'] = sections['NOTAS'].split('es el ganador de la partida.')[1]
sections['FIN DE RONDA'] ='Después de haber colocado a todos tus trabajadores activos'+sections['Bonos en los tracks de puntuación.'].split('Después de haber colocado a todos tus trabajadores activos')[1].replace('\n', '')
sections['Bonos en los tracks de puntuación.'] = sections['Bonos en los tracks de puntuación.'].split('Después de haber colocado a todos tus trabajadores activos')[0]
sections['VERSIÓN NAVARATNAS'] = 'Después de haber jugado a la versión básica'+sections['NOTAS'].split('Después de haber jugado a la versión básica')[1]
sections['NOTAS'] = ''

In [ ]:
# Mostrar secciones como ejemplo
for title, text in sections.items():
    print(f"Indice: {title}. Largo de seccion: {len(text)}")

Indice: OBJETIVO DEL JUEGO. Largo de seccion: 1054
Indice: SETUP. Largo de seccion: 1727
Indice: TRANSCURSO DEL JUEGO. Largo de seccion: 3199
Indice: ACCIÓN CONSTRUIR (CANTERA). Largo de seccion: 1337
Indice: TABLERO DE PROVINCIA. Largo de seccion: 1441
Indice: Las mejoras de edificios. Largo de seccion: 2481
Indice: ACCIÓN DE MERCADO (MERCADO). Largo de seccion: 1508
Indice: ACCIÓN DE PALACIO. Largo de seccion: 2069
Indice: ACCIÓN DE RÍO (Puerto). Largo de seccion: 1564
Indice: Karma. Largo de seccion: 7770
Indice: Obtener nuevos trabajadores.. Largo de seccion: 1486
Indice: Bonos en los tracks de puntuación.. Largo de seccion: 1635
Indice: FIN DE RONDA. Largo de seccion: 573
Indice: FINAL DE PARTIDA. Largo de seccion: 2513
Indice: NOTAS. Largo de seccion: 0
Indice: VERSIÓN NAVARATNAS. Largo de seccion: 2760
Indice: EL MÓDULO GANGA. Largo de seccion: 4006
Indice: MOGUL LEXIGON. Largo de seccion: 1
Indice: CRÉDITOS. Largo de seccion: 546


## Referencia rápida

In [ ]:
ref_rapida_path = drive_path+"datos/Rajas_of_the_Ganjes_Referencia_Rapida.pdf"
text_ref_rapida = extract_text_with_pypdf2(ref_rapida_path)

In [ ]:
print(text_ref_rapida)

GANGES (2 a 4 jugadores) Antes de empezar Decidir si se juega en modo normal o en modo Navaratna (ver el epígrafe Variante Navaratna más adelante ). Sortear jugador inicial , el cual se va rotando en sentido horario cada vez que se agoten los lacayos colocados sobre las acciones del tablero, a menos que un jugador obtenga el beneficio de la primera zona de los aposentos del palacio, que lo convierte en jugador inicial. Cada jugador recibe , según su color 6 lacayos, de los cuales 3 se colocan en las 3 zonas al efect o del tablero (uno en la casilla 20 del marcador de riqueza, otro en la casilla 15 del marcador de gloria y otro sobre el puente del río). 5 cubos, de los cuales 4 se colocan sobre el primer nivel de cada uno de los edificios del tablero y el restante sobr e el nivel 1 del indicador de karma . 1 ficha circular de bonificación, la cual se coloca inicialmente sobre la casilla 12 del marcador de riqueza con la imagen de bonificación de avance fluvial boca arriba . 1 barco, que

In [ ]:
text_ref_rapida = text_ref_rapida.replace('\uf0a7', '')

In [ ]:
print(text_ref_rapida)


GANGES (2 a 4 jugadores) Antes de empezar Decidir si se juega en modo normal o en modo Navaratna (ver el epígrafe Variante Navaratna más adelante ). Sortear jugador inicial , el cual se va rotando en sentido horario cada vez que se agoten los lacayos colocados sobre las acciones del tablero, a menos que un jugador obtenga el beneficio de la primera zona de los aposentos del palacio, que lo convierte en jugador inicial. Cada jugador recibe , según su color 6 lacayos, de los cuales 3 se colocan en las 3 zonas al efect o del tablero (uno en la casilla 20 del marcador de riqueza, otro en la casilla 15 del marcador de gloria y otro sobre el puente del río). 5 cubos, de los cuales 4 se colocan sobre el primer nivel de cada uno de los edificios del tablero y el restante sobr e el nivel 1 del indicador de karma . 1 ficha circular de bonificación, la cual se coloca inicialmente sobre la casilla 12 del marcador de riqueza con la imagen de bonificación de avance fluvial boca arriba . 1 barco, que

## Quick guide

In [ ]:
quick_guide_path = drive_path+"/datos/Rajas_of_the_Ganges_Quick_Rules_Guide.pdf"
text_quick_guide = extract_text_with_pypdf2(quick_guide_path)

In [ ]:
text_quick_guide = text_quick_guide.replace('\uf0b7', '')

In [ ]:
text_quick_guide


'Rajas of the Ganges Set-Up Place the game board in the middle of the table, according to the player count. o If playing with 3 players, use the 2 space cover markers to cover the 1st Quarry space and the 3rd Harbor space. o If playing the standard game, put the Yellow, Brown and Red yield tokens, and the 8 river tokens, in the box. See below for details on the Navaratnas Version and the Ganga Module (2 variants to the game!) Sort the Province Tiles by colo r, and then by the 3 different a nimal ba cks (snake, cow, tiger). o Place the 12 stacks face -down next to the board, then flip the top tile of each stack face -up. Mix the 8 white Y ield tokens face -down on the temple space (white building). Each player takes o 1 Province board (place the sta ndard side up the side without blank yield spots) o 1 Kali Statue board (with the 2 slots in the middle open for dice able to hold 10 dice (thats all it does, fyi) ) o 6 Workers (lay 1 worker sideways on the river bridge, on the 15 fame spac

## Fajas_of_the_Ganges_Automa_r1.0.pdf


In [ ]:
automa_variant_path = drive_path+"datos/Rajas_of_the_Ganges_Automa_r1.0.pdf"
text_automa_variant = extract_text_with_pypdf2(automa_variant_path)

In [ ]:
text_automa_variant = text_automa_variant.replace('Author: Mauro Gibertoni   Rev. 1.0 - 4 Dec 2017', '')


In [ ]:
text_automa_variant

'Rajas of the Ganges Automa mode Author Mauro Gibertoni Rev. 1.0 - 4 Dec 2017 This is a solo variant for Rajas of the Ganges boardgame. You play against a fake opponent the Automa. While you play following all standard rules, the Automa breaks some rules, so for you it will be very easy to take the action for the Automa. Set-up Prepare a standard set -up for wo players, but consider that the Automa doesnt need all the components, so you can leave in the box the following components - the province board ; - the four markers for palaces upgrade ; - the marker for karma points ; - the personal dice an d the Kali statue ; - if you play with Navaratnas version, select one brown yeld tile for you then remove 2 yeld tiles per color . Shuffle the Automa deck and place it close to the board with the blue side face up. Select randomly who will be the first player . Place the provice tiles in 12 piles and organize them in a 3 x 4 grid. Automas overview At each turn alternate your actions with the

## Foros

In [ ]:
# COMPORTAMIENTO RANDOM A VECES TRAE TODO, A VECES NO
def extraer_comentarios(url):
    """
    Extrae los comentarios de un foro de BoardGameGeek desde la página dinámica usando Selenium.
    Utiliza un chromedriver local.

    Args:
        url (str): URL del hilo de foro.

    Returns:
        list: Lista de comentarios extraídos.
    """
    # Configurar las opciones del navegador
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    # Configurar la ruta del driver
    service = Service('/usr/bin/chromedriver')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get('chrome://settings/clearBrowserData')
    try:
        # Abrir la página web
        driver.get(url)

        # Esperar hasta que los comentarios estén visibles en la página
        WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "gg-markup-safe-html"))
        )

        # Buscar todos los comentarios dentro de las etiquetas <gg-markup-safe-html>
        comentarios = driver.find_elements(By.CSS_SELECTOR, "gg-markup-safe-html")
        print(comentarios[0].text)
        # Extraer el texto de cada comentario
        comentarios_texto = [limpiar_texto(comentario.text.strip()) for comentario in comentarios]

        return comentarios_texto

    finally:
        # Cerrar el navegador
        driver.quit()

**estrategia, When is the Portuguese useful?**

In [ ]:
url_strategia_portuguese = "https://boardgamegeek.com/thread/1993283/when-is-the-portuguese-useful"
estrategia_portuguese = extraer_comentarios(url_strategia_portuguese)

Except for the case where you want to rush to get your worker on the bridge, I cannot understand why using a 6 on the Portuguese is useful. The sooner you race through the river, the less likely you are to benefit for those +1 river space bonuses and claim rewards caused by other events.

How else could the Portuguese be useful?


In [ ]:
estrategia_portuguese = [texto.replace('\n', '') for texto in estrategia_portuguese]
estrategia_portuguese = [texto.strip("'") for texto in estrategia_portuguese]

print(estrategia_portuguese)

['Except for the case where you want to rush to get your worker on the bridge, I cannot understand why using a 6 on the Portuguese is useful. The sooner you race through the river, the less likely you are to benefit for those 1 river space bonuses and claim rewards caused by other events. How else could the Portuguese be useful?']


In [ ]:
estrategia_portuguese = ['Except for the case where you want to rush to get your worker on the bridge, I cannot understand why using a 6 on the Portuguese is useful. The sooner you race through the river, the less likely you are to benefit for those 1 river space bonuses and claim rewards caused by other events. How else could the Portuguese be useful?', 'I use it frequently...like you mentioned, usually to rush and get my extra worker.', 'Yeah like already was said if your strategy is not to use the river then get that worker as fast as possible for an advantage', 'People tend to sit around on some of the early high value spaces, so jumping 6 ahead gets you into a cluster that no-one else is blocking you from. Also if you have the 1 presige for each 3 spaces moved along the river, then using the 6 spot is a given.', 'People tend to sit around on some of the early high value spaces, so jumping 6 ahead gets you into a cluster that no-one else is blocking you from. Also if you have the 1 presige for each 3 spaces moved along the river, then using the 6 spot is a given.', 'This. In the Navaratnas version, I used the 6 spot to move 6 spaces onto one of the 2-6 palace bonus spaces, and I used the 6 spot again, which dropped me on the 24th space on the river. The next turn I built a building, connecting the 1 Prestige for 3 River spaces bonus, which got me 8 prestige plus the prestige from the building, which also moved me around the corner over the river movement bonus, which moved me to the last space on the river, which got me another chunk of prestige. I think it totaled out to around 18 fame in the last movement, 22-40.']

**estrategia principiantes**

In [ ]:
url_estrategia_principiantes = 'https://boardgamegeek.com/thread/1942983/beginner-strategy-tips'
estrategia_principiantes = extraer_comentarios(url_estrategia_principiantes)

Hello everyone!

I just got this game, and we are going to play this game in like a week. And I was wondering if you could give me some beginning strategy tips. Like what to go for first, and what is the "best" to start with? I want to become good, before playing with my mates. Haha!


In [ ]:
estrategia_principiantes = [texto.replace('\n', '') for texto in estrategia_principiantes]


In [ ]:
estrategia_principiantes  = estrategia_principiantes[1] if (len(estrategia_principiantes)>=2) else estrategia_principiantes[0]

In [ ]:
estrategia_principiantes

'I have only played the game several times, but I think the best (and probably the most obvious) strategy is to build up your markets in your province as fast as possible, and then take the market action to score them frequently. Look and see which markets are available on the tiles and plan accordingly. The market action that allows you to score all three market types (one of each) does not cost a die, so go for diversity first if the tiles available are in that favor. Of course pick the markets with value 2 or 3 first. My last game I did not buy any market tiles that had a value of one (unless it has a building on it that will score well for fame.) Build up your building upgrades as fast as you can as well. In our second game my opponent ignored buildings and focused entirely on markets - and he lost. So do not ignore them. This is one of those games where you have so much you want to do each turn and you only get to do one thing! Also, get your extra meeple as soon as possible. And 

In [ ]:
estrategia_principiantes = 'I have only played the game several times, but I think the best (and probably the most obvious) strategy is to build up your markets in your province as fast as possible, and then take the market action to score them frequently. Look and see which markets are available on the tiles and plan accordingly. The market action that allows you to score all three market types (one of each) does not cost a die, so go for diversity first if the tiles available are in that favor. Of course pick the markets with value 2 or 3 first. My last game I did not buy any market tiles that had a value of one (unless it has a building on it that will score well for fame.) Build up your building upgrades as fast as you can as well. In our second game my opponent ignored buildings and focused entirely on markets - and he lost. So do not ignore them. This is one of those games where you have so much you want to do each turn and you only get to do one thing! Also, get your extra meeple as soon as possible. And dont forget the action where you can lay a tile on top of another tile. We sort of ignored that until our third game and realized it might be a good idea to connect to the borders of your province as much as possible even if the roads might not bend the way you want, because you can always cover a tile later (although there are restrictions.) This is quickly becoming one of my favorite games. I suggest skipping the basic game and playing with the advanced rules right away. Its not any more complicated and really makes the game shine quite a bit more. Good luck!'

**Fuente: foro, estrategia, guía para jugadores competitivos**

In [ ]:
url_guia_competidores = 'https://boardgamegeek.com/thread/2177187/the-guide-for-competitive-players'
guia_competidores = extraer_comentarios(url_guia_competidores)

Rajas of the Ganges is one of my favourite games. I like it because I see a way to work towards mastering it. I hope that this article will improve the gameplay of those with similar goals, as well as those on Yucata. This article is more tailored towards 2-player games (Navaratnas + Ganga module) but many of the principles also apply to higher player counts.

<Revised in Feb 24> I incorporated new views based on my ~10 games played as part of a Yucata King's Court tournament. (2-players, round robin). The most significant is I now see the value of picking the Great Mogul (#1). Besides updating the guide, I tried to improve the readability of the guide with ChatGPT's help. I am not very proficient with formatting on BGG, so I apologize in advance if it is still difficult to read!

Rajas of the Ganges Strategy Guide

Introduction

Rajas of the Ganges holds a special place among my favorite games, as it provides a clear path to mastery. This article aims to enhance the gameplay for those

In [ ]:
guia_competidores = [texto.replace('\n', '') for texto in guia_competidores]

guia_competidores = [re.sub(r'<.*?>', '', text) for text in guia_competidores]
guia_competidores

['Rajas of the Ganges is one of my favourite games. I like it because I see a way to work towards mastering it. I hope that this article will improve the gameplay of those with similar goals, as well as those on Yucata. This article is more tailored towards 2-player games (Navaratnas Ganga module) but many of the principles also apply to higher player counts. Revised in Feb 24 I incorporated new views based on my 10 games played as part of a Yucata Kings Court tournament. (2-players, round robin). The most significant is I now see the value of picking the Great Mogul (1). Besides updating the guide, I tried to improve the readability of the guide with ChatGPTs help. I am not very proficient with formatting on BGG, so I apologize in advance if it is still difficult to read! Rajas of the Ganges Strategy Guide Introduction Rajas of the Ganges holds a special place among my favorite games, as it provides a clear path to mastery. This article aims to enhance the gameplay for those with simi

In [ ]:
guia_competidores = ['Rajas of the Ganges is one of my favourite games. I like it because I see a way to work towards mastering it. I hope that this article will improve the gameplay of those with similar goals, as well as those on Yucata. This article is more tailored towards 2-player games (Navaratnas Ganga module) but many of the principles also apply to higher player counts. Revised in Feb 24 I incorporated new views based on my 10 games played as part of a Yucata Kings Court tournament. (2-players, round robin). The most significant is I now see the value of picking the Great Mogul (1). Besides updating the guide, I tried to improve the readability of the guide with ChatGPTs help. I am not very proficient with formatting on BGG, so I apologize in advance if it is still difficult to read! Rajas of the Ganges Strategy Guide Introduction Rajas of the Ganges holds a special place among my favorite games, as it provides a clear path to mastery. This article aims to enhance the gameplay for those with similar goals, specifically for 2-player games (Navaratnas Ganga module), but many principles also apply to higher player counts. At a Glance',
 '1. The Macro View Revised in Feb 24 - Mixed strategy Heavy Money Fame 2. Evaluating the Board State at the Start of the Game to Decide on a Strategy - 1 market of each kind 3 and 4 directional tiles (consider your dice as well) 3. Dice Management Revised in Feb 24 - 3 karma dice River 3 color dice River Dancer 2 dice River 1-for-2 dice 1 dice (assuming the dice color matches what you want) 4. How to Get Money - Different market spot - No. 4 - River spots - Connecting to the bottom of the board (longer than the top but more worthwhile in the long run) - 1 market (only worth if activating 3 or more) 5. How to Get Fame - No. 4 then build monuments - Board - River spots 6. How to Quickly Get Your 6 Workers New Section in Feb 24 - River is your good friend - If you have a good Money set-up (3 different markets), focus on river spots giving fame - If you have a good Fame set-up (min. 3 monument upgrades), focus on river spots giving money 7. What Yield Tiles to Pick New Section in Feb 24 - Make sure you have a good Brown end-game yield tile - If you have a bad Money engine, pick Money - If you are reaching the yield tile soon and lack dice, pick dice 8. Others - (Base Game) the most optimal way to build is to go down the center and build both ends - Withhold building to punish opponents for getting dice in the color of the tile you want and can get',
 'Macro View Mixed Money Fame The game concludes when the fame and money tracks intersect. Thus, players can opt to focus heavily on money, fame, or pursue a mixed strategy. In a balanced strategy, players benefit from both tracks, making it generally more advantageous. A slight emphasis on money is preferred due to better bonuses (river and dice) compared to fame (upgrade, karma, and river). However, a fame-heavy strategy can still be successful. In Raja of the Ganges, flexibility is key, and understanding your best options at any given moment is crucial. Evaluating the Board State at the Start of the Game to Decide on Strategy 1 market of each kind 3 and 4 directional tiles By default, seek market tiles of each kind (greenyellowred) providing 2-3 coins to quickly form 3 different markets for 6-9 coins. Two-markets-in-one tile and affordable market tiles are also viable. The 3-different-markets meeple spot offers the best VP-to-cost ratio, as it requires investment in only 3 tiles and costs no dice. If acquiring these tiles is challenging, consider going for monument upgrades. In such cases, focus on 3 or 4 directional monument tiles or those combining monuments and market tiles to build both the fame and money tracks simultaneously.',
 'For example, in the given illustration, consider 7 blue (since green Market is rare) and then either 5 Orange, 9 Green, or 7 Purple. This tile evaluation remains relevant throughout the game as you aim to build towards the bottom for the most rewarding outcomes. Deciding between sub-par 3 or 4 directional tiles (usually market tiles) and tiles that fit your monument upgrades will likely determine your chosen strategy, whether Heavy Money, Mixed, or Heavy Fame. Dice Management 3 karma dice River 3 color dice River Dancer 2 dice River 1-for-2 dice 1 dice Dice color matters only when building. Dice plays a crucial role in lubricating your VP engine. In order of preference, obtain dice through the following methods 1 River spot (1 die per karma) The uniform color aids in building tiles. 2 River spot (2 dice) is superior to 1-for-2 dice exchange because it advances the river track, providing better rewards and an extra meeple. 3 Updated Feb 24 Dancer While Dancer is more useful in the early game, it gradually loses its edge as you accumulate enough yield tiles. 4 1-for-2 dice exchange is usually better than acquiring 1 dice because more dice of the same color make building easier, and you get a free roll of the die you gave up. Think of it as receiving a new die, rerolling your die, and giving it a different color. Consider saving a die to acquire more dice, as this is typically the trade-off. With dice, focus on increasing both money and fame. Strategies for Resource Acquisition in Rajas of the Ganges How to Get Money - Different market spot - No. 4 - River spots - Connecting to the bottom of the board (longer than the top but more worthwhile in the long run) - 1 market (only worth if activating 3 or more) How to Get Fame - No. 4 then build monuments - Board - River spots How to Quickly Get Your 6 Workers New Section in Feb 24 In the Advanced game, you start with 3 workers and can unlock 3 more by advancing on the Money, Fame, and River tracks. Rushing for 6 workers is beneficial, outweighing the drawback of yielding tiles to opponents. This is because 1 worker is minimally worth 1 die, and the benefit of yield tiles diminishes as opponents acquire more. River is your good friend - If you have a good Money set-up (3 different markets), focus on river spots giving fame - If you have a good Fame set-up (min. 3 monument upgrades), focus on river spots giving money While its preferable to get 3 markets and then focus on fame for the extra worker, circumstances may not allow this, usually due to tile constraints. In such cases, focus on fame but make a conscious effort to obtain all the Money bonuses on the river. Anticipate whether your opponent is using the build or river locations and try to beat them to it or delay your visit to these locations until the next round if there are no compelling reasons to visit this round. The goal is to secure the worker on the Money track. The river is an excellent place to utilize low rolls and can yield various rewards. Climbing it strategically for the rewards you need (usually dice fame or money) is key. The Portuguese (6) turns out to be more useful than perceived, particularly when moving up the river without 6 is challenging. Keep this in mind when deciding whether to use the 6 dice to build tiles or ascend the river. 6 can also be a good option when landing on the river reward that allows you to pick one of 1 - 5 bonuses. What Yield Tiles to Pick New Section in Feb 24 Ensure you have a good Brown end-game yield tile - Based on my observation, players prefer taking yield tiles in this order RedBrown YellowWhite. - Starting with a bad brown yield token can be detrimental, and players should quickly exchange it for a good brown yield token. Only the bolded tiles below consistently provide 6 or more fame.',
 '1 fame per 3 river spots 1 fame per worker 1 fame per market 2 coins per monument upgrade 2 coins per tile connected 1 coin per building 2 fame per bend',
 'For white, yellow and red yield tiles, - If you have a weak Money engine, pick Money - If you are approaching the yield tile soon and lack dice, pick dice Others (Base Game) the most optimal way to build is to go down the center and build both ends - The bottom rewards are significantly more potent. Consider this scenario (and how you should generally build in the base game)',
 '4 Fame 7 money or 6 money 8 Money. If lacking dice, consider taking a detour to the 2nd bottom row first, as it is more valuable than going down the sides where you get 3 dice before reaching your rewards. - For the Advanced mode, route choice is flexible; pick a route that aligns with what you want to build. Withhold building to punish opponents for acquiring dice in the color of the tile you want and can get - This is an advanced consideration when players have a good sense of whether a given tile is sought after by others. Otherwise, its generally better to buy a tile first, see what that tile is, before obtaining dice to better adapt to the available tiles for purchase. Parting Remarks Critics have pointed out Rajas of the Ganges for being scripted on a strategic level, but the games essence lies in tactical decisions. Its 80 tactical, akin to navigating a maze in the fog. The fun lies in assessing the most likely path based on limited information. I hope this guide aids those striving to enhance their gameplay Addendum As nandblock rightly points out in the comments, the Master Builder is often overlooked. There are a few advantages to it, namely Converts dead ends to live paths (translating to creating dead ends for yield rewards and then over-building to continue the path) Provides the ability to snag a tile if you lack enough dice of the same color to build conventionally Saves coins. I have used it effectively too. Note that these advantages must be weighed against not advancing your path (since you are replacing and not adding a new tile). Some players use their 1 dice to get the Great Mogul (1). I think this is not worthwhile because most river spots are worth more than 2 fame, or it can be converted to a 6 for Portuguese or building tiles. I will probably only do it if I need the 2 fame to get my fame worker The leading player is 2 or 3 seats after me AND the player seated after me does not threaten me The next round will be the last round The player after me always picks the Great Mogul Update in Feb 24 In 2-player games, I have been humbled by players frequently picking the Great Mogul. The Great Mogul can be a very good pick if both players have developed their 3 markets. Lets say both players can get 6 coins from the spot, the Great Mogul can represent a 12-coins swing (6 to you, -6 to your opponent). This is also very good if it would help you or stop your opponent from getting a reward on the Money track. That said, you still have to weigh it against advancing 1 river step or converting to 6 for Portuguese or building tiles.',
 'Purchasing as many 3 value market tiles as is convenient close to the start of the game is always going to put you in a commanding position. Ive also had a lot of success in this game focusing on only having one colour of dice the entire game if possible. This allows me to give up any numbered die without much consequence while in most cases also stopping anybody else from using that colour for themselves. Theres generally enough market tiles available in any colour to progress comfortably without needing to compete for them. Its also quite easy to invest in a different colour if no tiles are of interest to you once your die stocks start to become low.',
 'Thats a interesting strategy. Are you able to obtain all the green dice (example) so others cant? Or do you mean discouraging others to take green because u can snatch the tile before they do? In this case, I believe its a sensible move. Hence my last point about withholding building. If it works for you, it works. I just think it makes more sense to take whatever dice colour I need to build a particular tile I want.',
 'Or do you mean discouraging others to take green because u can snatch the tile before they do?',
 'It just discourages them. I find as players get more experienced they all go for the strong early market tiles, as the early coin track bonuses are amazing compared to the fame ones. Making sacrifices and racing for tiles is risky and can be fatal if your timingrolling is poor. Keeping only one colour of dice is a much more relaxing way to play and has worked for me so far!',
 'Or do you mean discouraging others to take green because u can snatch the tile before they do?',
 'It just discourages them. I find as players get more experienced they all go for the strong early market tiles, as the early coin track bonuses are amazing compared to the fame ones. Making sacrifices and racing for tiles is risky and can be fatal if your timingrolling is poor. Keeping only one colour of dice is a much more relaxing way to play and has worked for me so far!',
 'I totally agree with your statements. In addition to better reward, makerts is clearly easier - the 1-of-each-market spot (at a time you have fewer workers to collect dice to build monument tiles), no. 4 spot giving immediate 3 coins. If you fail to get a particular tile, you can be stuck with dice that are not good for building. I can see how going pure colour can be more relaxing. Cheers!',
 'Useful discussion. Two points 1) I was surprised to see no discussion of the Master Builder. Im just getting back into playing Rajas after a break, and have been trying to leverage that spot more often--I find that its usually available without a fight, and the ability to overbuild a now-useless palace tile without spending money is great. (Also, it is more flexible than regular building because the 5 die and the other die you use can be different colours.) 2) Also, though of course its a last resort, its actually not too bad to use the 2-coins-and-reroll spot, simply because it requires no dice and is never contested.',
 'Useful discussion. Two points 1) I was surprised to see no discussion of the Master Builder. Im just getting back into playing Rajas after a break, and have been trying to leverage that spot more often--I find that its usually available without a fight, and the ability to overbuild a now-useless palace tile without spending money is great. (Also, it is more flexible than regular building because the 5 die and the other die you use can be different colours.) 2) Also, though of course its a last resort, its actually not too bad to use the 2-coins-and-reroll spot, simply because it requires no dice and is never contested.',
 'Thanks for keeping the discussion alive! 1) I didnt talk about the Master Builder cos, in theory, its inefficient to spend a 5 another dice to over-build when I can just buy another tile and connect to another bonus on the sides of the board. Well, you do bring up a good point that it can save 3 coins which I overlooked! And the 2 dice can be different colour. Im not sure how good they are compared to connecting to the sides of the board though. 2) Im not convinced the 2-coins-and-reroll spot is not too bad. Giving up 1 dice for 2 dice effectively allows you to reroll the dice that was given up AND gets you 1 dice. Cheers! Its been a long since I played Rajas of the Ganges too.',
 'FWIW heres a 3-player game that I just finished, and won, making use of both the Master Builder and reroll spots (and also hugging the sides rather than going to the centre, and mostly ignoring the yield tiles aside from one key cash-bump). Im not sure the same approach would have worked in other circumstances or with players who were actively trying to block me (in particular, they let me take palace tiles which scored very high), but judging by their ratings they are not slouches. httpswww.yucata.deenGameRajas9056284page Definitely a weird game because my board is totally underdeveloped compared to the other players by the end.',
 'FWIW heres a 3-player game that I just finished, and won, making use of both the Master Builder and reroll spots (and also hugging the sides rather than going to the centre, and mostly ignoring the yield tiles aside from one key cash-bump). Im not sure the same approach would have worked in other circumstances or with players who were actively trying to block me (in particular, they let me take palace tiles which scored very high), but judging by their ratings they are not slouches. httpswww.yucata.deenGameRajas9056284page Definitely a weird game because my board is totally underdeveloped compared to the other players by the end.',
 'Good game Nate! When I saw the (runaway) leader Master Building not once, not twice, but thrice, I had the suspicion it was you. Did you always have the 5 when you needed it? Underestimated the difficulty of connecting both yield tiles at the two ends. httpswww.yucata.deenGameRajas9067091page',
 'Thanks--which player were you? Looking at the final boards Im guessing zomgwtfbbq (the only other player who hit a brown yield bonus)? Ive actually never tried to fulfill 2 brown yield bonuses--obviously this would discourage the use of Master Builder, which would make it virtually impossible to reach both sides. I dont go searching for 5-pip dice, but once I have one (or a flippable 2-pip die) and there are some attractive high-value tiles, I try to hang on to it and look to see if I can chain a couple of builds. The final use of Master Builder in that game wasnt strictly necessary but it saved 2 money, so why not. Heh, there was also an instance of using the reroll there (for a cheap boost to the free-worker mark).',
 'Yup, thats me. Ill try to incorporate the Master Builder in the other games. So with a 2 or 5, you are less concerned if you hit a dead end? Now that I think about it, Master Builder lets you be a little reckless in getting yield tile reward and then over-building to continue having a live path. Did you do any of that in our game?',
 'Yeah, you can see that on move 55 I dead-ended the path with a cheap blue-4 tile, then fixed it with an orange-7 upgrade on move 63 so the path could resume. Then the reverse later on built a cheap orange-5 tile at move 74 (overpaying with an orange 6 die to keep the 5 in reserve) that forked to catch a yield tile, then put a high-Fame but dead-end green-9 tile on top of it at move 78.',
 'Im happy to say your advice to use the Master Builder more frequently has helped me. Hows your game against Balzi? httpswww.yucata.deenGameRajas9067090page move 72 I needed 1 fame to get my 5th worker. The over-build allowed me to get the fame. move 88 I needed 1 coin to get the money reward (1 river move). While I had empty space to build, the over-build saved me 2 coins. Considering I narrowly won the game 1 fame to -1 fame, made a consequential difference. httpswww.yucata.deenGameRajas9066219page move 51 If I did not take the monument with 2x buildings, Balzi would get 7 fame. This was early in the game when we were racing to get 15 fame. The over-build saved me 3 coins and a karma. Also consequentila as I narrowly won 1 fame to -2 fame. move 109 wouldnt have made a difference',
 'Added tip about the Master Builder (courtesy of nandblock aka Nate Dorward) and some other observations based on the recent 5 games that I played after a long hiatus.',
 'Am playing one game with Balzi a couple others at the moment--didnt really get off on a proper footing on that one, and one player (Katzenfloh) is about 10-delta ahead of everyone else. IIRC (without reviewing the game history in detail) the initial tile market was kind of crap, which led to a bottleneck. Glad that youve found some good uses for the Master Builder! I suspect that as with every worker placement game, theres a key element of group-meta here if a spot is perceived as desirable by most players, it will be contested and leveraged a lot. But my experience is that new and even experienced players rarely use the Master Builder, so it remains a cheap, usually-available action when other players have run up the cost of building to 2-3 coins. Looking at your revised notes any thoughts on the good vs bad brown yield tiles? I suspect wed agree that the one that rewards you with 1 Fame for every 3 spots on the River is the best (use the Portuguese a few times, and holy crap, you get 8 Fame for minimal effort and expense), and the one that gives you 2 Fame for every elbow bend is among the worst (since scoring it well requires crippling your board development).',
 'I mentioned my thoughts on the yield tiles at the end. I find 1 fame per worker the best. Simply because you are guaranteed 5 fame without doing anything you would not already have been doing. 1 fame per market has pros and cons over 1 fame per worker. On the upside, it has potential to go above 6. It has synergy with the money strategy (3 of 2 markets and 1 of the last market) and makes the last few River spots very powerful. On the downside, it requires more work and we know the tiles market might not always be favourable. 1 fame per 3 river is weaker because it requires rolling a 6at least once or twice. Sometimes the river slot 6 ahead is not what you want (eg you have no karma and its a karma spot, you only have 1 market when it triggers 3 markets). Sometimes you have competing use for the 6 (building or denying a tile that an opponent can build if you dont). But its still good because most time you want to get that 6th worker and each Portuguese means 2 fame if you eventually connect. Yea the bend is definitely the worst because its the worst... Although I have gotten 4 fame out of it in one game.',
 '1) I was surprised to see no discussion of the Master Builder. Im just getting back into playing Rajas after a break, and have been trying to leverage that spot more often--I find that its usually available without a fight, and the ability to overbuild a now-useless palace tile without spending money is great. (Also, it is more flexible than regular building because the 5 die and the other die you use can be different colours.)',
 'I completely agree with the 5 spot. Personally I am usually only going for one brown bonus tile. So after I get it or once I am close, I am constantly on the hunt for a 5 die. While saving on gold cost is nice, I think the bolded text on flexibility is by far the most important and undervalued part of the 5 spot. In the late game being able to quickly pivot between high value colored building tiles is super good to ensure you do not get blocked when going for those last few fame points or money points. In the early and mid game, if you build up dice for a color and get it taken from you, its usually not bad to have a couple dice laying around for a later play. But at the end of the game, you really just want to push for the end, so having a couple high valued dice become the wrong color because someone else blocked you is a much larger setback.',
 'Every game of RotG that I have played (admittedly, not very many) has been won convincingly by the player who adopted the Master Builder strategy, and concentrated on province tiles with buildings rather than markets. (Just to be clear, the Master Builder should only overbuild buildings, not markets.) One obvious reason for this, not mentioned above, is that you need only half as many fame points to end the game, if your money doesnt move, as you need money to end it, if your fame doesnt move. Clearly, this would not matter if fame was twice as hard to come by as money, but I dont think this is the case. For example, a tile with two buildings on it can give you 8 fame, which is as good as 16 money on the track. Using the Master Builder, you could build that tile for one worker, zero money, and two dice, one of which can be any colour, albeit it must be a 5. I cant immediately think of a way to get 16 money (net) for less than that. Another benefit of going down the fame route via Master Builder is that it seems to make it easier to get your first extra worker than if you take the money route. In my last game, even though I started out aiming to get my extra worker from the money track - as did the other players - I found myself acquiring fame almost without trying (I think it was because the cheapest tiles, or the ones with the most helpful road layouts, happened to have buildings on them) so I switched tactics and still gained my worker before the other players. Again, as others have pointed out, overbuilding tiles makes it easier to claim the special yields, as you dont have to worry about ensuring that you have an exit road from the first tile you build in a space (yes, I am aware that you cannot claim special yields from the overbuilding tile). Finally, once you decide that fame is your bag, spaces like the Great Mogul and the river spaces that give fame or building upgrades become worth using, which is good because the other players are less likely to be interested in them.',
 'One obvious reason for this, not mentioned above, is that you need only half as many fame points to end the game, if your money doesnt move, as you need money to end it, if your fame doesnt move. Clearly, this would not matter if fame was twice as hard to come by as money, but I dont think this is the case.',
 'Thanks for chiming in. I wouldnt say Fame is less than twice as difficult to get than Money. Fame Firstly, to get to the point that buildings with Master Builder can generate 7-8 Fames, 3-4 upgrades are required, one of which comes from reaching 5 Fames. This requires luck to get the 4 dice. Secondly, luck and skill are required to reveal the correct buildings and pick the correct upgrades. If not, you are getting 5-6 Fames per building. This is also assuming you seldom clash with your opponents over getting a building. The only time when I want to pick up Fame buildings early is when I want the 4-directional ones to continue my path. Fame buildings are simply more powerful later in the game when you have picked your upgrades. Money Money is very different. While markets also requires setup in the form of buildings, markets require fewer dice (free for the 3 diff market and 1 dice for the single colour market) and less restriction on the number on the dice (usually a 2 or 3 is enough for the single colour market.',
 'Finally, once you decide that fame is your bag, spaces like the Great Mogul and the river spaces that give fame or building upgrades become worth using, which is good because the other players are less likely to be interested in them.',
 'In my article, I argued for a balanced strategy. If it is necessary to slant one way, it should be money. Given the money-slant, Fame is scarcer and more useful to obtain when comparing 1 Fame vs 2 Money. I dont think anyone should ignore the river spaces with Fame just because it is Fame and not money. The river spaces with 1 building upgrade are bad. If I recall correctly, the spaces near it give like 3 Fames or 6 coins. You would need to build 4 buildings of the upgrade-type before you are better off. Thats not even considering that the 4 Fames are spaced out over 4 turns and not immediate.',
 'Being the first player is crucial on 2-player games and not so much in 3. Yet I prefer to take it, when possible. For example, when moving 1 space on the river is not interesting for me.',
 'Agree, Krotek. As with everything, you have to weigh against your options. -1 karma to use as 6 to buy a tile or fly 6 river steps are also things to consider. It all depends on the players judgment.',
 'How do you feel about the 2-player game? I played a 2-player game yesterday and lost badly to my opponent who got 2 early upgrades of a monument that came out far more commonly than any other monument. They ended up building a bunch of them resulting in getting tons of fame. I didnt feel I stood a chance from the get go but I went heavy into one type of market from the start and had trouble with rollskarma all game. I feel like in a 3-4 game there might be more competition for province tiles so my opponent wouldnt have been able to get so many of that one type of monument. Do you feel the 2-player game suffers from this or in other ways?',
 'I havent played RotG in a long while but I think 2P has some differences from 3P and 4P. It should still be a good game. 1) zero-sum game - denying your opponent is as good as scoring yourself (ignoring the MoneyFame bonuses). Using your example, normally you wouldnt build monuments that only give you 2 fame but this is viable in a 2P. You dont progress as fast but so does your opponent. 2) increased importance of the different markets spot - there is only one spot. You have to weigh it against other good options like 2 dancer etc. 3) choice of buildings is more deliberate - in a 2P, buildings only refresh if you or your opponent buys one. This means you have greater control over what buildings to get (by getting dice of that colour). It also means you need to be aware what buildings your opponent is going for and decide whether you should beat him to it.',
 'Posting here since you all are skilled players The different market spot seems to be OP and jockeying for it every round is not very interesting to me in a game that otherwise has a lot to love. 1) How do you beat the player that focuses on it? 2) any thoughts about nerfing the spot by requiring a dice? Appreciate any insight you can give!']

**Fuente: foro, reviews.**

Los foros consultados fueron:

- "Rajas of the Ganges: An Awesome Game that Exceeded My Expectations"
- "Silver Duck Reviews: Rajas of the Ganges"
- "A Disappointing First Impression of Rajas"
- "Patrick Reviews Rajas of the Ganges"

In [ ]:
rev_1 = extraer_comentarios('https://boardgamegeek.com/thread/2282811/rajas-of-the-ganges-an-awesome-game-that-exceeded')

This is my first review for BGG. I've really enjoyed Rajas so far so it seems like a great game to start with! I love midweight Euros and a big appeal of this game for me was the stated 60-75 minute play time. I often play games with coworkers at lunch and we have an hour to play. For that reason, I don't get to play heavier games like Great Western Trail very often. Games that have seen regular play lately include Rajas, Tiny Towns, Tournay, Jaws, Res Arcana and Century: Spice Road.

I bought Rajas of the Ganges a few weeks ago after months of delayed gratification... it's been on my wishlist for a while. I've spent a fair amount of time traveling in India so the theme was automatically a win for me.

The art and components are fantastic. 48 dice of varying colors, wooden meeples, nice thick tiles for province building. I also really love the Kali component for storing dice. The only component I've found lacking is the player 1 elephant - sure, it's cute, but it easily falls apart. Th

In [ ]:
rev_2 = extraer_comentarios('https://boardgamegeek.com/thread/2780072/silver-duck-reviews-rajas-of-the-ganges')

Plays: 4 at player counts: 2 + 3

Appearance

The front of the box shows a beautiful landscape and river with a deity holding dice and is full of colour and detail. The back of the box shows a 2 player game in progress showcasing the majority of the components and gives a description of the theme of the game. The game has a good insert which holds the components well and makes setup a lot quicker. The player boards are well laid out but a little bland until you lay tiles on them during the game. The tiles are more detailed with various spices and buildings, along with a large number and colour to denote what they require to be built. The wooden coloured player components are easy to distinguish from one another and include some shaped meeples along with some bland cubes. The main board is a particular highlight of the components, it is very colourful and detailed and doesn’t look overwhelming. Finally the dice are very good quality in bright colours with a nice weight to them.

Gamepla

In [ ]:
rev_3 = extraer_comentarios("https://boardgamegeek.com/thread/2832640/a-disappointing-first-impression-of-rajas")

I have only played Rajas once at 2p and I must say that I found it to be an utterly underwhelming and overly frustrating experience.

The problem with Rajas is that it overwhelmingly relies on luck. Not the sort of fun luck you'll find in Camel Up or Quacks of Quedlinburg but the sort of luck that makes you wanna flip the table. There is no question that the most important aspect of the game is your personal player board. This is your engine that drives your money and karma production and if it isn't impressive, you're simply not gonna win. How does one make an impressive player board? With lots of dice pips, which means you either need a lot of dice or to roll well. If your opponent rolls better that you or happens to pick up all of the bonus tiles that award extra dice they will leave you for dead, and that's exactly what happened in my game. Sure there are ways to mitigate poor dice rolls with the use of "karma" but it just isn't enough as karma itself is costly and hard to come by.

In [ ]:
rev_4 = extraer_comentarios("https://boardgamegeek.com/thread/1995134/patrick-reviews-rajas-of-the-ganges")

2-4 players
60-90 minutes
medium
Tile laying, worker placement


Theme and Mechanics

In Rajas of the Ganges, players are rich noble type people from India (please forgive my complete ignorance of Indian culture for this review, I am not well informed about this particular theme) and they're working to improve their estate and earn the most money and [prestige, fame, points, something like that] to win the game. The theme really does come through, especially with the tile laying/building up their estates. It's hard for me to comment too much on theme and mechanics relating, because I'm not too familiar with this topic, but it does seem like the mechanics are somewhat abstracted. It also seems like the setting could be changed without effecting game play, which makes the theme feel somewhat disconnected at times.

Players each have an estate board with their palace on it, and several empty spaces, as well as a place to store dice, and 3 workers. There are 2 tracks around the outside boa

In [ ]:
rev_1 = [texto.replace('\n','') for texto in rev_1]
rev_1

['This is my first review for BGG. Ive really enjoyed Rajas so far so it seems like a great game to start with! I love midweight Euros and a big appeal of this game for me was the stated 60-75 minute play time. I often play games with coworkers at lunch and we have an hour to play. For that reason, I dont get to play heavier games like Great Western Trail very often. Games that have seen regular play lately include Rajas, Tiny Towns, Tournay, Jaws, Res Arcana and Century Spice Road. I bought Rajas of the Ganges a few weeks ago after months of delayed gratification... its been on my wishlist for a while. Ive spent a fair amount of time traveling in India so the theme was automatically a win for me. The art and components are fantastic. 48 dice of varying colors, wooden meeples, nice thick tiles for province building. I also really love the Kali component for storing dice. The only component Ive found lacking is the player 1 elephant - sure, its cute, but it easily falls apart. This is n

In [ ]:
rev_1 = ['This is my first review for BGG. Ive really enjoyed Rajas so far so it seems like a great game to start with! I love midweight Euros and a big appeal of this game for me was the stated 60-75 minute play time. I often play games with coworkers at lunch and we have an hour to play. For that reason, I dont get to play heavier games like Great Western Trail very often. Games that have seen regular play lately include Rajas, Tiny Towns, Tournay, Jaws, Res Arcana and Century Spice Road. I bought Rajas of the Ganges a few weeks ago after months of delayed gratification... its been on my wishlist for a while. Ive spent a fair amount of time traveling in India so the theme was automatically a win for me. The art and components are fantastic. 48 dice of varying colors, wooden meeples, nice thick tiles for province building. I also really love the Kali component for storing dice. The only component Ive found lacking is the player 1 elephant - sure, its cute, but it easily falls apart. This is nitpicking, though. I have played this game at 2, 3 and 4 players and have loved it at all counts. Turns go fairly quickly. Interaction is indirect but theres a fair amount of tension due to the competition for building tiles and placing workers. I always find myself paying attention to the turns of others. Rajas also combines dice rolling, tile laying and worker placement in an extremely satisfying combination. The worker placement vibe gives me a Lords of Waterdeep feel, which is a big positive. Players start with 3 workers but can unlock up to two more, so all players are racing to get an advantage (however brief) where they get additional actions in a turn. The tile laying comes into play when building out provinces that include markets (for money) and monuments (for fame) - plus reaching certain spots leads to bonuses. This adds a nice, classic Carcassonne vibe. The individual player boards for provinces are excellent and the rules for building out provinces are simple yet engaging. The dice rolling aspect is also excellent. I hate luck-based games and Im generally not a fan of dice. Games like King of Tokyo bore the heck out of me. However, I love games where I can mitigate andor manipulate dice, and this is one area where ROTG really shines. Dice are rolled as soon as they are acquired. I am almost always excited by what I roll because I have so many options. Low rolls can be turned to high rolls by using karma (and high rolls are desirable for gaining buildings). However, low rolls can be desirable in their own right - for instance, the dancer can only be activated with a die roll of 2 (which results in 2 fresh dice to add to a players pool along with a bonus tile) and the yogi is activated with a die roll of 3 (a free die plus 2 karma). Each player also has a boat on the river that can be moved only with a roll of 1-3, triggering special bonuses. In the rare instances I get a mediocre roll and find myself stuck, its generally because I havent done a good job replenishing karma. My gaming group have found multiple strong strategies for winning this game. Being resourceful with dice and balancing fame and money are all important. Also, everyone Ive played with so far (4 other people plus myself) have all really liked this game. Best of all, this game can be played in about an hour. I played a 3-player game a few days ago that was tense, thinky and immersive and led to all three players nearly winning the game despite the fact that we all pursued drastically different strategies. I was amazed that we completed the game in an hour. It felt like wed played a meatier game, which is a huge plus for me. As much as Id like to bring a big ol Euro to the table at lunch, its just not an option for us at work! I give this game a 9 out of 10 (and I havent given any games a 10 just yet, so 9 is as high as I go). Highly recommended!',
 'Nice review and nice to see someone not complaining about one strategy being stronger than all others I love this game and play it whenever I get the a chance at it.',
 'I think its a pretty underrated game. It doesnt necessarily have a whole bunch of strategies and routes to victory to explore compared to some of the other praised euros. But I like it for that reason. Its more about adapting to the board, your dice rolls and your opponents rather than what strategy am I going to explore today (until you run out of strategies to explore). It has plenty of decisions space every turn and plays super fast once everyone knows what they are doing, without feeling light at all.',
 'Ive seen plenty of complaints about a money-heavy strategy being the only way to go and I took a picture of my last game to prove thats not the case',
 'Yellow player triggered the game end with fame and money being almost exactly balanced. Red player (me) had fame and money overlap by 2 during last turn, taking the lead. I admit to having a very money-heavy strategy. Green player tied with me despite having a very fame-heavy strategy. I won due to the tiebreaker. It looked like green was going to win most of the game and a few different decisions at the end wouldve led to a win for him. All of those approaches can lead to a win! And yes, its not Great Western Trail... why should it be? Plenty of great decisions and strategies to keep this game fun and engaging for a long time!',
 'I think its a pretty underrated game.',
 'its ranked in the top 200... if theres one thing this game is not, is underrated, specially considering it has been rated by almost 6,000 users... Its evident that whoever owns this game, does also really enjoy this game.',
 'Top 200 might not be underrated, but I do think this one is worthy of cracking the top 100. Id take it over other similarly weighted worker placement games like Stone Age and Lords of Waterdeep (though I like the latter a lot)... and those are both top 100.',
 'Glad to read another review of this wonderful game! Love me some Rajas. Theres a goodie box coming out by the end of this year',
 'A goodie box??!! I will keep an eye out for that... Also curious to see if this game will eventually get some more expansions. That will also probably help for staying power in the rankings.',
 'A goodie box??!! I will keep an eye out for that... Also curious to see if this game will eventually get some more expansions. That will also probably help for staying power in the rankings.',
 'The goodie box does that. Adds expansions to the games. They are minor though. More like adding modules to add more spaces or different type of tiles into the game. There was a promo that added the first module (or mini expansion called mango village). There are a few promos that add different tiles with powers other than the regular market or building tiles. The goodie box will include the mango village with 2 new tiles, and add 2 new modules into the game with a few other things.']

In [ ]:
rev_1 = [texto.replace('\n','') for texto in rev_1]
rev_1

['This is my first review for BGG. Ive really enjoyed Rajas so far so it seems like a great game to start with! I love midweight Euros and a big appeal of this game for me was the stated 60-75 minute play time. I often play games with coworkers at lunch and we have an hour to play. For that reason, I dont get to play heavier games like Great Western Trail very often. Games that have seen regular play lately include Rajas, Tiny Towns, Tournay, Jaws, Res Arcana and Century Spice Road. I bought Rajas of the Ganges a few weeks ago after months of delayed gratification... its been on my wishlist for a while. Ive spent a fair amount of time traveling in India so the theme was automatically a win for me. The art and components are fantastic. 48 dice of varying colors, wooden meeples, nice thick tiles for province building. I also really love the Kali component for storing dice. The only component Ive found lacking is the player 1 elephant - sure, its cute, but it easily falls apart. This is n

In [ ]:
rev_2 = [texto.replace('\n',' ') for texto in rev_2]
rev_2

['Plays 4 at player counts 2 3 Appearance The front of the box shows a beautiful landscape and river with a deity holding dice and is full of colour and detail. The back of the box shows a 2 player game in progress showcasing the majority of the components and gives a description of the theme of the game. The game has a good insert which holds the components well and makes setup a lot quicker. The player boards are well laid out but a little bland until you lay tiles on them during the game. The tiles are more detailed with various spices and buildings, along with a large number and colour to denote what they require to be built. The wooden coloured player components are easy to distinguish from one another and include some shaped meeples along with some bland cubes. The main board is a particular highlight of the components, it is very colourful and detailed and doesnt look overwhelming. Finally the dice are very good quality in bright colours with a nice weight to them. Gameplay The 

In [ ]:
rev_2 = ['Plays 4 at player counts 2 3 Appearance The front of the box shows a beautiful landscape and river with a deity holding dice and is full of colour and detail. The back of the box shows a 2 player game in progress showcasing the majority of the components and gives a description of the theme of the game. The game has a good insert which holds the components well and makes setup a lot quicker. The player boards are well laid out but a little bland until you lay tiles on them during the game. The tiles are more detailed with various spices and buildings, along with a large number and colour to denote what they require to be built. The wooden coloured player components are easy to distinguish from one another and include some shaped meeples along with some bland cubes. The main board is a particular highlight of the components, it is very colourful and detailed and doesnt look overwhelming. Finally the dice are very good quality in bright colours with a nice weight to them. Gameplay The aim of the game is to make your fame and wealth track markers intersect one another, these run in parallel around the board in opposite directions. When a players 2 markers intersect the end of the game is triggered, with the player with the biggest difference between the 2 markers winning if multiple players tokens intersected. Each turn players will place workers, pay any costs associated with the spot and carry out the action listed. These costs vary from moving back on the money track to paying a specific colour or numbered dice. Players will continue to place workers until all workers have been placed and the board will then reset with players collecting back all their workers and the next round can begin as long as the end game hasnt been triggered. There are 4 main different worker placement areas the quarry, market place, palace and harbour. Using the quarry worker area you can build a province tile into your personal province. There are 12 different province tiles on offer at any time, 3 for each different coloured dice. Your must pay the listed money cost when placing your worker along with matching coloured dice of equal to or greater pip value than the tile you are acquiring. You must place these tiles with a direct path back to your starting palace on your province. You gain any yields that you connect to on the edge of your province and receive the money from any markets on the tile. You also receive fame for any buildings on the tile based on your position on that buildings scroll track on the main board, so between 2 and 4 fame. There are various actions and bonuses on the board that allow you to move up on the 4 building scroll tracks. The market worker area allows you to collect income from the markets in your province, either from the same type of market or different ones. When activating the same market you must pay a dice and can only activate as many markets as the pip value of the dice. The palace has 3 main areas the terrace, balconies and the chamber. In the terrace and balconies you can either collect a specific coloured dice of your choice or trade one coloured dice for 2 of a different coloured dice. Whenever you collect dice you roll them and place them onto your Kali statue if you have room. You can always spend karma (if you have it) to flip a dice when you need to pay a cost. In the chambers you can spend a dice of a specific number to activate one of the characters for a specific action. These include gaining dice, karma, scroll movement, taking first player and building over province tiles in your area. In the harbour you can move your boat 1 to 3 spaces on the river and take the listed bonus on the space you end your movement on. The amount of spaces you move depends on the value of the dice spent. Boats cannot be on the same space and you simply skip spaces opponents boats are present on. The bonuses are very similar to the other actions on the board but towards the end reward dice, money or fame based on a condition such as number of karma, scroll movement or markets built in your province. You start the game with 3 workers but can unlock up to 3 more from the fame, money and river tracks when your corresponding marker passes them. Play continues until the end of the game when one players score markers intersect. Play continues until every player has played an even number of turns and then the winner is determined. The turns flow very naturally in this game with simple to resolve actions and costs to pay. The game plays 2 to 4 players and an unofficial solo automa is also available on bgg. Theme and Game Length The game has a good theme, different to many other Euro games of this weight but it doesnt really come through in the actions you are performing. Why are you trying to get wealth and fame to meet, who are these characters and why does the river give the bonuses it does. Gaining workers makes slightly more sense as when you get more famous on the 3 tracks more people are willing to work for your province. The game doesnt drag at all and is actually quite fast for the level of decisions offered. The game scales well with 2 different sizes of the board to tighten up the worker placement spots. The game takes about 20 minutes per player which is very good considering the weight of decisions you make. Strategy There are two main ways to score points in this game, wealth and fame. It is very hard to just work on just one of these and win so over the course of the game you will be earning both. Players will differ in their overall strategy but there is only the one way to win, intersecting your markers. A focus you must have is to get more workers as quickly as possible. The two workers that can be unlocked on the wealth and fame tracks are the easiest to obtain. The wealth track also offers multiple other bonuses so is the track you should target at the start of the game. If other players have more workers than you for too long you have no chance of winning with the yield tiles offering too little in compensation for the worker placement actions they are taking. Each fame is worth 2 money so this obviously looks more appealing to try and race up but without the track bonuses this shouldnt be an early game focus. Instead you should look to increase your scrolls for the 4 building types. If you specialise in one or two building types other players may build tiles with that building type to prevent you from building it. Also with 4 different building types the tiles that may come out may not be the ones you get extra fame for so its probably best to spread your scroll movement out. A space that you cannot underestimate is the space to activate up to 3 different markets in your province. You want to make sure to build 3 different high value markets as soon as possible to take advantage of this space. This spot (or spots in the 34 player games) is usually the first spot players go to at the start of each round from the mid game onwards. It costs no dice and can yield anywhere from 3 to 9 money which is better than any other spot due to having no costs. Activating multiple markets of the same type can also be good but requires a lot of the same market to be built which depends a lot of the tiles that come out. Many of the spots cost dice of various values with every value having a potential good use. If you get lots of high value dice you should be using these to build tiles, trying not to waste too much pip value. The lower value dice can be used for some of the characters or more likely the river. The river is sort of split into two halves by your unlockable worker with the first half offering small bonuses like dice, scroll movement and fame. The later half offers powerful one off bonuses like activating multiple markets or gaining fame and money based on a condition. It is perfectly viable to skip most of the early bonuses and plan to maximise these powerful one off bonuses instead. With so many spots costing dice you always want to make sure you dont run out. You should never take the terrace actions to gain one dice as it is so inefficient and should instead look to use bonuses to acquire dice or trade one dice for two. In the higher player count games you also want to make sure you are not 3rd or 4th in turn order for long due to some worker placement spots being clearly better than others and you can really fall behind if you are not careful. There have been some runaway leader problems in the games that I have played of this. During the mid game it became obvious who was going to win generally because of the market setup they had on their board. This meant one worker could gain them 6 12 money with each action which was much stronger than the other players. Newer players will struggle with experienced players with both tracks to manage and not being able to distinguish which worker placement spots are better than others. The game has modest replayability as you are aiming to do the same thing each time and the way the tiles come out and what dice values you get will change each game. The game also comes with several modules which significantly help replayability and make the game better in my opinion with ways to customise your province. The game can end very suddenly and can be difficult to plan for so you need to be building towards the end of the game constantly. Accessibility The game is light to medium weight with simple to perform and resolve actions but some setup on your province tile required to do well. There is also some consideration needed for when and where to place your workers to maximise their benefits. The game is very easy to teach with obvious iconography and the main board layout also helps. The rulebook is excellent, very clearly laid out rules with multiple examples and text descriptions of each worker placement spot. The game takes up a decent amount of table space with each players province, the main board and places for tiles and dice. Final Thoughts Rajas of the Ganges is a good next step worker placement Euro game with simple to understand rules and great components. The game has good flow to it with your points increasing steadily before exploding towards the end. One of my main problems is the market space on the main board especially the activate 3 different ones in your province. This is way too strong due to having no cost and makes first player way too powerful in this game with players needing to shift it multiple rounds in a row to mitigate this. I really like the scoring in this game its fairly unique and offers different considerations unlike normal Euro style games. I also like the fact that all values of dice have uses in this game so even though low numbers arent good for building you can use them on characters and the river or to trade. A solid light to medium weight game. 710 If you enjoyed this review please leave a thumbs up and also think out about checking out one of the 120 reviews available on my blog (with pictures), Silver Duck Reviews, and subscribing to let you know when new reviews go live.']

In [ ]:
rev_2 = [texto.replace('\n',' ') for texto in rev_2]
rev_2

['Plays 4 at player counts 2 3 Appearance The front of the box shows a beautiful landscape and river with a deity holding dice and is full of colour and detail. The back of the box shows a 2 player game in progress showcasing the majority of the components and gives a description of the theme of the game. The game has a good insert which holds the components well and makes setup a lot quicker. The player boards are well laid out but a little bland until you lay tiles on them during the game. The tiles are more detailed with various spices and buildings, along with a large number and colour to denote what they require to be built. The wooden coloured player components are easy to distinguish from one another and include some shaped meeples along with some bland cubes. The main board is a particular highlight of the components, it is very colourful and detailed and doesnt look overwhelming. Finally the dice are very good quality in bright colours with a nice weight to them. Gameplay The 

In [ ]:
rev_3 = [texto.replace('\n',' ') for texto in rev_3]
rev_3

['I have only played Rajas once at 2p and I must say that I found it to be an utterly underwhelming and overly frustrating experience. The problem with Rajas is that it overwhelmingly relies on luck. Not the sort of fun luck youll find in Camel Up or Quacks of Quedlinburg but the sort of luck that makes you wanna flip the table. There is no question that the most important aspect of the game is your personal player board. This is your engine that drives your money and karma production and if it isnt impressive, youre simply not gonna win. How does one make an impressive player board? With lots of dice pips, which means you either need a lot of dice or to roll well. If your opponent rolls better that you or happens to pick up all of the bonus tiles that award extra dice they will leave you for dead, and thats exactly what happened in my game. Sure there are ways to mitigate poor dice rolls with the use of karma but it just isnt enough as karma itself is costly and hard to come by. In my

In [ ]:
rev_3 = ['I have only played Rajas once at 2p and I must say that I found it to be an utterly underwhelming and overly frustrating experience. The problem with Rajas is that it overwhelmingly relies on luck. Not the sort of fun luck youll find in Camel Up or Quacks of Quedlinburg but the sort of luck that makes you wanna flip the table. There is no question that the most important aspect of the game is your personal player board. This is your engine that drives your money and karma production and if it isnt impressive, youre simply not gonna win. How does one make an impressive player board? With lots of dice pips, which means you either need a lot of dice or to roll well. If your opponent rolls better that you or happens to pick up all of the bonus tiles that award extra dice they will leave you for dead, and thats exactly what happened in my game. Sure there are ways to mitigate poor dice rolls with the use of karma but it just isnt enough as karma itself is costly and hard to come by. In my game I never felt like I had a chance of winning, yet I feel like I played as well as I could have with what I had. I absolutely do not feel that I had any chance of winning the game with my dice rolls and bonus tiles draws when compared to my opponents. My choices just seemed to have little to no agency; all the agency was in the dice rolls and that just isnt fun for me There are some interesting ideas in here, but thats about it. The interacting famemoney tracks seem interesting at first glance but when you really think about it it turns out that it couldve just been one track, not two. If something were to award you with one fame, it couldve just been two dollars (or rupees) to the same effect. Sure, there are bonuses to be had as you progress along the individual tracks but that only serves to force you to attempt a little bit of everything and dis-incentivizes focusing on one particular strategy. Overall, Rajas is a very generic, bland euro game with far too much dependence on luck. In my game it never felt like I was playing against my opponent, but rather I was playing against the game and the game was being much, much nicer to my opponent than it was to me. I would be hard pressed to play this again.',
 'Totally the opposite of every play Ive had of this, including my first. The game has less luck than a lot of games using dice. Some spots can only be visited with low rolls and many of them are powerful. Maybe you just dont understand the game and cant figure out how to value what those options are? Whatever the reason, its not the games fault. Raja is a top 10 all-time game for me and is likely to remain that way.',
 'Rajas was such a hit with me, I play against a difficult automa I printed cards for and have played multiplayer. There were always things to do with different dice, high or low, and everything is cleverly put together. So you have only played once at two player, have you played any other ways? Maybe it was just a really unlucky game and your opponent was luckier and also stole the areas youd want, before you. If there was only one track it wouldnt be that much fun.... A lot would seem odd too, spending money to gain more money? Fame and crossing on the track make it more fun. The Dice Charmers version of this game is also super fun. I always wrote this off since I am a solo player, but always wanted the game. So glad I found the automa and then did end up recruiting newbie type gamers. Easy to teach and understand. This game instantly jumped into my top nine. What are some games you love, so we know if it is even your style? To each their own of course!',
 'I know that some of the low dice-roll spaces are valuable, but not nearly as valuable as buying good tiles. I doubt one could ever win this game with consistently low die rolls.',
 'Maybe it was just a really unlucky game and your opponent was luckier - exactly my point.',
 'Maybe it was just a really unlucky game and your opponent was luckier - exactly my point.',
 'Ah yes, but dice rolling isnt often so perfectly one sided, so my point is that your first experience is more of a fluke than an actual measure of how the game usually plays. I mean, of course its your right to dislike it and find is disappointing. I just think your first game sounds like a fluke. The game is usually tight and not such a runaway winloss.',
 'Also it does seem very Euro-introductory, but I love the simplicity and easy to figure out iconography. Easy to teach to newer gamers interesting in expanding. I also love the roll of dice. Maybe one of the goodies boxes would fix the problems you are having. I am going to look closer at them for myself.',
 'The problem with Rajas is that it overwhelmingly relies on luck.',
 'No it doesnt. Its a dice-driven game, and like all dice-driven games there is an inherent amount of randomness (or luck as you call it). But overwhelmingly reliant on luck? No, because there are multiple ways to mitigate dice values that arent working for you, and Karma is just one of them. You have one particular dice that isnt working for you? Trade it for 2 other different colored dice and get a fresh roll. There are good spots for low value dice rolls as well as higher value ones. As for building up your player board, start modestly and work your way up - the default presentation of the tiles is very even and fair in that there should always be something you can afford to build. It may not be the perfect tile for you at that given moment (it rarely is) but a creative player should be able to find a way to make something work.',
 'I would be hard pressed to play this again.',
 'Thats too bad. It really is a terrific game but if youre confident that one play was enough to put you off for good, then it is what it is.',
 'I know that some of the low dice-roll spaces are valuable, but not nearly as valuable as buying good tiles. I doubt one could ever win this game with consistently low die rolls.',
 'It took me a bit to figure out how to finagle this scenario, but the key is karma and the river. If you can time out your karma right, you can snag a bunch of low value tiles to get your bonuses. The two space is also seriously useful in these scenarios since you can get up to three dice of your choice. Additionally, dont feel bad if you take the re-roll action, its not great, but a bit of cash in your pocket and a better roll can be a good thing. Weirdly enough, I tend to struggle with consistent high value rolls, particularly in early game.',
 'The problem with Rajas is that it overwhelmingly relies on luck.',
 'No it doesnt. Its a dice-driven game, and like all dice-driven games there is an inherent amount of randomness (or luck as you call it). But overwhelmingly reliant on luck? No, because there are multiple ways to mitigate dice values that arent working for you, and Karma is just one of them. You have one particular dice that isnt working for you? Trade it for 2 other different colored dice and get a fresh roll. There are good spots for low value dice rolls as well as higher value ones. As for building up your player board, start modestly and work your way up - the default presentation of the tiles is very even and fair in that there should always be something you can afford to build. It may not be the perfect tile for you at that given moment (it rarely is) but a creative player should be able to find a way to make something work.',
 'I would be hard pressed to play this again.',
 'Thats too bad. It really is a terrific game but if youre confident that one play was enough to put you off for good, then it is what it is.',
 'Yeah, these are all somewhat valid points, but if Im having to take all these actions trying to get more dice and reroll my low value dice while my partner is not then Im inherently at a disadvantage. Not to mention that they consistently picked up extra dice with the 2-pip die action while I was getting the less-useful bonuses. It was not a level playing field. Maybe its just a me problem though because I dont like luck in my euros. And you may call it randomness, but I definitely call it bad luck when I pick 4 dice and roll thee ones and one two when Im specifically in need of high numbers, and have already used up all my karma on my earlier lousy rolls.',
 'My experience with the game is the opposite of the OP. We bought it shortly after we played it. We enjoy dice rolling games, even when we roll bad (my wife is the queen of bad rolling), but she enjoys this one with me. Too bad you (OP) didnt like it, but to each his own.',
 'Thanks for your post. Always good to read a less favourable but well presented opinion. I enjoy Rajas, but for me the strength of the markets and getting the right dice to grab a set is more of an issue than simply pip value. Markets can be really strong and theres this forced tension of feeling like you have to get market sets otherwise someone else will and theyll spin up an engine and pull away. Rather than have other directions to go in. It comes up in various threads and Im not sure Ive ever been completely satisfied by the arguments that the game is fine as is. So I find it hard to grab it off the shelf. The problem may be mine. I tend to play games in a way where I avoid strategies that look obviously strong. I dont know why, but I just dont want to be doing the obvious moves all the time because then I dont really have any agency. Its scripted. The last few games of ignoring an obvious market grab have left me way behind the pace, watching people accumulate money quickly whilst my engine stalls. The argument seems to be that Rajas isnt a game of diverse strategies. Its a game of balance. Sometimes you have to take those tiles because otherwise you are being suboptimal and leaving a great move for someone else. And I guess I should play with that mindset and enjoy the tension around the scramble for good tiles. But to bring this full circle, therein lies the luck part of the game for me. If a great tile comes up and I convert one dice to two of the colour I need and I get low values, its taken too long to nab the tile before someone else snaffles it. Sure theres karma and keeping a good dice pool, but as the OP alludes to, sometimes it feels like youve been situationally shafted when it really does matter that you roll high. All of this does make me want to play it again though lol',
 'I enjoy Rajas, but for me the strength of the markets and getting the right dice to grab a set is more of an issue than simply pip value. Markets can be really strong and theres this forced tension of feeling like you have to get market sets otherwise someone else will and theyll spin up an engine and pull away. Rather than have other directions to go in.',
 'In my opinion, this is a bit of a problem and people often make weak arguments when dismissing it that dont really hold up in practice. I think the main issue is that there are a lot of opportunities to use your markets over and over again while simultaneously making the action stronger every round, but not that many opportunities to make your building scoring ramp up in the same way. You can counter it a little bit by blocking market spaces, but this can also be to your detriment because you effectively lose a turn if you arent also going for markets. I found that some of the mini-expansions fix this issue by simply introducing more options to upgrade your building scoring throughout the game. They also mix in some powerful tiles that make the other player think twice about blindly going for the high value market tiles, because then they might leave you a tile which has good synergy with fame scoring, leaving you to create a similar runaway scenario like they are trying to do with their high value market tiles. Ive already played one three-player game where my opponent completely ignored me while building his wealth engine, and in the final two rounds I ran nearly two-thirds of the fame track and won by a landslide.',
 'I have to disagree. There is a constant flow of dice and I cannot see how you can be unlucky all the time. There are 2 powerful strategies going for markets or going for buildings. In a 2 player games if you go for the opposite of what your oponent is doing or focus on one of those strategies if your oponent is doing a little bit of both you have many chances to win. You need to use wisely your buildings and your workers. I will give you that sometimes you run out of dice and luck, but you have to build up your engine and understand that your oponents will also eventually run out of dice and luck. I mean, you roll tens of dice in this game, it is statistically impossible to always run out of luck. Seems to me that you have to give it another chance and find a strategy that fits your play. In the end if you lose by a little bit for a bad roll at least you had a great time competing.',
 'My take away is that you should stay away from games with dice.',
 'Well thats good advice to avoid disappointment, but is a case of throwing the baby out with the bath water. Theres a whole spectrum of how much the dice in games can affect the outcome. At one end would be something like roll a 1-3 you lose, 4-6 you win. At the other would be a game where the dice create chaos but in a perfectly balanced way. I agree that Rajas is more to the latter end, but I do think that my biggest gripe (the importance of being able to get the set of market tiles) is exacerbated by needing certain values of certain dice at the right time. If the markets felt a little less required then Id be way more willing to accept that enough dice are rolled that whoever navigates the chaos, by whatever route, is more often than not the worthy winner. So in my view the dice arent the problem per se, but combined with the requirement for certain values at important times, they can cause frustration, exactly per the OPs reference to the player board engine. This is a light to medium euro though, and for many all of that isnt going to hamper their enjoyment. Like I said before, all this does have me thinking I should give it a run again. And try to suppress my need to not be cheesy. It could be my bloody minded refusal to do the obvious thing in the hope theres more divergent paths thats the issue. Maybe its just a narrower game than I wanted it to be and preparing for those tiles and exploiting them, along with all other players trying to do the same, will create the natural balance im looking for regardless of dice rolls.',
 'If you think the game is all about the player board and only high dice rolls matter than you are not analyzing the game deep enough. You are just looking at what on the surface level of what you think is most important. The river bonuses and palace spots are very strong. The two tracks are different because they have different bonuses along the way. Money gives you a lot of bonuses throughout the game. Fame gives you additional ability to get fame early on and access to another worker. The way you play against the opponent is to look at their board and colorvalue of dice and use that to anticipate what they may be trying to do. Then either get there first or play around it.',
 'Rajas is one of my favourite games. What I like about it is that you still have options if you get low dice values. The number Two palace spot is very strong as is the river. Also you have Karma to mitigate the dice you get. Dont forget tile overbuilding which can be very strong in certain circumstances as you only have to pay the difference. For the different market space you dont even need a dice. It shines at higher player counts.',
 'rizzou No, you are right, and I certainly dont think its only about high rolls. Nor do I think the player board is the only consideration. I get the variety of options, and i know the river can combo very nicely with other stuff. I have sympathy for the OP, even if I dont agree entirely. From around a dozen plays, mostly 2p but not exclusively, in the vast majority of games the player that exploited the assorted market the most won. Thats my issue. So subjectively, but with some supporting data, tiles that provide a variety of market goods of 2 or 3 coins each are more important. And they require certain dice colours and values. And therefore there will be occasions where I have to spend a couple of turns working to get the dice pips I need to get the tiles I really want, and I dont get them. In a 2 player game that can be swingy if you miss and they hit. The OP had quickly identified situations where they wanted certain values and were frustrated not getting them. I agree that in many of those situations there were probably equally good options away from building tiles, but Id also bet some of those situations were where they wanted those juicy market tiles and our thoughts on the game align. grandpawizard had said theres rarely a runaway winner and again thats not my experience where one player is able to get ahead with those markets. Many times I was happy to call the game 23 of the way through because it was crystal clear who would win. But some of this is personal taste. A lot of people love this game. And I have enjoyed the puzzle at times even though I felt it was a touch flawed as a competition. Just the other day I was reading Carolines recent blog post (subscribe if you havent already people!) where Rajas got a good mention. I for example really enjoy Spacecorp, a game that takes 2.5 hrs and has objectively bad tile draws. Why am I ok with that and less inclined to let it slide here? Who knows? We are a funny bunch! CarolineBlack - perhaps player count is part of the problem. I only had a handful of games at higher counts and pretty much every one of those games had one or two new players. Maybe the game needs players to know about, and to fight for, the market goods tiles to spread them around and keep those who dont get them in touch. Much in the same way auction games often dont work if one or more of the players are inexperienced and throw out the balance.',
 'Thanks for your post. Always good to read a less favourable but well presented opinion. I enjoy Rajas, but for me the strength of the markets and getting the right dice to grab a set is more of an issue than simply pip value. Markets can be really strong and theres this forced tension of feeling like you have to get market sets otherwise someone else will and theyll spin up an engine and pull away. Rather than have other directions to go in. It comes up in various threads and Im not sure Ive ever been completely satisfied by the arguments that the game is fine as is. So I find it hard to grab it off the shelf. The problem may be mine. I tend to play games in a way where I avoid strategies that look obviously strong. I dont know why, but I just dont want to be doing the obvious moves all the time because then I dont really have any agency. Its scripted. The last few games of ignoring an obvious market grab have left me way behind the pace, watching people accumulate money quickly whilst my engine stalls. The argument seems to be that Rajas isnt a game of diverse strategies. Its a game of balance. Sometimes you have to take those tiles because otherwise you are being suboptimal and leaving a great move for someone else. And I guess I should play with that mindset and enjoy the tension around the scramble for good tiles. But to bring this full circle, therein lies the luck part of the game for me. If a great tile comes up and I convert one dice to two of the colour I need and I get low values, its taken too long to nab the tile before someone else snaffles it. Sure theres karma and keeping a good dice pool, but as the OP alludes to, sometimes it feels like youve been situationally shafted when it really does matter that you roll high. All of this does make me want to play it again though lol',
 'Well articulated. This is quite literally what happened to me. I just couldnt get a good market set!',
 'My take away is that you should stay away from games with dice.',
 'There are plenty of games that I love that use dice well. Most notable Castles of Burgundy, which may not be a perfect system but does offer much better ways to mitigate bad dice rolls.',
 'rizzou No, you are right, and I certainly dont think its only about high rolls. Nor do I think the player board is the only consideration. I get the variety of options, and i know the river can combo very nicely with other stuff. I have sympathy for the OP, even if I dont agree entirely. From around a dozen plays, mostly 2p but not exclusively, in the vast majority of games the player that exploited the assorted market the most won. Thats my issue. So subjectively, but with some supporting data, tiles that provide a variety of market goods of 2 or 3 coins each are more important. And they require certain dice colours and values. And therefore there will be occasions where I have to spend a couple of turns working to get the dice pips I need to get the tiles I really want, and I dont get them. In a 2 player game that can be swingy if you miss and they hit. The OP had quickly identified situations where they wanted certain values and were frustrated not getting them. I agree that in many of those situations there were probably equally good options away from building tiles, but Id also bet some of those situations were where they wanted those juicy market tiles and our thoughts on the game align. grandpawizard had said theres rarely a runaway winner and again thats not my experience where one player is able to get ahead with those markets. Many times I was happy to call the game 23 of the way through because it was crystal clear who would win. But some of this is personal taste. A lot of people love this game. And I have enjoyed the puzzle at times even though I felt it was a touch flawed as a competition. Just the other day I was reading Carolines recent blog post (subscribe if you havent already people!) where Rajas got a good mention. I for example really enjoy Spacecorp, a game that takes 2.5 hrs and has objectively bad tile draws. Why am I ok with that and less inclined to let it slide here? Who knows? We are a funny bunch! CarolineBlack - perhaps player count is part of the problem. I only had a handful of games at higher counts and pretty much every one of those games had one or two new players. Maybe the game needs players to know about, and to fight for, the market goods tiles to spread them around and keep those who dont get them in touch. Much in the same way auction games often dont work if one or more of the players are inexperienced and throw out the balance.',
 'As you said, the game depends on players knowing which tiles are good and fighting over them. In fact, in a 2p game you have less reason to say you find yourself locked out of a good tile because the available tiles only change by at most 1 between ur turns. It should be apparent when your opponent will get the good tile before you (and you should plan for another good tile). Regarding the 3 market location, you have 50 first pick. Unless your opponent picked the 1 dice location which I dont find a good use most of the time because you can convert it to a 6 and jump 6 River spaces. Compelling to get markets, yes, but not OP. Yes runaway games are possible because this game is maybe 70 skill.',
 'The market being over-centralizing to me is a feature of the game, not a bug. Many euro gamers seem to have the expectation that each path of victory needs to be perfectly balanced. But there is nothing wrong with having certain paths be more desirable and thus skew the game towards them. Particularly in a worker placement game where you can strategize and fight over it, it just adds a different dimension to the game rather than just letting everyone spread out and avoid each other.',
 'My current rating for this game is 710 (Good Usually Willing to Play). Rajas helped my partner and I to try heavier euros because even though it is not a very complex game, it looks a bit overwhelming. It kind of demystified euros in our household. Good game, I have not tried the expansions yet, but I would like to.',
 'Mango Village is okay. If u want to play a more forgiving game, try it. Those tiles require two dice of same number so even your double 1s can be put to good use. And also, the tiles are 4-way. I personally prefer a tighter game with trade-off between good market tile (low dice total but 2 or 3 way) and 4-way bad tiles.']

In [ ]:
rev_3 = [texto.replace('\n',' ') for texto in rev_3]
rev_3

['I have only played Rajas once at 2p and I must say that I found it to be an utterly underwhelming and overly frustrating experience. The problem with Rajas is that it overwhelmingly relies on luck. Not the sort of fun luck youll find in Camel Up or Quacks of Quedlinburg but the sort of luck that makes you wanna flip the table. There is no question that the most important aspect of the game is your personal player board. This is your engine that drives your money and karma production and if it isnt impressive, youre simply not gonna win. How does one make an impressive player board? With lots of dice pips, which means you either need a lot of dice or to roll well. If your opponent rolls better that you or happens to pick up all of the bonus tiles that award extra dice they will leave you for dead, and thats exactly what happened in my game. Sure there are ways to mitigate poor dice rolls with the use of karma but it just isnt enough as karma itself is costly and hard to come by. In my

In [ ]:
rev_4 = [texto.replace('\n',' ') for texto in rev_4]
rev_4

['2-4 players 60-90 minutes medium Tile laying, worker placement Theme and Mechanics In Rajas of the Ganges, players are rich noble type people from India (please forgive my complete ignorance of Indian culture for this review, I am not well informed about this particular theme) and theyre working to improve their estate and earn the most money and prestige, fame, points, something like that to win the game. The theme really does come through, especially with the tile layingbuilding up their estates. Its hard for me to comment too much on theme and mechanics relating, because Im not too familiar with this topic, but it does seem like the mechanics are somewhat abstracted. It also seems like the setting could be changed without effecting game play, which makes the theme feel somewhat disconnected at times. Players each have an estate board with their palace on it, and several empty spaces, as well as a place to store dice, and 3 workers. There are 2 tracks around the outside boarder of 

In [ ]:
rev_4 = ['2-4 players 60-90 minutes medium Tile laying, worker placement Theme and Mechanics In Rajas of the Ganges, players are rich noble type people from India (please forgive my complete ignorance of Indian culture for this review, I am not well informed about this particular theme) and theyre working to improve their estate and earn the most money and prestige, fame, points, something like that to win the game. The theme really does come through, especially with the tile layingbuilding up their estates. Its hard for me to comment too much on theme and mechanics relating, because Im not too familiar with this topic, but it does seem like the mechanics are somewhat abstracted. It also seems like the setting could be changed without effecting game play, which makes the theme feel somewhat disconnected at times. Players each have an estate board with their palace on it, and several empty spaces, as well as a place to store dice, and 3 workers. There are 2 tracks around the outside boarder of the board going in opposite directions (money and fame). The player who can make these to tracks cross eachother the farthest wins the game. On their turn, players send one of their workers to one of several actions spaces (your standard worker placement rules) These actions are broken down into different areas that have similar effects; purchasing tiles for the estate, moving up the river track for various bonuses, gaining dice, gaining income, or using a specialist. Many of these actions have a money andor dice cost. The dice are used like resources. When they are gained, they are rolled and stored. There are 4 different colored dice that can be used for different actions in different ways. The tile laying action uses dice as a cost; each tile has a value and a color cost, and player must pay that many pips in that color to gain the tile. Moving up the river cost dice, the higher the number, the more spaces you can move. There are 6 action spaces (numbered 1-6) that require a specific number in any color to use, the marketincome space allows you to gain income from x number of markets (where x is the value of the die spent) and so on. There are also several action spaces that allow you to trade a die of one color for 2 dice of another color. There are several ways to manipulate the dice, and there are some good things you can do with bad dice rolls. The main focus of the game is the tile laying. Tiles will have markets (3 types) and buildings (4 types) that will score money and fame respectively when the tile is laid. The markets will also provide income when the market action is taken. The buildings will score fame based on the players progress on that buildings track. The tiles also dipict roads in various configurations. All roads must lead back to the palace in order to be legally placed. The perimiter of the player boards also have bonuses that are gained when a player connects the bonus to the palace with roads. each of the 3 tracks (money, fame, and river) offer a myriad of bonuses for certain points along the track (or every space in the case of the river) from improving buildings, fame, money, dice, karma (which is spent to flip dice over), bonus tiles, and extra workers. Art and Components This game looks fantastic. The board is absolutely beautiful, the box looks fantastic. All the art on everything in the game is stellar; very near to the best art in my game collection in my opinion. I have heard some say that the board looks very busy, and that that is confusing. I can understand that complaint, but I dont agree. It is far less confusing once you learn the game, and it just looks SO good. The quality of the cardboard tiles and board are standard good quality cardboard. There are no cards. The dice are bigger, translucent, good quality plastic dice with the rounded corners. The wooden worker pieces are cool looking, unique meeples, but Ive heard that they are not consistently cut, and my copy is no exception. Several of them are leaning (feet not cut straight) and one is missing half its head, and a friends copy had similar problems, but these issues are fairly minor. Theres a little multi-armed statue (ganesh? pardon my ignorance, again) that serves no other purpose but to hold your dice for you, but its really cool, and keeps the confusion low as to what dice are in your supply, and which were spent, and it just looks cool. Replability and Expansions The game is very replayable as is, and it comes with some variants and tokens to increase the variability; from changing the bonuses on the river spaces, to limiting dice storage (making the game more strategic), to changing the estate bonuses, and I would suggest using the variants in the rulebook right from the beginning for experienced gamers, because they increase the strategic choices in the game. There is one mini expansion,',
 ', which adds some unique tiles that came in a gaming advent calendar. I have no experience with this expansion, but as soon as I found out about it, I ordered it (which was literally as I was writing this review) The Up Side This is a very fun worker placementtile laying game. I think it combines those two mechanisms beautifully. It is very strategic, and all the pieces (mechanically) interlock very well. Its fairly easy to pivot strategies when necessary, and there is usually a way to figure out how to do what needs be done when your actions are taken by other players, but it usually takes some creative thinking. The game is absolutely gorgeous. The Down Side Im having trouble coming up with a down side. I guess I wish there were more tiles, or more variety on the tiles, like maybe an extra mechanism included in the tile laying portion. You wont go through all the tiles in a single game, but you also wont be surprised by cool new tiles showing up, because theyre all essentially the same (some roads, a building or two, or a market or two). Very minor complaint. Another complaint Ill mention, but do not agree with, is that the board is too busy looking. I wish it could play 5 players. Final Thoughts I absolutely love this game. This is one where Ill always be happy to play it, and Ill insta-buy any expansion sight unseen. Its the perfect weight and length, very little down time, lots of meaningful decisions and a unique scoring mechanism (which isnt really score at all) 9 out of 10 httpswww.coolstuffinc.commainsearch.php?pasearchOnName...',
 'Thanks for the review. A friend just bought this, so I hope to be playing it quite soon...',
 'Thanks for the review. A friend just bought this, so I hope to be playing it quite soon...',
 'Ive found that for some reason it doesnt appeal to people. Im not sure why, its absolutely beautiful, but people in my group werent keen to try it, but after they did, it was pretty unanimously loved. Hope you enjoy it!']

In [ ]:
rev_4 = [texto.replace('\n',' ') for texto in rev_4]
rev_4

['2-4 players 60-90 minutes medium Tile laying, worker placement Theme and Mechanics In Rajas of the Ganges, players are rich noble type people from India (please forgive my complete ignorance of Indian culture for this review, I am not well informed about this particular theme) and theyre working to improve their estate and earn the most money and prestige, fame, points, something like that to win the game. The theme really does come through, especially with the tile layingbuilding up their estates. Its hard for me to comment too much on theme and mechanics relating, because Im not too familiar with this topic, but it does seem like the mechanics are somewhat abstracted. It also seems like the setting could be changed without effecting game play, which makes the theme feel somewhat disconnected at times. Players each have an estate board with their palace on it, and several empty spaces, as well as a place to store dice, and 3 workers. There are 2 tracks around the outside boarder of 

## Video review
Se descargó el audio del video del siguiente link: https://www.youtube.com/watch?v=mSRN0DShdFM a fin de procesarlo más rápido que si fuese un video.

In [ ]:
def split_audio(file_path, num_parts):
        '''
        Descripción:
        Divide un archivo de audio en varias partes iguales. Cada parte es exportada como un archivo WAV individual.

        -------------------------------------------------------------------------------------------------

        Parámetros:
        - file_path (str): Ruta del archivo de audio en formato WAV que se desea dividir.
        - num_parts (int): Número de partes en las que se debe dividir el archivo de audio.

        -------------------------------------------------------------------------------------------------

        Retorna:
        - list: Lista de rutas de los archivos de audio resultantes, cada uno correspondiente a una parte del audio original.

        -------------------------------------------------------------------------------------------------

        Procedimiento:
        1. Carga el archivo de audio en formato WAV utilizando `AudioSegment.from_wav()`.
        2. Calcula la duración total del audio y divide esa duración entre el número de partes deseadas (`num_parts`).
        3. Itera a través del número de partes, cortando el audio en fragmentos de duración `part_duration`.
          - Para cada parte, calcula el inicio y el final, asegurando que la última parte abarque hasta el final del audio.
        4. Exporta cada parte como un archivo WAV con un nombre de formato `part_X.wav`.
        5. Añade la ruta de cada archivo de parte a la lista `parts`.
        6. Devuelve la lista de rutas de las partes generadas.
        '''

        audio = AudioSegment.from_wav(file_path)
        duration = len(audio)
        part_duration = duration // num_parts
        parts = []
        for i in range(num_parts):
            start = i * part_duration
            end = start + part_duration if i < num_parts - 1 else duration
            part = audio[start:end]
            part_path = f"part_{i+1}.wav"
            part.export(part_path, format="wav")
            parts.append(part_path)
        return parts

# Transcribir una parte de audio
def transcribe_audio(audio_file):
    '''
    Descripción:
    Transcribe el contenido de un archivo de audio a texto utilizando el servicio de reconocimiento de voz de Google.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - audio_file (str): Ruta del archivo de audio que se desea transcribir. El archivo debe ser compatible con el formato
      que la librería `SpeechRecognition` puede procesar (por ejemplo, WAV).

    -------------------------------------------------------------------------------------------------

    Retorna:
    - str: El texto transcrito del audio. Si no se puede entender el audio, retorna un mensaje indicando este hecho.
      Si hay un error de conexión con el servicio, retorna un mensaje de error.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Crea un objeto `Recognizer` de la librería `SpeechRecognition` para realizar la transcripción.
    2. Abre el archivo de audio utilizando `AudioFile` y carga el contenido del audio.
    3. Intenta transcribir el audio utilizando el servicio de reconocimiento de voz de Google.
    4. Si la transcripción es exitosa, devuelve el texto transcrito.
    5. Si no se puede entender el audio (por ejemplo, por ruido o problemas en la calidad), devuelve el mensaje
      "No se pudo entender el audio".
    6. Si ocurre un error al intentar conectar con el servicio de Google, captura el error y devuelve un mensaje
      indicando el problema de conexión.
    '''

    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        return "No se pudo entender el audio"
    except sr.RequestError as e:
        return f"Error al conectar con el servicio: {e}"

def process_and_transcribe_audio(audio_path, num_parts=4):
    '''
    Descripción:
    Divide un archivo de audio en varias partes, transcribe cada parte utilizando el servicio de reconocimiento de voz de Google,
    y luego junta todas las transcripciones en un único texto.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - audio_path (str): Ruta del archivo de audio que se desea procesar y transcribir. El archivo debe ser compatible con el formato que la librería `SpeechRecognition` puede procesar (por ejemplo, WAV).
    - num_parts (int, opcional): Número de partes en las que se debe dividir el archivo de audio. El valor predeterminado es 4.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - str: El texto transcrito del audio completo, concatenado de todas las partes procesadas.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Llama a la función `split_audio` para dividir el archivo de audio en el número de partes especificado (o por defecto, en 4 partes).
    2. Inicializa una cadena vacía `full_text` para almacenar el texto transcrito.
    3. Itera sobre las partes del audio:
      - Para cada parte, llama a la función `transcribe_audio` para obtener su transcripción.
      - Añade la transcripción a `full_text`.
      - Elimina el archivo temporal de la parte de audio procesada utilizando `os.remove`.
    4. Devuelve el texto completo, que es la concatenación de las transcripciones de todas las partes del audio.
    '''

    # Dividir el archivo en partes
    audio_parts = split_audio(audio_path, num_parts)

    # Transcribir cada parte y juntar el texto
    full_text = ""
    for part in audio_parts:
        full_text += transcribe_audio(part) + " "
        os.remove(part)

    return full_text

In [ ]:
# No se pudo descar el archivo
#texto_video_review = process_and_transcribe_audio("/content/tp-folder/data/review.wav")

In [ ]:
texto_video_review = "hello and welcome to Maple Mentor reviews I'm Jared and with me is Holly today we're reviewing rajas of the Ganges this game is from Hutch and R&R games and it's for 2 to 4 play in about an hour to hour and a half recommends ages 12 and up and this is a worker placement game set in the you know the the India River Ganges setting and you are all trying to gain wealth and fame as a exerting your influence over the region now you have three you can get a fourth or did I start with three get fourth you start with three workers you can gain up to two more but there's three ways that you can gain workers there's this is a game with two tracks there's an income track around one side and then a Fame on the other side and so they run parallel to each other in opposite directions and allows you to basically have this kind of race to increase your phone so that your two tokens will meet for pass and that's the race the end and whoever surpass their tokens the farthest wins the game so you know if you want to just kind of add up how many spaces do you have it is a worker placement game but there are a lot of dice in this game which are essentially used for resources the four colors of dice can be used for certain to pay for certain places to go like it might say pay an orange to go here and then it'll give you two purple dice back something like that or there is a section of the board called I think the palace where they're numbered 1 to 6 and you would spend one of your dice collected of that matching number to place a worker there there's also the purchase area where you can go and pay money to go there and then you collect a tile from those face up and available in the market there's three stacks for each color so 12 stocks and you have to spend the the right color dice that match what's showing on the tile and the value of that guy on the tile you have to have at least that many Pips on the dice you spend you can spend more than one die as you might need to specially if it's like a value 9 or 10 but any extra pits are lost so you just lose the dice but you add those tiles to your personal board because you're building out your own Palace from a center space and then orthogonally out following the roads that are also found on these new tiles there are bonuses you can get on the outside edges of the board if you can connect a roads to those bonuses you get more money more and whenever you play a new Temple in your Palace you get the score points based on where your truck so everyone starts on the far left of the track and throughout the game you can do things to move it forward which will give you more points whenever you add that Temple to your Palace another portion of the game play is you have a boat along the river itself and if you go to one of the space station I have one two or three to advance the boat that number spaces and where you land that's where you collect the benefit and one of the ways you can gain a new worker is by moving your boat for far enough on the river to get to the bridge and you get a worker that way you can also get a worker for advancing a number of spaces on the income or up to a certain point on the same but you can only ever get two more so Holly what are some things that you like about the game I like trying to make a pattern on your own player board making Pathways to get bonuses that will either help you through the game or in the end of the game I enjoy the function of being able to trade one color of ice for other colors that you may need it I don't know I really like dice certainly the colors in here are really bright and sparkly like they're translucent colored dice so you know if you like that and you like bright colors and rolling like trying to put together different you know it's different for x amount of points on my personal space over here so pretty much you're going to have a lot of options with all the values that you'll end up rolling but if there's a specific space that you're trying to get to or thing that you're trying to do and you've got the wrong numbers and things like to its opposite side so it turns one into a 6 for example or vice versa so that can be very useful to help both find you know the specific space on the palace you want to go to or if you're trying to get a high value to buy a tile or something like that the karma is a good feature and it's a bonus that you can gain with certain actions so you know you get it back yeah it's a lot of fun I like how it looks and you know plays pretty pretty pretty smoothly it just keeps going around and around until well there is the first player token so we do around but the first player token doesn't move unless someone take it as a lot of these games this is a unique kind of game and if you like tile placement it has that as well and collection of things it's you know it came out in 2017 and still plays great today so definitely check it out "
texto_video_review

"hello and welcome to Maple Mentor reviews I'm Jared and with me is Holly today we're reviewing rajas of the Ganges this game is from Hutch and R&R games and it's for 2 to 4 play in about an hour to hour and a half recommends ages 12 and up and this is a worker placement game set in the you know the the India River Ganges setting and you are all trying to gain wealth and fame as a exerting your influence over the region now you have three you can get a fourth or did I start with three get fourth you start with three workers you can gain up to two more but there's three ways that you can gain workers there's this is a game with two tracks there's an income track around one side and then a Fame on the other side and so they run parallel to each other in opposite directions and allows you to basically have this kind of race to increase your phone so that your two tokens will meet for pass and that's the race the end and whoever surpass their tokens the farthest wins the game so you know i

In [ ]:
!pip install youtube_transcript_api
from youtube_transcript_api import YouTubeTranscriptApi
transcript = YouTubeTranscriptApi.get_transcript('mSRN0DShdFM')
transcript

[{'text': 'thank you hello and welcome to meeple',
  'start': 7.22,
  'duration': 5.68},
 {'text': "Mentor reviews I'm Jared and with me is",
  'start': 10.08,
  'duration': 5.639},
 {'text': "Holly today we're reviewing rajas of the",
  'start': 12.9,
  'duration': 7.379},
 {'text': 'ganjis this game is from Hutch and r r',
  'start': 15.719,
  'duration': 7.261},
 {'text': "games and it's for two to four players",
  'start': 20.279,
  'duration': 4.801},
 {'text': 'plays in about an hour to hour and a',
  'start': 22.98,
  'duration': 3.08},
 {'text': 'half', 'start': 25.08, 'duration': 3.539},
 {'text': 'recommends ages 12 and up', 'start': 26.06, 'duration': 5.2},
 {'text': 'and this is a worker placement game set',
  'start': 28.619,
  'duration': 6.6},
 {'text': 'in the you know the the India River',
  'start': 31.26,
  'duration': 8.16},
 {'text': 'Ganges uh setting and you are but all',
  'start': 35.219,
  'duration': 8.641},
 {'text': 'trying to gain wealth and fame uh as a',

## Recapitulando
Se tiene información sobre las siguientes fuertes

In [ ]:
sections

{'OBJETIVO DEL JUEGO': 'Tu tarea es desarrollar tu provincia con la ayuda de tus trabajadores y el uso inteligente de tus dados acumulados. Al final, debes ganar la carrera con una combinación de riqueza y fama. El track de fama y dinero van paralelos entre sí, en direcciones opuestas alrededor del tablero. Los marcadores de fama se mueven en el sentido de las agujas del reloj, mientras que los marcadores de dinero en sentido contrario a las agujas del reloj. A medida que vas construyendo y expandiéndote, intenta aumentar tanto la fama como tu riqueza para que tu marcador de fama y dinero se crucen. El primer jugador que cruce o coincida su marcador de fama y su marcador de dinero aumentará sus probabilidades de ganar.4 marcadores de beneficio1 (doble cara) tablero (2 jugadores o 3 y 4 jugadores)4 Estatuas de Kali 4 tableros de provincia 24 trabajadores (6 por color) 4 marcadores de fama (1 por color)4 marcadores de dinero (1 por color)48 dados (12 por color) 1 elefante de jugador inic

In [ ]:
text_ref_rapida

'GANGES (2 a 4 jugadores) Antes de empezar Decidir si se juega en modo normal o en modo Navaratna (ver el epígrafe Variante Navaratna más adelante ). Sortear jugador inicial , el cual se va rotando en sentido horario cada vez que se agoten los lacayos colocados sobre las acciones del tablero, a menos que un jugador obtenga el beneficio de la primera zona de los aposentos del palacio, que lo convierte en jugador inicial. Cada jugador recibe , según su color 6 lacayos, de los cuales 3 se colocan en las 3 zonas al efect o del tablero (uno en la casilla 20 del marcador de riqueza, otro en la casilla 15 del marcador de gloria y otro sobre el puente del río). 5 cubos, de los cuales 4 se colocan sobre el primer nivel de cada uno de los edificios del tablero y el restante sobr e el nivel 1 del indicador de karma . 1 ficha circular de bonificación, la cual se coloca inicialmente sobre la casilla 12 del marcador de riqueza con la imagen de bonificación de avance fluvial boca arriba . 1 barco, qu

In [ ]:
text_quick_guide


'Rajas of the Ganges Set-Up Place the game board in the middle of the table, according to the player count. o If playing with 3 players, use the 2 space cover markers to cover the 1st Quarry space and the 3rd Harbor space. o If playing the standard game, put the Yellow, Brown and Red yield tokens, and the 8 river tokens, in the box. See below for details on the Navaratnas Version and the Ganga Module (2 variants to the game!) Sort the Province Tiles by colo r, and then by the 3 different a nimal ba cks (snake, cow, tiger). o Place the 12 stacks face -down next to the board, then flip the top tile of each stack face -up. Mix the 8 white Y ield tokens face -down on the temple space (white building). Each player takes o 1 Province board (place the sta ndard side up the side without blank yield spots) o 1 Kali Statue board (with the 2 slots in the middle open for dice able to hold 10 dice (thats all it does, fyi) ) o 6 Workers (lay 1 worker sideways on the river bridge, on the 15 fame spac

In [ ]:
text_automa_variant

'Rajas of the Ganges Automa mode Author Mauro Gibertoni Rev. 1.0 - 4 Dec 2017 This is a solo variant for Rajas of the Ganges boardgame. You play against a fake opponent the Automa. While you play following all standard rules, the Automa breaks some rules, so for you it will be very easy to take the action for the Automa. Set-up Prepare a standard set -up for wo players, but consider that the Automa doesnt need all the components, so you can leave in the box the following components - the province board ; - the four markers for palaces upgrade ; - the marker for karma points ; - the personal dice an d the Kali statue ; - if you play with Navaratnas version, select one brown yeld tile for you then remove 2 yeld tiles per color . Shuffle the Automa deck and place it close to the board with the blue side face up. Select randomly who will be the first player . Place the provice tiles in 12 piles and organize them in a 3 x 4 grid. Automas overview At each turn alternate your actions with the

In [ ]:
estrategia_portuguese


['Except for the case where you want to rush to get your worker on the bridge, I cannot understand why using a 6 on the Portuguese is useful. The sooner you race through the river, the less likely you are to benefit for those 1 river space bonuses and claim rewards caused by other events. How else could the Portuguese be useful?',
 'I use it frequently...like you mentioned, usually to rush and get my extra worker.',
 'Yeah like already was said if your strategy is not to use the river then get that worker as fast as possible for an advantage',
 'People tend to sit around on some of the early high value spaces, so jumping 6 ahead gets you into a cluster that no-one else is blocking you from. Also if you have the 1 presige for each 3 spaces moved along the river, then using the 6 spot is a given.',
 'People tend to sit around on some of the early high value spaces, so jumping 6 ahead gets you into a cluster that no-one else is blocking you from. Also if you have the 1 presige for each 3 

In [ ]:
estrategia_principiantes

'I have only played the game several times, but I think the best (and probably the most obvious) strategy is to build up your markets in your province as fast as possible, and then take the market action to score them frequently. Look and see which markets are available on the tiles and plan accordingly. The market action that allows you to score all three market types (one of each) does not cost a die, so go for diversity first if the tiles available are in that favor. Of course pick the markets with value 2 or 3 first. My last game I did not buy any market tiles that had a value of one (unless it has a building on it that will score well for fame.) Build up your building upgrades as fast as you can as well. In our second game my opponent ignored buildings and focused entirely on markets - and he lost. So do not ignore them. This is one of those games where you have so much you want to do each turn and you only get to do one thing! Also, get your extra meeple as soon as possible. And 

In [ ]:
guia_competidores


['Rajas of the Ganges is one of my favourite games. I like it because I see a way to work towards mastering it. I hope that this article will improve the gameplay of those with similar goals, as well as those on Yucata. This article is more tailored towards 2-player games (Navaratnas Ganga module) but many of the principles also apply to higher player counts. Revised in Feb 24 I incorporated new views based on my 10 games played as part of a Yucata Kings Court tournament. (2-players, round robin). The most significant is I now see the value of picking the Great Mogul (1). Besides updating the guide, I tried to improve the readability of the guide with ChatGPTs help. I am not very proficient with formatting on BGG, so I apologize in advance if it is still difficult to read! Rajas of the Ganges Strategy Guide Introduction Rajas of the Ganges holds a special place among my favorite games, as it provides a clear path to mastery. This article aims to enhance the gameplay for those with simi

In [ ]:
reviews_foro = [rev_1,rev_2,rev_3,rev_4]
reviews_foro

[['This is my first review for BGG. Ive really enjoyed Rajas so far so it seems like a great game to start with! I love midweight Euros and a big appeal of this game for me was the stated 60-75 minute play time. I often play games with coworkers at lunch and we have an hour to play. For that reason, I dont get to play heavier games like Great Western Trail very often. Games that have seen regular play lately include Rajas, Tiny Towns, Tournay, Jaws, Res Arcana and Century Spice Road. I bought Rajas of the Ganges a few weeks ago after months of delayed gratification... its been on my wishlist for a while. Ive spent a fair amount of time traveling in India so the theme was automatically a win for me. The art and components are fantastic. 48 dice of varying colors, wooden meeples, nice thick tiles for province building. I also really love the Kali component for storing dice. The only component Ive found lacking is the player 1 elephant - sure, its cute, but it easily falls apart. This is 

In [ ]:
texto_video_review


"hello and welcome to Maple Mentor reviews I'm Jared and with me is Holly today we're reviewing rajas of the Ganges this game is from Hutch and R&R games and it's for 2 to 4 play in about an hour to hour and a half recommends ages 12 and up and this is a worker placement game set in the you know the the India River Ganges setting and you are all trying to gain wealth and fame as a exerting your influence over the region now you have three you can get a fourth or did I start with three get fourth you start with three workers you can gain up to two more but there's three ways that you can gain workers there's this is a game with two tracks there's an income track around one side and then a Fame on the other side and so they run parallel to each other in opposite directions and allows you to basically have this kind of race to increase your phone so that your two tokens will meet for pass and that's the race the end and whoever surpass their tokens the farthest wins the game so you know i

## Carga información guardada

In [ ]:
# Guardar información

pd_info = {
    'sections' : sections,
    'text_ref_rapida' : text_ref_rapida,
    'text_quick_guide': text_quick_guide,
    'text_automa_variant': text_automa_variant,
    'estrategia_portuguese': estrategia_portuguese,
    'estrategia_principiantes': estrategia_principiantes,
    'guia_competidores': guia_competidores,
    'texto_video_review': texto_video_review,
    'reviews_foro': reviews_foro
}

with open(drive_path+'/datos/pd_info.pkl', 'wb') as f:
    pickle.dump(pd_info, f)

In [ ]:
with open(drive_path+'/datos/pd_info.pkl', 'rb') as f:
    pd_info_loaded = pickle.load(f)


In [ ]:
for texto in pd_info_loaded.values():
  print('----------texto------')
  print(texto)
  print('\n\n')

----------texto------
{'OBJETIVO DEL JUEGO': 'Tu tarea es desarrollar tu provincia con la ayuda de tus trabajadores y el uso inteligente de tus dados acumulados. Al final, debes ganar la carrera con una combinación de riqueza y fama. El track de fama y dinero van paralelos entre sí, en direcciones opuestas alrededor del tablero. Los marcadores de fama se mueven en el sentido de las agujas del reloj, mientras que los marcadores de dinero en sentido contrario a las agujas del reloj. A medida que vas construyendo y expandiéndote, intenta aumentar tanto la fama como tu riqueza para que tu marcador de fama y dinero se crucen. El primer jugador que cruce o coincida su marcador de fama y su marcador de dinero aumentará sus probabilidades de ganar.4 marcadores de beneficio1 (doble cara) tablero (2 jugadores o 3 y 4 jugadores)4 Estatuas de Kali 4 tableros de provincia 24 trabajadores (6 por color) 4 marcadores de fama (1 por color)4 marcadores de dinero (1 por color)48 dados (12 por color) 1 el

## Creación de base de dato vectorial

In [ ]:
# Cargar Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

**Pasar texto a chunk**

In [ ]:
def chuncker(text):
    '''
    Descripción:
    Divide un texto largo en fragmentos más pequeños, manteniendo una superposición entre ellos, utilizando un divisor de texto basado en el número de caracteres.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - text (str): El texto largo que se desea dividir en fragmentos más pequeños.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - list: Lista de fragmentos de texto, donde cada fragmento tiene un tamaño limitado, con una superposición de caracteres entre ellos.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Utiliza el `RecursiveCharacterTextSplitter` para dividir el texto en fragmentos de un tamaño máximo de 400 caracteres,
      con una superposición de 30 caracteres entre fragmentos.
    2. Llama al método `split_text` para dividir el texto en fragmentos según la configuración de tamaño y superposición.
    3. Devuelve la lista de fragmentos generados.
    '''

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=30)
    texts = text_splitter.split_text(text)
    return texts

In [ ]:
chunk_reglas_generales = []
## se agregan las reglas extraidas del pdf de reglas original
for section, texto in sections.items():
    if section not in ['OBJETIVO DEL JUEGO','CRÉDITOS', 'NOTAS','MOGUL LEXIGON', 'VERSIÓN NAVARATNAS']:
        chunk_reglas_generales.append((chuncker(texto), section))


## se agregan las reglas extraidas del pdf guia rapida
chunk_reglas_generales.append((chuncker(text_ref_rapida), 'reglas guia rapida'))

## se agregan chunks guia rapida
chunk_reglas_generales.append((chuncker(text_quick_guide), 'reglas guia rapida'))

In [ ]:
chunk_reglas_generales

[(['El jugador que haya sacado el valor más bajo en los dados se convierte en el jugador principal. En caso de empate, el último jugador que comió comida india. El jugador inicial colocará el elefante junto a su estatua de Kali y pondrá su marcador de riqueza en el espacio 3 en el track de riqueza. El jugador a su izquierda 4, el siguiente 5, y así sucesivamente. El jugador que vaya último comenzará',
   'que vaya último comenzará con menos dinero. Coloca el tablero en el centro de la mesa con el lado correspondiente al número de jugadores (2 y 3 o 4). En una partida a 3 jugadores, cubre los e spacios marcadoscon l as losetas de coberturaPon los 48 dados junto al tablero de juego como reserva general. No es necesario Losetas de producción amarillas, rojas, y marrones (úsalas solo en la versión',
   '(úsalas solo en la versión Navaratnas) y las 8 losetas de río.Ordena las losetas de provincia, primero por colores y luego por los símbolos a sus espaldas (serpiente, vaca, tigre), así obte

In [ ]:
chunk_reglas_especificas = []
chunk_reglas_especificas.append((chuncker(sections['VERSIÓN NAVARATNAS']), 'VERSION NAVARATNAS'))
chunk_reglas_especificas.append((chuncker(text_automa_variant), 'VERSION AUTOMA'))

In [ ]:
chunk_reglas_especificas

[(['Después de haber jugado a la versión básica para uno o más jugadores, puedes optar por la versión Navaratnas para jugadores avanzados (Las 9 Gemas ver página 24, Mogul Lexicon Raja Man Singh I). Aplica estas modificaciones Puedes activar los 3 trabajadores inactivos. Así que puedes jugar con hasta 6 trabajadores. Juega con la parte trasera de la estatua de kali, pudiendo almacenar solo 8 dados.',
   'almacenar solo 8 dados. Juega por la otra parte del tablero de provincia. Aquí también, las producciones especiales están impresas en el borde pudiéndolas conectar con tu red de caminos. Sin embargo estas producciones son menores, pero se pueden mejorar durante el transcurso de la partida. Al comienzo de la partida, mezcla las loseta de producción marrón. Cada jugador elige una al azar, la revela',
   'elige una al azar, la revela y la coloca en uno de los dos espacios disponibles en tu tablero de provincia. Decide que producción especial sustituir en tu tablero de provincia por tu nue

In [ ]:
chunk_objetivo_juego = []
chunk_objetivo_juego.append((chuncker(sections['OBJETIVO DEL JUEGO']),'OBJETIVO DEL JUEGO'))
chunk_objetivo_juego

[(['Tu tarea es desarrollar tu provincia con la ayuda de tus trabajadores y el uso inteligente de tus dados acumulados. Al final, debes ganar la carrera con una combinación de riqueza y fama. El track de fama y dinero van paralelos entre sí, en direcciones opuestas alrededor del tablero. Los marcadores de fama se mueven en el sentido de las agujas del reloj, mientras que los marcadores de dinero en',
   'los marcadores de dinero en sentido contrario a las agujas del reloj. A medida que vas construyendo y expandiéndote, intenta aumentar tanto la fama como tu riqueza para que tu marcador de fama y dinero se crucen. El primer jugador que cruce o coincida su marcador de fama y su marcador de dinero aumentará sus probabilidades de ganar.4 marcadores de beneficio1 (doble cara) tablero (2 jugadores o 3',
   'tablero (2 jugadores o 3 y 4 jugadores)4 Estatuas de Kali 4 tableros de provincia 24 trabajadores (6 por color) 4 marcadores de fama (1 por color)4 marcadores de dinero (1 por color)48 da

In [ ]:
estrategia_principiantes


'I have only played the game several times, but I think the best (and probably the most obvious) strategy is to build up your markets in your province as fast as possible, and then take the market action to score them frequently. Look and see which markets are available on the tiles and plan accordingly. The market action that allows you to score all three market types (one of each) does not cost a die, so go for diversity first if the tiles available are in that favor. Of course pick the markets with value 2 or 3 first. My last game I did not buy any market tiles that had a value of one (unless it has a building on it that will score well for fame.) Build up your building upgrades as fast as you can as well. In our second game my opponent ignored buildings and focused entirely on markets - and he lost. So do not ignore them. This is one of those games where you have so much you want to do each turn and you only get to do one thing! Also, get your extra meeple as soon as possible. And 

In [ ]:
chunk_estrategias = []
chunk_estrategias = [(chuncker(texto), 'Estrategia personaje portuguese') for texto in estrategia_portuguese]
chunk_estrategias += [(chuncker(estrategia_principiantes), 'Estrategia principiantes') ]
chunk_estrategias += [(chuncker(texto) ,  'Estrategia Competitiva')for texto in guia_competidores]

In [ ]:
chunk_estrategias


[(['Except for the case where you want to rush to get your worker on the bridge, I cannot understand why using a 6 on the Portuguese is useful. The sooner you race through the river, the less likely you are to benefit for those 1 river space bonuses and claim rewards caused by other events. How else could the Portuguese be useful?'],
  'Estrategia personaje portuguese'),
 (['I use it frequently...like you mentioned, usually to rush and get my extra worker.'],
  'Estrategia personaje portuguese'),
 (['Yeah like already was said if your strategy is not to use the river then get that worker as fast as possible for an advantage'],
  'Estrategia personaje portuguese'),
 (['People tend to sit around on some of the early high value spaces, so jumping 6 ahead gets you into a cluster that no-one else is blocking you from. Also if you have the 1 presige for each 3 spaces moved along the river, then using the 6 spot is a given.'],
  'Estrategia personaje portuguese'),
 (['People tend to sit aroun

In [ ]:
chunk_opinones = []
chunk_opinones = [chuncker(texto[0]) for texto in reviews_foro]
chunk_opinones += chuncker(texto_video_review)

**Base vectorial**

In [ ]:

#Eliminar la colección "vec_db"
#client.delete_collection("vec_db")
#client.clear_system_cache()

In [ ]:
# Configurar el cliente de ChromaDB con persistencia
client = chromadb.Client(Settings(persist_directory=drive_path+"/bases_datos/vector_db"))

# Crear una colección llamada "vec_db"

vec_db = client.create_collection("vec_db")

In [ ]:
def lista_chunks(chunks_list):
    '''
    Descripción:
    Combina listas anidadas dentro de una lista principal, y filtra elementos cuyo tamaño sea menor a 10, devolviendo una lista plana con los elementos válidos.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - chunks_list (list): Una lista que puede contener sublistas o elementos individuales. Las sublistas serán aplanadas, y los elementos con longitud menor a 10 serán excluidos.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - list: Una lista plana que contiene todos los elementos de las sublistas que tengan un tamaño mayor o igual a 10, además de los elementos individuales que cumplen con este criterio.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Inicializa una lista vacía `lista_c` para almacenar los elementos resultantes.
    2. Itera sobre cada elemento de `chunks_list`:
      - Si el elemento es una sublista, agrega cada uno de sus elementos a `lista_c`.
      - Si el elemento no es una sublista y tiene una longitud mayor o igual a 10, se agrega directamente a `lista_c`.
    3. Devuelve la lista `lista_c` con los element
    '''

    lista_c = []
    for lista in chunks_list:
        if type(lista) == list:
            for i in lista:
                lista_c.append(i)
        elif len(lista)<10:
            continue
        else:
            lista_c.append(lista)
    return lista_c

In [ ]:
def fill_db(documents, metadata, base_id):
    '''
    Descripción:
    Procesa una lista de documentos y sus metadatos, actualizando los metadatos correspondientes y organizando los documentos en fragmentos adecuados para su almacenamiento en una base de datos.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - documents (list): Lista de documentos a procesar, que pueden ser tuplas (con texto y metadatos) o solo texto.
    - metadata (dict): Diccionario de metadatos asociados a los documentos, que se actualizarán durante el procesamiento.
    - base_id (int): Identificador base para el almacenamiento o referencia de los documentos procesados (aunque no se utiliza en el fragmento proporcionado).

    -------------------------------------------------------------------------------------------------

    Retorna:
    - None: La función no devuelve nada. Actualiza los metadatos de los documentos y organiza los textos internamente.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Inicializa un contador `i` para iterar a través de los documentos.
    2. Para cada documento en `documents`:
      - Si el documento es una tupla, extrae el texto y los metadatos asociados, procesando el texto con la función `lista_chunks` y asignando los valores de metadatos a las claves correspondientes.
      - Si el documento no es una tupla y contiene una lista de textos, procesa la lista con `lista_chunks` y toma el primer fragmento de texto.
      - Si el documento es un solo texto, lo asigna directamente a `texto`.
    3. La función no devuelve valor, pero actualiza el diccionario `metadata` con los valores correspondientes.
    '''

    i = 0
    for doc in documents:

        if type(doc) == tuple:
            texto_lista, metadata_val = doc
            texto = lista_chunks(texto_lista)
            print(texto)
            texto = texto[0]
            for key, value in metadata.items():
                metadata[key] = metadata_val


        else:
            texto_lista = doc
            if type(texto_lista) == list:
                texto = lista_chunks(texto_lista)
                texto = texto[0]
            else:
                texto = texto_lista



        print(metadata, texto_lista)


        if detect(texto) != 'es':
            texto = GoogleTranslator(source=detect(texto), target='es').translate(texto)

        current_id = f'{base_id}_{i}'  # Asegúrate de IDs únicos
        embedding = embed(texto).numpy().flatten().tolist() # Genera el embedding del documento

        # Depuración
        print(f"ID: {current_id}, Embedding Length: {len(embedding)}, Metadata: {metadata}, texto: {texto}")

        try:
            vec_db.add(
                documents=[texto],
                metadatas=[metadata],
                ids=[current_id],  # Asegúrate de que sea una lista
                embeddings=[embedding]  # Asegúrate de que sea una lista
            )
        except Exception as e:
            print(f"Error al agregar {current_id}: {e}")

        i += 1


In [ ]:
# Llamada a la función
fill_db(chunk_reglas_generales, {'Reglas del juego': 0}, 'RG')
fill_db(chunk_reglas_especificas, {'Reglas del juego':0}, 'RE')
fill_db(chunk_estrategias, {'Estrategias': 0}, 'EST')
fill_db(chunk_opinones, {'Opinion': 'opinion del juego'}, 'OP')
fill_db(chunk_objetivo_juego, {'Objetivo del juego': 0}, 'OBJ')

['El jugador que haya sacado el valor más bajo en los dados se convierte en el jugador principal. En caso de empate, el último jugador que comió comida india. El jugador inicial colocará el elefante junto a su estatua de Kali y pondrá su marcador de riqueza en el espacio 3 en el track de riqueza. El jugador a su izquierda 4, el siguiente 5, y así sucesivamente. El jugador que vaya último comenzará', 'que vaya último comenzará con menos dinero. Coloca el tablero en el centro de la mesa con el lado correspondiente al número de jugadores (2 y 3 o 4). En una partida a 3 jugadores, cubre los e spacios marcadoscon l as losetas de coberturaPon los 48 dados junto al tablero de juego como reserva general. No es necesario Losetas de producción amarillas, rojas, y marrones (úsalas solo en la versión', '(úsalas solo en la versión Navaratnas) y las 8 losetas de río.Ordena las losetas de provincia, primero por colores y luego por los símbolos a sus espaldas (serpiente, vaca, tigre), así obtendremos 

#Parte 3: Base Tabular
La base tabular tendrá como objetivo almacenar:

* Datos estadísticos: jugadores totales, por mes, etc.
* Dificultad del juego.
* Cantidad de fichas.
* Precio promedio entre sitios web.
* Tiempo de juego
* Edad recomendada.
* Puntuación.

In [ ]:
def obtener_estadisticas_y_valores(url):
    '''
    Descripción:
    Recoge estadísticas y valores de una página web utilizando Selenium, extrayendo información relevante sobre estadísticas de juegos y jugadores a partir de elementos HTML específicos.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - url (str): La URL de la página web de la cual se extraerán las estadísticas.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - list: Una lista de diccionarios, donde cada diccionario contiene una "Entidad" y su "Valor" correspondiente, representando una estadística de la página.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Configura el entorno de Selenium para utilizar Chrome en modo headless (sin interfaz gráfica) y con opciones específicas para evitar problemas de entorno.
    2. Abre la URL proporcionada y espera a que los elementos relevantes en la página sean visibles.
    3. Extrae las entidades (títulos) y los valores asociados de los elementos de la página:
      - Cada entidad es un título que describe un tipo de estadística.
      - Los valores son los números o datos asociados a cada entidad.
    4. Realiza una serie de transformaciones en los nombres de las entidades para estandarizarlos a términos más comprensibles.
    5. Almacena cada par de entidad y valor en la lista `resultados`.
    6. Si ocurre algún error al intentar extraer los datos de algún elemento, lo omite y continúa con el siguiente.
    7. Cierra el navegador y devuelve la lista de resultados con las estadísticas extraídas.
    '''

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    # Configurar la ruta del driver
    service = Service('/usr/bin/chromedriver')

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)
    # Crear una lista para almacenar los resultados
    resultados = []

    # Espera explícita para asegurarse de que la página cargue completamente
    try:
        # Espera a que los elementos <li> con la clase "outline-item" sean visibles
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.outline-item'))
        )

        # Encuentra todos los elementos <li> con la clase "outline-item"
        items = driver.find_elements(By.CSS_SELECTOR, 'li.outline-item')

        # Itera sobre los elementos y extrae la entidad y el valor
        for item in items:
            try:
                # Encuentra la entidad (titulo) dentro del div con clase "outline-item-title"
                entidad = item.find_element(By.CSS_SELECTOR, 'div.outline-item-title').text.strip()

                if entidad == 'All time Plays':
                    entidad ='Total of players'
                elif entidad == 'This Month':
                    entidad ='Players this Month'
                elif entidad == 'Own':
                    entidad ='Players who own the game'
                elif entidad == 'Prev. Owned':
                    entidad ='Players who owned the game before but not now'
                elif entidad == 'For Trade':
                    entidad ='Players who have the game for trade'
                elif entidad == 'Want In Trade':
                    entidad ='Players who want the game in trade'
                elif entidad == 'Wishlist':
                    entidad ='Players who have the game in their wishlist'
                elif entidad == 'Has Parts':
                    entidad ='Players who have parts of the game'
                elif entidad == 'Want Parts':
                    entidad ='Players who want parts of the game'

                # Encuentra el valor (rating) dentro del div con clase "outline-item-description"
                valor = item.find_element(By.CSS_SELECTOR, 'div.outline-item-description a').text.strip()

                # Almacena el resultado en el formato 'Entidad: Valor'
                resultados.append({'Entidad': entidad, 'Valor': valor})
            except Exception as e:
                # Si ocurre un error al extraer los datos de una entidad, lo omite
                pass
    except Exception as e:
        print(f"Error al esperar elementos: {e}")

    # Cierra el navegador
    driver.quit()

    # Devuelve la lista de resultados
    return resultados

In [ ]:
url_stats = 'https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges/stats'
valores = obtener_estadisticas_y_valores(url_stats)

In [ ]:
valores

[{'Entidad': 'Avg. Rating', 'Valor': '7.728'},
 {'Entidad': 'No. of Ratings', 'Valor': '14,865'},
 {'Entidad': 'Weight', 'Valor': '2.90 / 5'},
 {'Entidad': 'Comments', 'Valor': '2,153'},
 {'Entidad': 'Fans', 'Valor': '707'},
 {'Entidad': 'Overall Rank', 'Valor': '158'},
 {'Entidad': 'Strategy Rank', 'Valor': '123'},
 {'Entidad': 'All Time Plays', 'Valor': '60,340'},
 {'Entidad': 'Players this Month', 'Valor': '2'},
 {'Entidad': 'Players who own the game', 'Valor': '19,830'},
 {'Entidad': 'Players who owned the game before but not now',
  'Valor': '2,096'},
 {'Entidad': 'Players who have the game for trade', 'Valor': '194'},
 {'Entidad': 'Players who want the game in trade', 'Valor': '681'},
 {'Entidad': 'Players who have the game in their wishlist', 'Valor': '3,841'},
 {'Entidad': 'Players who have parts of the game', 'Valor': '13'},
 {'Entidad': 'Players who want parts of the game', 'Valor': '5'}]

In [ ]:
df_valores = pd.DataFrame(valores)
df_valores['Embedding'] = df_valores.apply(lambda row: embed(f"{row['Entidad']}: {row['Valor']}").numpy().flatten().tolist(), axis=1)
df_valores['Entidad'].value_counts()

,count
Entidad,
Avg. Rating,1
No. of Ratings,1
Weight,1
Comments,1
Fans,1
Overall Rank,1
Strategy Rank,1
All Time Plays,1
Players this Month,1


In [ ]:
# Guardamos la base de datos
ruta_valores_csv = drive_path+'/bases_datos/tabular_db.csv'
# df_valores.to_csv(ruta_valores_csv,index=False)

**carga de la base de datos tabular**

In [ ]:
df_valores = pd.read_csv(ruta_valores_csv)
df_valores.drop(columns=['Embedding'], inplace=True)
df_valores

,Entidad,Valor
0,Avg. Rating,7.729
1,No. of Ratings,"14,856"
2,Weight,2.90 / 5
3,Comments,"2,151"
4,Fans,707
5,Overall Rank,157
6,Strategy Rank,122
7,All Time Plays,"60,308"
8,Players this Month,391
9,Players who own the game,"19,824"


#Parte 4: modelos clasificadores

## Modelo binomial

In [ ]:
dataset_clasificador = pd.read_excel(drive_path+'datos/dataset_clasificador.xlsx')
dataset_clasificador.head()

,consulta,clasificación
0,¿Cuántas personas tienen el juego disponible p...,ESTADÍSTICA GENERAL DEL JUEGO
1,¿Cuántos jugadores desean obtener el juego med...,ESTADÍSTICA GENERAL DEL JUEGO
2,¿Cuántos jugadores tienen el juego en su lista...,ESTADÍSTICA GENERAL DEL JUEGO
3,¿Es posible ver cuántos jugadores tienen parte...,ESTADÍSTICA GENERAL DEL JUEGO
4,¿Cuántos jugadores están buscando partes del j...,ESTADÍSTICA GENERAL DEL JUEGO


In [ ]:
dataset_clasificador['clasificación'].unique()

array(['ESTADÍSTICA GENERAL DEL JUEGO', 'OBJETIVO DEL JUEGO',
       'ESTRATEGIA COMPETITIVA', 'REGLAS GENERALES',
       'ESTRATEGIA PRINCIPIANTES', 'REGLAS AUTOMATA', 'REGLAS NAVARATNA',
       'ESTRATEGIA PERSONAJE PORTUGUESE', 'OPINIONES',
       'INFORMACION DISEÑADORES', 'INFORMACION ARTISTA',
       'INFORMACION CREADORES DEL JUEGO', 'INFORMACION IDIOMAS DEL JUEGO',
       'RELACIONES ENTRE CREADORES', 'PUBLICADORES DEL JUEGO',
       'PREMIOS GANADOS DEL JUEGO',
       'OTROS IDIOMAS EN LOS QUE ESTA EL JUEGO',
       'INFORMACION ILUSTRADORES DEL JUEGO'], dtype=object)

In [ ]:
X = dataset_clasificador['consulta']
y = dataset_clasificador['clasificación']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_embed = X_train.apply(lambda x: embed(x).numpy().flatten())

X_test_embed =  X_test.apply(lambda x: embed(x).numpy().flatten())
X_train_embed = np.array(X_train_embed.tolist())
X_test_embed =  np.array(X_test_embed.tolist())

In [ ]:
# Entrenar un clasificador como Logistic Regression
clasificador_consulta = LogisticRegression(max_iter=1000, class_weight='balanced')
clasificador_consulta.fit(X_train_embed, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
y_pred = clasificador_consulta.predict(X_test_embed)


In [ ]:
print(classification_report(y_test, y_pred))


                                        precision    recall  f1-score   support

         ESTADÍSTICA GENERAL DEL JUEGO       1.00      1.00      1.00         3
                ESTRATEGIA COMPETITIVA       0.67      1.00      0.80         2
       ESTRATEGIA PERSONAJE PORTUGUESE       1.00      1.00      1.00         3
              ESTRATEGIA PRINCIPIANTES       1.00      0.86      0.92         7
                   INFORMACION ARTISTA       1.00      0.75      0.86         4
       INFORMACION CREADORES DEL JUEGO       0.00      0.00      0.00         3
         INFORMACION IDIOMAS DEL JUEGO       1.00      0.67      0.80         3
    INFORMACION ILUSTRADORES DEL JUEGO       0.00      0.00      0.00         1
                    OBJETIVO DEL JUEGO       1.00      1.00      1.00         1
                             OPINIONES       1.00      1.00      1.00         5
OTROS IDIOMAS EN LOS QUE ESTA EL JUEGO       0.50      1.00      0.67         1
             PREMIOS GANADOS DEL JUEGO 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Se puede observar que se tiene un f1-score relativamente aceptable para las clases, a excepción de objetivo el juego, clase que parece no estar diferenciando bien.

In [ ]:
clasificador_consulta.predict(embed(['Quien fue el creador del juego?']))


array(['INFORMACION ILUSTRADORES DEL JUEGO'], dtype=object)

In [ ]:
clasificador_consulta.predict(embed(['quien creo el juego?']))


array(['INFORMACION ILUSTRADORES DEL JUEGO'], dtype=object)

In [ ]:
clasificador_consulta.predict(embed(['Cual es el objetivo del juego?']))


array(['OBJETIVO DEL JUEGO'], dtype=object)

In [ ]:
clasificador_consulta.predict(embed(['Qué puntaje tiene el juego?']))


array(['ESTADÍSTICA GENERAL DEL JUEGO'], dtype=object)

##Modelo LLM.

In [ ]:
def get_class(frase: str)->str:
    # Tu clave API de Hugging Face

    api_key = config('HUGGINGFACE_TOKEN', userdata.get('HUGGINGFACE_TOKEN'))
    client_HF = InferenceClient(api_key= api_key)
    #client_HF = InferenceClient(api_key= get_token())


    # Lista de clases de categorías
    classes = [
        "ESTADISTICA GENERAL", 'DISEÑADORES', 'CREADORES', 'IDIOMAS DEL JUEGO', 'PUBLICADORES DEL JUEGO',
        'REGLAS GENERALES', 'REGLAS ESPECÍFICAS AUTOMATA', 'REGLAS ESPECIFICAS NAVARATNA', 'OBJETIVO DEL JUEGO',
        'ESTRATEGIA COMPETIDORES', 'ESTRATEGIA PRINCIPIANTES', 'ESTRATEGUIA PERSONAJE PORTUGUESE', 'AÑO DE CREACIÓN',
        'OTROS IDIOMAS'
    ]

   # Prompt para clasificación
    chat_prompt = [{
        "role": "system",
        "content": f"Eres un modelo de clasificación. La tarea es clasificar frases en una de estas categorías: [{', '.join(classes)}]."
                   " Dada una frase debes solo escribir la categoria clasificada sin escribir la respuesta en oración."
                   " Tienes prohibido dar clasificaciones por fuera de las clases mencionadas o escribir palabras que no sean las de las clases"
                   " Por ejemplo, si el usuario escribe 'Como puedo hacer para ganar?' debes devolver: ESTRATEGIA PRINCIPIANTES"
                   "Otro ejemplo: si el usuario escribe 'En qué año se hizo el juego? debes devolver: AÑO DE CREACIÓN "
                   "Por ejemplo si el usuario escribe 'Qué puntaje tiene el juego?' debes devolver: ESTADISTICA GENERAL"
    }, {
        "role": "user",
        "content": frase
    }]

     # Realizar la solicitud de completado de chat usando el cliente
    completion = client_HF.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=chat_prompt,
        max_tokens=50,
        temperature=0.1
    )


    # Imprimir solo la clasificación sin contexto adicional
    return completion.choices[0].message['content']

In [ ]:
get_class('Cual es el objetivo del juego?')


'OBJETIVO DEL JUEGO'

In [ ]:
get_class('Cuándo gana un jugador?')


'OBJETIVO DEL JUEGO'

In [ ]:
get_class('De que forma comienza el jugador a jugar?')


'REGLAS GENERALES'

In [ ]:
get_class('¿que puedo hacer para ganar?')


'ESTRATEGIA PRINCIPIANTES'

In [ ]:
get_class('¿Para que puedo usar al personaje portuguese?')


'ESTRATEGIA PERSONAJE PORTUGUESE'

In [ ]:
get_class('Como hago para jugar el juego?')


'REGLAS GENERALES'

In [ ]:
get_class('quienes crearon el juego?')


'CREADORES'

In [ ]:
get_class('Qué puntaje tiene el juego?')


'ESTADISTICA GENERAL'

In [ ]:
get_class('cuantos jugadores han jugado a lo largo de la historia')


'ESTADISTICA GENERAL'

Se puede ver un mejor rendimiento del llm que de la regresión logística por lo que se utilizara este en adelante para clasificar la temática del usuario y a partir de alli determinar a qué fuente de datos redirigir la consulta.

## Obtención de la base

In [ ]:
def get_database(clas):
    '''
    Descripción:
    Determina el tipo de base de datos (vector, gráfico o tabular) según una clase de temática dada.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - clas (str): La clase de temática que se quiere clasificar. Se utiliza para identificar si corresponde a una base de datos vectorial, gráfica o tabular.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - str: El tipo de base de datos correspondiente a la clase de temática. Puede ser 'vector_db', 'graph_db' o 'tabular_db'.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Define tres listas con las temáticas correspondientes a cada tipo de base de datos:
      - `tematica_vector_db` para bases de datos vectoriales.
      - `tematica_graph_db` para bases de datos gráficas.
      - `tematica_tabular_db` para bases de datos tabulares.
    2. Verifica en cuál de las listas se encuentra la clase proporcionada (`clas`).
    3. Si la clase pertenece a `tematica_vector_db`, retorna 'vector_db'.
    4. Si la clase pertenece a `tematica_graph_db`, retorna 'graph_db'.
    5. Si la clase pertenece a `tematica_tabular_db`, retorna 'tabular_db'.
    6. Si la clase no pertenece a ninguna de estas listas, la función no retorna nada, lo cual puede ser considerado como un caso por manejar.
    '''

    tematica_vector_db = ['REGLAS GENERALES', 'REGLAS ESPECÍFICAS AUTOMATA', 'REGLAS ESPECIFICAS NAVARATNA', 'OBJETIVO DEL JUEGO','ESTRATEGIA COMPETIDORES', 'ESTRATEGIA PRINCIPIANTES', 'ESTRATEGUIA PERSONAJE PORTUGUESE']
    tematica_graph_db = ['DISEÑADORES', 'CREADORES', 'IDIOMAS DEL JUEGO', 'PUBLICADORES DEL JUEGO', 'AÑO DE CREACIÓN', 'OTROS IDIOMAS']
    tematica_tabular_db = ["ESTADISTICA GENERAL"]

    if clas in tematica_vector_db:
        return 'vector_db'
    if clas in tematica_graph_db:
        return 'graph_db'
    if clas in tematica_tabular_db:
        return 'tabular_db'

In [ ]:
print(get_database(get_class('Qué puntaje tuvo el juego?')))


tabular_db


#**Parte 5:**

**Queries dinámicas para base de datos de gragos**

In [ ]:
entidades = [
    "Rajas_of_the_Ganges", "Inka_Brand", "Markus_Brand", "Dennis_Lohausen",
    "HUCH!", "999_Games", "Devir", "Dice_Realm", "DV_Games", "Egmont_Polska",
    "Fabrika_Igr", "Game_Harbor", "HOT_Games", "nostalgia (III)", "R&R_Games",
    "India", "Hinduism", "Ganges", "I Ragià del Gange", "Raja's van de Ganges",
    "Раджи Ганга", "ガンジスの藩王", "갠지스의 라자", "2019 - MinD-Spielepreis Complex Game Nominee",
    "2018 - Juego del Año Recommended", "2018 - International Gamers Award - General Strategy: Multi-player Winner",
    "2018 - International Gamers Award - General Strategy: Multi-player Nominee", "2017 - Meeples Choice Award Nominee", "2017"
]

# Lista de relaciones
relaciones = [
    ("Rajas_of_the_Ganges", "designedBy", ["Inka_Brand", "Markus_Brand"]),
    ("Rajas_of_the_Ganges", "artBy", ["Dennis_Lohausen"]),
    ("Rajas_of_the_Ganges", "publishedBy", [
        "HUCH!", "999_Games", "Devir", "Dice_Realm", "DV_Games", "Egmont_Polska",
        "Fabrika_Igr", "Game_Harbor", "HOT_Games", "nostalgia (III)", "R&R_Games"
    ]),
    ("Rajas_of_the_Ganges", "setIn", ["India"]),
    ("Rajas_of_the_Ganges", "religionAssociatedWith", ["Hinduism"]),
    ("Rajas_of_the_Ganges", "alsoKnownAs", [
        "Ganges", "I Ragià del Gange", "Raja's van de Ganges", "Раджи Ганга",
        "ガンジスの藩王", "갠지스의 라자"
    ]),
    ("Rajas_of_the_Ganges", "awardedOrNominated", [
        "2019 - MinD-Spielepreis Complex Game Nominee", "2018 - Juego del Año Recommended",
        "2018 - International Gamers Award - General Strategy: Multi-player Winner",
        "2018 - International Gamers Award - General Strategy: Multi-player Nominee",
        "2017 - Meeples Choice Award Nominee"
    ]),
    ("Rajas_of_the_Ganges", "Release_year", ["2017"])
]

In [ ]:
def get_query(input_text: str) -> str:
    """
    Genera una consulta SPARQL utilizando Qwen a partir de la entrada del usuario.
    """
    # Tu clave API de Hugging Face
    api_key = config('HUGGINGFACE_TOKEN', userdata.get('HUGGINGFACE_TOKEN'))
    client_HF = InferenceClient(api_key= api_key)
    # URL de la API de Hugging Face para la clasificación de texto
    api_url = "https://api-inference.huggingface.co/models/Qwen/Qwen2.5-72B-Instruct"

    chat_prompt = [{
        "role": "system",
        "content": f"""
        Eres un modelo que realiza consultas a una base de datos RDF sobre el juego de mesa "Rajas of the Ganjes" a raíz de una frase del usuario.
        La base tiene la siguiente estructura: las entidades son {entidades}, las relaciones son {relaciones}.
        Asegúrate de diferenciar bien entre diseñadores y creadores.
        Debes escribir SOLAMENTE UNA consulta URI con SPARQL que se adecue a la frase del usuario.
        Es importante que solo respondas con una sola consulta.
        Por ejemplo, si recibes la frase: '¿Cuándo se creó el juego?' deberías solamente
        devolver: "SELECT (STRAFTER(STR(?Year), 'http://wiki_rajas.org/') AS ?name) WHERE {{ ?game a <http://wiki_rajas.org/Game> . ?game <http://wiki_rajas.org/Release_year> ?Year }}"
        Es importante que respetes todos los corchetes, llaves y entidades luego de ?."""
    }, {
        "role": "user",
        "content": input_text
    }]

    # Realizar la solicitud de completado de chat usando el cliente
    completion = client_HF.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=chat_prompt,
        max_tokens=500,
        temperature=0.1
    )

    # Extraer y devolver la consulta SPARQL generada
    return completion.choices[0].message['content']

In [ ]:
# Obtener la consulta SPARQL generada
sparql_query = get_query("Quién publicó el juego?")

# Imprimir la consulta generada
print("Consulta SPARQL generada:")
print(sparql_query)

Consulta SPARQL generada:
SELECT (STRAFTER(STR(?Publisher), 'http://wiki_rajas.org/') AS ?name) WHERE { ?game a <http://wiki_rajas.org/Game> . ?game <http://wiki_rajas.org/publishedBy> ?Publisher }


In [ ]:
results = g.query(sparql_query)

In [ ]:
for resultado in results:
  print(resultado.name)

HUCH%21
999_Games
Devir
Dice_Realm
DV_Games
Egmont_Polska
Fabrika_Igr
Game_Harbor
HOT_Games
nostalgia_%28III%29
R%26R_Games


In [ ]:
for row in results:
    print(row)

(rdflib.term.Literal('HUCH%21'),)
(rdflib.term.Literal('999_Games'),)
(rdflib.term.Literal('Devir'),)
(rdflib.term.Literal('Dice_Realm'),)
(rdflib.term.Literal('DV_Games'),)
(rdflib.term.Literal('Egmont_Polska'),)
(rdflib.term.Literal('Fabrika_Igr'),)
(rdflib.term.Literal('Game_Harbor'),)
(rdflib.term.Literal('HOT_Games'),)
(rdflib.term.Literal('nostalgia_%28III%29'),)
(rdflib.term.Literal('R%26R_Games'),)


In [ ]:
results

**Búsqueda vector database**

In [ ]:
lista_chunks_vec_db = vec_db.get()['documents']

In [ ]:
lista_chunks_vec_db

['El jugador que haya sacado el valor más bajo en los dados se convierte en el jugador principal. En caso de empate, el último jugador que comió comida india. El jugador inicial colocará el elefante junto a su estatua de Kali y pondrá su marcador de riqueza en el espacio 3 en el track de riqueza. El jugador a su izquierda 4, el siguiente 5, y así sucesivamente. El jugador que vaya último comenzará',
 'El Juego se desarrolla en varias rondas, cada una de las cuales consta de varias etapas. En cada etapa, los jugadores, por turno - en sentido de las agujas del reloj, comenzando por el jugador inicial - podrá colocar un trabajador, pagar costes, si es necesario, e inmediatamente llevar a cabo la acción asociada. Este proceso continua hasta que nadie pueda colocar trabajadores. Después de eso cada',
 'Coloca una loseta de provincia en tu provincia y obtén puntos de fama yo dinero. Para desarrollar tu provincia, necesitas enviar trabajadores a la cantera. dependiendo del lugar donde coloque

In [ ]:
# Función para tokenizar en español
def tokenize_spanish(chunks: List[str]) -> List[List[str]]:
    """
    Tokeniza una lista de textos en español, dividiendo cada texto en palabras individuales
    y filtrando los caracteres no alfanuméricos.

    Utiliza el tokenizador de la librería NLTK para procesar textos en español.
    Para cada fragmento de texto, se obtienen los tokens (palabras) en minúsculas y se descartan
    los caracteres no alfanuméricos (como puntuación).

    Parámetros:
    - chunks (List[str]): Una lista de cadenas de texto en español que se desean tokenizar.

    Retorna:
    - List[List[str]]: Una lista de listas, donde cada sublista contiene las palabras tokenizadas
      (en minúsculas y sin caracteres no alfanuméricos) de cada fragmento de texto de entrada.
    """
    tokenized_chunks = []

    for text in chunks:
        tokens = word_tokenize(text, language='spanish')
        tokenized_chunks.append([word.lower() for word in tokens if word.isalnum()])

    return tokenized_chunks

# Función para obtener los mejores resultados de BM25
def extract_bm_25(query_text: str, top_n: int = 5) -> List[str]:
    """
    Extrae los documentos más relevantes utilizando el modelo BM25 basado en una consulta de texto.

    Parámetros:
    - query_text (str): La consulta de texto que se desea usar para la búsqueda.
    - top_n (int, opcional): El número de documentos más relevantes que se desean obtener. El valor predeterminado es 5.

    Retorna:
    - List[str]: Una lista de tuplas, cada una con un documento y su puntuación BM25 asociada,
      correspondiente a los documentos más relevantes.

    Ejemplo:
    >>> extract_bm_25("inteligencia artificial", top_n=3)
    [("documento1", 3.45), ("documento2", 2.89), ("documento3", 2.76)]

    Requiere:
    - Una lista global `lista_chunks_vec_db` que contenga los documentos de la base de datos,
      y la función `tokenize_spanish` que realiza la tokenización de los textos.
    """

    # Crear el modelo BM25
    tokenized_docs = tokenize_spanish(lista_chunks_vec_db)
    bm25 = BM25Okapi(tokenized_docs)

    # Tokenizar la consulta
    tokenized_query = tokenize_spanish([query_text])[0]

    # Obtener las puntuaciones BM25 para la consulta
    scores = bm25.get_scores(tokenized_query)

    # Ordenar los resultados según las puntuaciones BM25
    sorted_indices = np.argsort(scores)[::-1]
    top_indices = sorted_indices[:top_n]

    return [(lista_chunks_vec_db[i], scores[i]) for i in top_indices]

def semantic_search(query_text: str, top_n: int = 5):
    """
    Realiza una búsqueda semántica en una base de datos de vectores, utilizando la consulta proporcionada
    y devolviendo los documentos más relevantes según su similitud con la consulta.

    La función genera un embedding para la consulta de texto utilizando un modelo de embeddings, luego realiza
    una búsqueda en la base de datos de vectores (como chromadb), recuperando los documentos más cercanos
    y sus respectivas distancias a la consulta.

    Parámetros:
    - query_text (str): El texto de la consulta que se utilizará para la búsqueda.
    - top_n (int, opcional): El número de resultados más relevantes que se desean obtener. El valor predeterminado es 5.

    Retorna:
    - List[Tuple[str, float]]: Una lista de tuplas, cada una conteniendo un documento y su distancia
      respecto a la consulta, ordenados por relevancia.
    """
    # Consultar los embeddings de la consulta
    embed_consulta = embed([query_text]).numpy().tolist()

    # Realizar la consulta en la base de datos de vectores (chromadb)
    query_consulta = {
        'query_embeddings': embed_consulta,
        'n_results': top_n,  # Limitar los resultados a `top_n`
    }

    # Consultar en la base de datos
    results = vec_db.query(**query_consulta)

    # Obtener documentos y distancias
    documents = results.get('documents')
    distances = results.get('distances')

    # Combinar documentos y distancias en una lista de tuplas
    combined_results = [(documents[0][i], distances[0][i]) for i in range(len(documents[0]))]

    return combined_results

In [ ]:
def rerank_results(query_text: str, top_n: int = 5) -> List[str]:
    """
    Realiza un re-ranking entre los resultados obtenidos por BM25 y la búsqueda semántica.
    Escala los puntajes de BM25 al rango 0-2 para que coincidan con las distancias de ChromaDB.

    Args:
        query_text (str): La consulta de búsqueda.
        top_n (int): Número de resultados principales a devolver (por defecto 5).

    Returns:
        List[str]: Los documentos re-rankeados.
    """
    # Obtener los resultados de BM25
    bm25_results = extract_bm_25(query_text, top_n=top_n)

    # Obtener los resultados de la búsqueda semántica (ChromaDB)
    semantic_results = semantic_search(query_text, top_n=top_n)

    # Obtener los puntajes de BM25 para normalizarlos al rango 0-2
    bm25_scores = [bm25_score for _, bm25_score in bm25_results]
    min_bm25_score = min(bm25_scores)
    max_bm25_score = max(bm25_scores)

    # Crear un diccionario para combinar los puntajes
    combined_results = {}

    # Añadir los resultados de BM25 al diccionario, escalados al rango 0-2
    for doc, bm25_score in bm25_results:
        # Normalización Min-Max al rango 0-2
        if max_bm25_score != min_bm25_score:
            scaled_bm25_score = 2 * (bm25_score - min_bm25_score) / (max_bm25_score - min_bm25_score)
        else:
            scaled_bm25_score = 1  # Si todos los puntajes de BM25 son iguales, asignar un valor arbitrario (1 en este caso)

        combined_results[doc] = {'bm25': scaled_bm25_score, 'semantic': 0}

    # Añadir los resultados de la búsqueda semántica al diccionario
    for doc, semantic_distance in semantic_results:
        semantic_score = 2 - (semantic_distance)  # Convertir distancia a similitud (0 -> 2, 2 -> 0)
        if doc not in combined_results:
            combined_results[doc] = {'bm25': 0, 'semantic': semantic_score}
        else:
            combined_results[doc]['semantic'] = semantic_score

    # Crear una lista con los documentos y sus puntajes combinados
    reranked_docs = [
        (doc, combined_results[doc]['bm25'] + combined_results[doc]['semantic'])
        for doc in combined_results
    ]

    # Ordenar los resultados por el puntaje combinado
    reranked_docs_sorted = sorted(reranked_docs, key=lambda x: x[1], reverse=True)

    # Devolver los documentos re-rankeados
    return reranked_docs_sorted[:top_n]

In [ ]:
semantic_search('Qué jugador empieza el juego?')


[('El jugador que haya sacado el valor más bajo en los dados se convierte en el jugador principal. En caso de empate, el último jugador que comió comida india. El jugador inicial colocará el elefante junto a su estatua de Kali y pondrá su marcador de riqueza en el espacio 3 en el track de riqueza. El jugador a su izquierda 4, el siguiente 5, y así sucesivamente. El jugador que vaya último comenzará',
  1.1987712383270264),
 ('Tan pronto como un jugador logre que su marcador de fama y su marcador de riqueza se equiparen en valores o se crucen, pasándose el uno al otro, se dispara el final de la partida. La ronda actual se terminará. Es decir, todos los jugadores que estén entre el jugador que ha activado el final de la partida y el jugador inicial actual todavía tendrán la oportunidad de ganar y podrá colocar un',
  1.297184705734253),
 ('Entonces, sabes que lo recuperas, sí, es muy divertido, me gusta cómo se ve y sabes que se juega bastante, bastante, bastante bien, sigue dando vuelta

In [ ]:
extract_bm_25('Qué jugador empieza el juego?')


[('Excepto en el caso en el que quieras apresurarte para llevar a tu trabajador al puente, no puedo entender por qué es útil usar un 6 en los portugueses. Cuanto antes atravieses el río, menos probabilidades tendrás de beneficiarte de esas bonificaciones de espacio de 1 río y reclamar recompensas causadas por otros eventos. ¿De qué otra manera podrían ser útiles los portugueses?',
  6.634060680443012),
 ('El jugador que haya sacado el valor más bajo en los dados se convierte en el jugador principal. En caso de empate, el último jugador que comió comida india. El jugador inicial colocará el elefante junto a su estatua de Kali y pondrá su marcador de riqueza en el espacio 3 en el track de riqueza. El jugador a su izquierda 4, el siguiente 5, y así sucesivamente. El jugador que vaya último comenzará',
  4.8548482211526185),
 ('Solo he jugado el juego varias veces, pero creo que la mejor estrategia (y probablemente la más obvia) es construir tus mercados en tu provincia lo más rápido posib

In [ ]:
resultados_reranked = rerank_results('Qué jugador empieza el juego?', top_n=5)


In [ ]:
resultados_reranked


[('Excepto en el caso en el que quieras apresurarte para llevar a tu trabajador al puente, no puedo entender por qué es útil usar un 6 en los portugueses. Cuanto antes atravieses el río, menos probabilidades tendrás de beneficiarte de esas bonificaciones de espacio de 1 río y reclamar recompensas causadas por otros eventos. ¿De qué otra manera podrían ser útiles los portugueses?',
  2.0),
 ('El jugador que haya sacado el valor más bajo en los dados se convierte en el jugador principal. En caso de empate, el último jugador que comió comida india. El jugador inicial colocará el elefante junto a su estatua de Kali y pondrá su marcador de riqueza en el espacio 3 en el track de riqueza. El jugador a su izquierda 4, el siguiente 5, y así sucesivamente. El jugador que vaya último comenzará',
  1.3932647275077705),
 ('Tan pronto como un jugador logre que su marcador de fama y su marcador de riqueza se equiparen en valores o se crucen, pasándose el uno al otro, se dispara el final de la partida

**Consulta dinámica para la base de dato tabular**

In [ ]:
def get_tabular_query(input_text: str) -> str:
    """
    Genera una consulta SPARQL utilizando Qwen a partir de la entrada del usuario.
    """
    # Tu clave API de Hugging Face
    api_key = config('HUGGINGFACE_TOKEN', userdata.get('HUGGINGFACE_TOKEN'))
    client_HF = InferenceClient(api_key= api_key)
    # URL de la API de Hugging Face para la clasificación de texto
    api_url = "https://api-inference.huggingface.co/models/Qwen/Qwen2.5-72B-Instruct"

    chat_prompt = [{
        "role": "system",
        "content": f"""
        Eres un modelo que realiza consultas a una base de datos tabular en dataframe de pandas sobre el juego de mesa "Rajas of the Ganjes" a raíz de una frase del usuario.
        La base tiene la siguiente estructura: las columnas son: {df_valores.columns}, donde los valores son:
        Entidad: {df_valores['Entidad']}.
        Avg.rating contiene el puntaje medio del juego.
        Debes escribir SOLAMENTE UNA consulta de búsqueda en dataframe que se adecue a la frase del usuario.
        Es importante que solo respondas con una sola consulta.
        Por ejemplo, si recibes la frase: '¿Cuántos jugadores han jugado el juego?' deberías solamente
        devolver: 'df_valores[df_valores['Entidad'] == 'All Time Plays']['Valor'].values[0]'"""
    }, {
        "role": "user",
        "content": input_text
    }]

    # Realizar la solicitud de completado de chat usando el cliente
    completion = client_HF.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=chat_prompt,
        max_tokens=500,
        temperature=0.1
    )

    # Extraer y devolver la consulta SPARQL generada
    return completion.choices[0].message['content']

In [ ]:
eval(get_tabular_query('¿cuántas comentarios tiene el juego?'))


'2,151'

In [ ]:
query = get_tabular_query('¿Cuántos jugadores han jugado el juego?')

In [ ]:
eval(query)

'60,308'

# Parte 6: Armado LLM para usuario

In [ ]:
class llm_rajas_of_the_ganjes():
    """
    Esta clase define un chatbot interactivo diseñado para responder consultas sobre el juego *Rajas of The Ganges*
    utilizando un modelo de lenguaje y diferentes fuentes de datos.

    La clase permite realizar consultas, obtener contextos de diversas bases de datos (como bases de datos vectoriales,
    gráficas o tabulares), y generar un prompt para interactuar con un modelo de lenguaje.

    Métodos:
    - __init__: Inicializa el cliente de inferencia y define los atributos necesarios para el chatbot.
    - get_context: Obtiene el contexto necesario para responder a una consulta, basándose en el tipo de base de datos.
    - make_prompt: Crea un prompt estructurado para la interacción con el modelo de lenguaje.
    - hablar: Inicia una conversación con el chatbot, permite al usuario hacer consultas y recibir respuestas del modelo.
    """

    def __init__(self):
        """
        Inicializa el cliente de inferencia (InferenceClient) con una clave de API y establece los atributos
        iniciales para el chatbot.
        """
        api_key = config('HUGGINGFACE_TOKEN', userdata.get('HUGGINGFACE_TOKEN'))
        self.client_rotg = InferenceClient(api_key=api_key)
        self.info = None
        self.chat_prompt = None

    def get_context(self, consulta):
        """
        Obtiene el contexto adecuado para una consulta dada, determinando el tipo de base de datos (vectorial, gráfica o tabular).

        Dependiendo de la base de datos asignada, la función consulta y recupera los datos necesarios
        para proporcionar una respuesta relevante.

        Parámetros:
        - consulta (str): La consulta del usuario para la que se necesita obtener el contexto.

        Retorna:
        - self.info (List or None): La información contextual relevante obtenida de la base de datos.
        """
        classification = get_class(consulta)
        db = get_database(classification)
        if db == 'vector_db':
            self.info = rerank_results(consulta)
        elif db == 'graph_db':
            query = get_query(consulta)
            self.info = [result.name[0:] for result in g.query(query)]
        elif db == 'tabular_db':
            query = get_tabular_query(consulta)
            self.info = [eval(query)]
        else:
            self.info = None

        return self.info

    def make_prompt(self, consulta):
        """
        Crea un prompt estructurado para interactuar con el modelo de lenguaje, utilizando el contexto
        obtenido de la base de datos y la consulta del usuario.

        Parámetros:
        - consulta (str): La consulta del usuario para la que se necesita generar el prompt.

        Retorna:
        - chat_prompt (List[Dict]): Una lista de diccionarios que contiene el prompt estructurado con
          el rol de sistema y usuario para la interacción con el modelo.
        """
        chat_prompt = [{
            "role": "system",
            "content": f"""
                Eres un modelo que responde preguntas acerca del juego *Rajas of The Ganges*.
                Es importante que sólo respondas en base a la siguiente información: {self.info}.
                Es muy importante que contestes en oración. Si la información proporcionada es un número, responde con ese número como parte de la oración.
                Sólo si la información es "None", responde 'No tengo información para responder la consulta'."""
        }, {
            "role": "user",
            "content": consulta
        }]
        return chat_prompt

    def hablar(self):
        """
        Inicia una conversación interactiva con el usuario, permitiendo realizar consultas y recibir respuestas
        generadas por el modelo de lenguaje. El ciclo de conversación se mantiene hasta que el usuario escribe
        'chau' para salir.

        El ciclo de interacción involucra la obtención de contexto, la creación del prompt y la generación de respuestas
        del modelo, que se imprimen como respuestas del agente.
        """
        consulta = 'c'
        print('Bienvenido a Rajas of The Ganjes Chatbot! para salir escribe "chau"')
        while True:
            consulta = input("Escribe tu consulta, para salir despídete con 'chau'':")
            if consulta == 'Chau' or consulta == 'chau':
              print('Gracias por usar Rajas of The Ganjes Chatbot, hasta la próxima!')
              break
            print(f"User: {consulta}")
            self.info = self.get_context(consulta)
            print(self.info)
            self.chat_prompt = self.make_prompt(consulta)
            # Realizar la solicitud de completado de chat usando el cliente
            completion = self.client_rotg.chat.completions.create(
                model="Qwen/Qwen2.5-72B-Instruct",
                messages=self.chat_prompt,
                max_tokens=500,
                temperature=0.5
            )
            resp = completion.choices[0].message['content']
            if detect(resp) != detect(consulta):
              resp = GoogleTranslator(source=detect(resp), target=detect(consulta)).translate(resp)
            print(f"Agente: {resp}")

In [ ]:
llm_rotg = llm_rajas_of_the_ganjes()


In [ ]:
llm_rotg.hablar()


Bienvenido a Rajas of The Ganjes Chatbot! para salir escribe "chau"
Escribe tu consulta, para salir despídete con 'chau'':quién son los creadores
User: quién son los creadores
['Inka_Brand', 'Markus_Brand']
Agente: Los creadores de Rajas del Ganges son Inka Brand y Markus Brand.
Escribe tu consulta, para salir despídete con 'chau'':chau
Gracias por usar Rajas of The Ganjes Chatbot, hasta la próxima!


#Parte 7: Agente

In [ ]:
# Descarga de Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Iniciamos Ollama en background
!rm -f ollama_start.sh
!echo '#!/bin/bash' > ollama_start.sh
!echo 'ollama serve' >> ollama_start.sh
# Make the script executable
!chmod +x ollama_start.sh
!nohup ./ollama_start.sh &

!ollama pull llama3.2 > ollama.log

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏ 759 KB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   2% ▕▏  34 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   4% ▕▏  71 MB/2.0 

In [ ]:
!ollama list


NAME               ID              SIZE      MODIFIED      
llama3.2:latest    a80c4f17acd5    2.0 GB    4 seconds ago    


In [ ]:
import logging

# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
def search_wikipedia(query: str, lang: str = 'es') -> str:
    query = f"{query} para el juego Rajas of the Ganjes"
    logger.info(f"Llamando a Wikipedia con query: {query}, lang: {lang}")
    try:
        wikipedia.set_lang(lang)
        result = wikipedia.summary(query, sentences=3)
        logger.info(f"Resultado Wikipedia encontrado: {result[:100]}...")
        return result
    except Exception as e:
        logger.error(f"Error en búsqueda de Wikipedia: {str(e)}")
        return "No se encontró información"


def get_info_graph_db(consulta):
  query = get_query(consulta)
  info = [result.name[0:] for result in g.query(query)]
  return info


def get_info_tabular_db(consulta):
  query = get_tabular_query(consulta)

  info = [eval(query), query]
  return info


def get_info_vector_db(consulta):
  info = rerank_results(consulta)
  return info


In [ ]:
# Crear las herramientas para el agente
tools_list = [
    FunctionTool.from_defaults(fn=get_info_graph_db, description="Busca información en la base de datos de grafos. Usar: texto de consulta"),
    FunctionTool.from_defaults(fn=get_info_tabular_db, description="Busca información en la base de datos tabular. Usar: texto de consulta"),
    FunctionTool.from_defaults(fn=get_info_vector_db, description="Busca información en la base de datos vectorial. Usar: texto de consulta"),
    #FunctionTool.from_defaults(fn=get_class, description="Busca temáticas asociadas a la consulta. Usar: texto de consulta"),

]

In [ ]:
# Configurar el LLM de Ollama para usar Llama 3.2
llm = Ollama(
    model="llama3.2:latest",
    request_timeout=100.0,  # Aumentado el timeout
    temperature=0.1,  # Reducida la temperatura para respuestas más deterministas
    context_window=4096  # Aumentado el contexto
)


Settings.llm = llm
import time
!nohup litellm --model ollama/llama3.2:latest --port 8000 > litellm.log 2>&1 &

In [ ]:
agent = ReActAgent.from_tools(
    tools_list,
    llm=llm,
    verbose=True,
    chat_formatter=ReActChatFormatter(),
    system_prompt="""Estás diseñado para responder preguntas acerca del juego 'Rajas of the Ganges' utilizando **exclusivamente información de las bases de datos tabular, vectorial y de grafos, brindada por las herramientas que se mencionana a continuación***. Asegúrate de seguir estrictamente las instrucciones para cada consulta.

          ## Herramientas disponibles:
          1. **get_info_graph_db**: Busca información en la base de datos de grafos. Parámetro: texto de consulta.
          2. **get_info_tabular_db**: Busca información en la base de datos tabular. Parámetro: texto de consulta.
          3. **get_info_vector_db**: Busca información en la base de datos vectorial. Parámetro: texto de consulta.

          ### Temáticas de cada base:
          - **Grafos**: DISEÑADORES, CREADORES, IDIOMAS DEL JUEGO, PUBLICADORES DEL JUEGO, AÑO DE CREACIÓN DEL JUEGO, OTROS IDIOMAS.
          - **Tabular**: ESTADÍSTICA GENERAL.
          - **Vectorial**: REGLAS GENERALES, REGLAS ESPECÍFICAS AUTOMATA, REGLAS ESPECÍFICAS NAVARATNA, OBJETIVO DEL JUEGO, ESTRATEGIA COMPETIDORES, ESTRATEGIA PRINCIPIANTES, ESTRATEGIA PERSONAJE PORTUGUESE.

          ### Instrucciones para cada consulta:
          1. **Analiza la consulta** para determinar la información necesaria.
          2. **Usa las herramientas disponibles una por una**, pasando como parámetro **exactamente** la consulta recibida.
          3. **Combina los resultados** en una respuesta final clara y concisa.

          ### Reglas adicionales:
          - **Nunca modifiques la consulta del usuario**: Cada vez que recibas una consulta nueva, **olvida la anterior**. La nueva consulta debe ser tratada de manera independiente y sin influencias previas.
          - **Prioriza las herramientas** según la relación temática con la consulta. Asegúrate de seleccionar la herramienta que mejor se ajuste a la consulta de acuerdo con las temáticas proporcionadas.
          - Si la pregunta requiere información de **más de una herramienta**, usa **todas** las herramientas necesarias para obtener una respuesta completa.
          - **Nunca** respondas con información que no provenga directamente de las herramientas.
          - El formato de salida debe ser **estricto** y seguir el ejemplo que se proporciona.

          ### Ejemplo de interacción:
          #### Consulta 1:
          "¿Cuándo finaliza la partida?"
          - **Pensamiento (Thought)**: Necesito buscar información sobre las reglas del juego.
          - **Acción (Action)**: `get_info_vector_db`
          - **Entrada de acción (Action Input)**: "¿Cuándo finaliza la partida?"
          - **Observación (Observation)**: "La partida finaliza cuando un jugador logra que sus marcadores de fama y riqueza se equiparen..."
          - **Respuesta final (Final Answer)**: La partida finaliza cuando un jugador logra que sus marcadores de fama y riqueza se equiparen, lo que dispara el final de la partida.

          #### Consulta 2:
          "Año de creación del juego?"
          - **Pensamiento (Thought)**: Necesito buscar información sobre el año de creación del juego.
          - **Acción (Action)**: `get_info_graph_db`
          - **Entrada de acción (Action Input)**: "Año de creación del juego"
          - **Observación (Observation)**: "El juego fue creado en 2017."
          - **Respuesta final (Final Answer)**: El juego fue creado en 2017.

          ### IMPORTANTE:
          - **No uses información de consultas anteriores** para la nueva consulta.
          - **Nunca alteres la consulta recibida**. Siempre debes utilizar el texto exacto como parámetro.
          - **Siempre asegúrate de usar todas las herramientas necesarias** antes de no dar una respuesta completa.
          """
          ,
                react_chat_history=False,
              context="""
              Eres un asistente útil que responde en español. Recuerda las salidas de todas las herramientas para usarlas en la respuesta final.
              """)


# Función para interactuar con el agente
def chat_con_agente():
    """
    Función para interactuar con el agente ReAct.
    """
    query = input("")
    while query != 'chau':
      try:
          if not query.strip():
              return "La consulta está vacía"

          response = agent.chat(query)
          print(response)
      except Exception as e:
          print(f"Error al procesar la consulta: {str(e)}")
      query = input("")

* "Necesito una estrategia para principiantes",
* "Año de creación del juego?",
* "¿Quién feu el creador del juego?",
* "Con qué nombres se conoce el juego?",
* "De qué nacionalidad son sus creadores?",
* "Con qué religión se asocia el juego?",
* "Qué puntaje obtuvo el juego?",
* "¿Cuántos jugadores en el mes tuvo?"

In [ ]:
chat_con_agente()

Necesito una estrategia para principiantes
> Running step f52b77a1-2d54-4774-beac-41ac6eefadea. Step input: Necesito una estrategia para principiantes
Error al procesar la consulta: timed out
chau
